# TP Introducción a la Inteligencia Artificial

In [ ]:
import matplotlib.pyplot as plt
import random
import pickle
import os
import json
import logging
from datetime import datetime, timedelta
from itertools import combinations, product

def _leerArchConf(tipo, archivo):
    print("#algoritmo('{}','{}')".format(tipo,archivo))
    arch = open(archivo)
    if (tipo == "TSPLIB"):
        # Las primeras 3 lineas del archivo se obvian
        for a in range(3):
            arch.readline()
        # Se lee la dimension del problema
        try:
            cantCiu = int(arch.readline().split("DIMENSION: ")[1])
        except IndexError:
            cantCiu = int(arch.readline().split("DIMENSION : ")[1])
        # Las siguientes 2 lineas del archivo se obvian
        for a in range(2):
            arch.readline()

        vec_ciudades = []
        vec_ciudades_map = []

        line = arch.readline()
        aux = [a for a in line.split(' ') if a != '' and a != '\n']
        while (line != 'EOF\n'):
            try:
                vec_ciudades.append([int(a) for a in aux])
            except:
                vec_ciudades.append([float(a) for a in aux])
            line = arch.readline()
            aux = [a for a in line.split(' ') if a != '' and a != '\n']

        # Se arman las tuplas de ciudades [(nro1, [x1, y1]),(nro2, [x2, y2]),...]
        vec_ciudades_map.append(map(lambda e: (e[0], e[1:]), vec_ciudades))

        # Se arma la lista de las tuplas de ciudades
        vec_ciudades_list = [list(a) for a in vec_ciudades_map][0]

        vec_costCiu = []
        aux = [a for a in vec_ciudades_list]
        while (len(aux) > 0):
            ciudad_actual = aux.pop(0)
            for b in aux:
                costo_x = (ciudad_actual[1][0] - b[1][0])**2
                costo_y = (ciudad_actual[1][1] - b[1][1])**2
                costo = (costo_x+costo_y)**(1/2)
                vec_costCiu.append(costo)
    elif (tipo == "TSP_IN"):
        cantCiu = int(arch.readline().split(';')[0])
        vec_costCiu = [int(a) for a in arch.readline().split(';') if a != '\n']
        vecmat_costCiu = []
    else:
        return -1

    arch.close()

    vecmat_costCiu = []
    vec_costCiu_aux = [a for a in vec_costCiu]
    for a in range(cantCiu):
        
        #TODO: optimizar -> c no se usa y se puede hacer mejor un append con ceros
        # Se inicializa en ceros el vector de costos de las ciudades que se utilizara
        # para llenar la matriz de costos
        for c in range(a+1):
            vecmat_costCiu.append(0)
        
        # se llena el triangulo superior de la matriz
        for b in range(cantCiu-a-1):
            vecmat_costCiu.append(vec_costCiu_aux[0])
            vec_costCiu_aux = vec_costCiu_aux[1:]

    # se traspone el triangulo superior y se almacenan sus valores
    for a in range(cantCiu):
        for b in range(cantCiu):
            if b < a:
                # si la columna es menor a la fila, se copia el valor de su opuesto
                vecmat_costCiu[cantCiu*a+b] = vecmat_costCiu[cantCiu*b+a]

    # se termina de componer la matriz
    mat_costCiu = []
    for a in range(cantCiu):
        mat_costCiu.append(vecmat_costCiu[:cantCiu])
        vecmat_costCiu = vecmat_costCiu[cantCiu:]

    # se compone el objeto (dict) para agilizar el calculo de costos (costo de N a M).
    obj_costCiu = dict()
    for a in range(1,cantCiu+1):
        indice_1 = "ciudad{}".format(a)
        obj_costCiu[indice_1] = dict()
        for b in range(1,cantCiu+1):
            if (a != b):
                indice_2 = "ciudad{}".format(b)
                obj_costCiu[indice_1][indice_2] = mat_costCiu[a-1][b-1]

    return cantCiu, vec_costCiu, mat_costCiu, obj_costCiu

#cantCiu, vec_costCiu, mat_costCiu, obj_costCiu = _leerArchConf('TSP_IN','TSP/AGD_TSP_IN_04.txt') #[3, 4, 1, 2, 3] con costo: 7
#cantCiu, vec_costCiu, mat_costCiu, obj_costCiu = _leerArchConf('TSP/AGD_TSP_IN_05.txt') #[5, 4, 1, 2, 3, 5] ó  [2, 4, 5, 1, 3, 2] ambos con costo: 10
#cantCiu, vec_costCiu, mat_costCiu, obj_costCiu = _leerArchConf('TSP/AGD_TSP_IN_06.txt') #[4, 5, 1, 6, 3, 2, 4] con costo: 11
#cantCiu, vec_costCiu, mat_costCiu, obj_costCiu = _leerArchConf('TSP/AGD_TSP_IN_07.txt') #[1, 3, 5, 7, 6, 4, 2, 1] con costo: 25
#cantCiu, vec_costCiu, mat_costCiu, obj_costCiu = _leerArchConf('TSP/AGD_TSP_IN_08.txt') #tiene un doble lazo...
#cantCiu, vec_costCiu, mat_costCiu, obj_costCiu = _leerArchConf('TSP_IN','TSP/AGD_TSP_IN_10.txt')
#print(mat_costCiu)

## Regla 1
> Se seleccionará, como caminos candidatos a formar el recorrido óptimo, a los "dos" de menor costo (de los N-1) que acceden a cada nodo.<br>
Se supone que seleccionando estos caminos se podrá entrar y salir de cada nodo por los accesos más cortos.<br>
De esta manera, se reduce a 2.N el número de caminos a considerar.<br>
De los 2.N caminos seleccionados, se eliminan aquellos que estan repetidos.<br>
Usando la propiedad Cij = Cji

Para AGD TSP 06<br>
R1 = [[1, 3], [1, 5], [2, 4], [2, 3], [4, 5], [2, 5], [1, 6], [2, 6]]

In [ ]:
# se hallan los 2 menores costos de cada ciudad

# se armara una matriz de 2 columnas por n filas
# [[origen, destino], [origen2, destino2], ... ]
def regla1(cantCiu, mat_costCiu):
    mat_regla1 = []
    mat_cs = []
    for y in range(cantCiu):
        fila = mat_costCiu[y].copy()
        fila.remove(0) # se elimina el costo 0 de la fila correspondiente
        for a in range(2):
            minvalor = min(fila)
            maxvalor = max(fila)
            minindice = fila.index(minvalor)
            fila[minindice]=maxvalor+1
            if (minindice >= y):
                minindice += 2
            else:
                minindice += 1
            # calculo [origen, destino]
            origen = y+1
            destino = minindice
            # ordeno por numero de ciudad ascendente
            if (origen > destino):
                aux = origen
                origen = destino
                destino = aux
            if (depurando == True):
                print("C{}{}: {}".format(origen, destino, minvalor))
            # almaceno el resultado parcial (sin quitar duplicados)
            mat_regla1.append([origen,destino])
            # almaceno el resultado final (quitando duplicados)
            if ([origen, destino] not in mat_cs):
                mat_cs.append([origen,destino])
    return mat_cs

#mat_cs = regla1(cantCiu, mat_costCiu)
#print("mat_cs: {}".format(mat_cs))

## Regla 2
> Computar EAi para todos los nodos<br>
> Para el cómputo del número de arcos (NAi) que concurren en un nodo, basta con contar las repeticiones de un mismo subíndice, en los costos de los caminos seleccionados por R1 (ya sea en la posición i como en la j).<br>

In [ ]:
def calcularExcesoArcos(cantCiu, mat_cs):
    vec_excesoArcos = []
    for a in range(cantCiu):
        vec_excesoArcos.append(-2)
    for [origen,destino] in mat_cs:
        vec_excesoArcos[origen-1] += 1
        vec_excesoArcos[destino-1] += 1
    return vec_excesoArcos

def calcularEc(vec_excesoArcos):
    ec = 0
    for a in vec_excesoArcos:
        ec += a
    ec = int(ec/2)
    return ec

#vec_excesoArcos = calcularExcesoArcos(cantCiu, mat_cs)
#print("vec_excesoArcos: {}".format(vec_excesoArcos))

#ec = calcularEc(vec_excesoArcos)
#print("ec: {}".format(ec))


## Regla 3
> Ordenar de mayor a menor la lista Caminos Seleccionados (CS) por la Regla 1<br>
> Ordenar de menor a mayor los subíndices de los CS de forma que para todo Cij i<j

In [ ]:
def ordenarCostosCS(mat_costCiu, mat_cs, reverse=True):
    vec_costos = []
    for [origen, destino] in mat_cs:
        vec_costos.append(mat_costCiu[origen-1][destino-1])

    # Se procede a ordenar Caminos Seleccionados ó CS (mat_cs) de mayor a menor en función a los costos (vec_costos)
    vec_costos,mat_cs_ordenado = zip(*sorted(zip(vec_costos,mat_cs), reverse=reverse))
        #zip genera un array (lista) de tuplas con, [(Fn[0],Gn[0],Hn[0],listAbierta[0]),(Fn[1],Gn[1],Hn[1],listAbierta[1]),(...)]
        #sorted(zip) ordena dicho array en orden ascendente
        #zip(*) hace un unpack del primer zip, es decir, deshace el primer zip y desglosa la salida en tuplas.
    # se arreglan las tuplas anteriores para que resulten arrays (listas)
    vec_costos,mat_cs_ordenado = list(vec_costos),list(mat_cs_ordenado)
    
    return vec_costos, mat_cs_ordenado

#vec_costos,mat_cs = ordenarCostosCS(mat_costCiu, mat_cs, reverse=True)
#print("desp ordenarCostosCS: vec_costos: {}, mat_cs_ordenado:{}".format(vec_costos,mat_cs))

## Regla 4
> Explorar la lista CS de mayor a menor y remover los caminos Cij de las lista CS que cumplan con EAi>0 y EAj>0.<br>
> Actualizar EAi=EAi-1 y EAj=EAj-1, EC=EC-1 después de cada eliminación.<br>
> Si EC=0 Terminar (verificar la existencia de más de un lazo).<br>

In [ ]:
# se remueven los caminos Cij de la lista CS que cumplan con EAi>0 y EAj>0
# actualizando EAi, EAj y EC

# TODO: se puede hacer, analizando el costo, que por cada par EAi>0 y EAj>0
# se desglose una rama de posibilidades para que el resultado final no dependa
# del orden en el cual se va eliminando los caminos Cij

# ====================== REGLA 4 ===========================
def removerCaminosCS(ec, vec_excesoArcos, mat_cs_ordenado):
    mat_cs_removido = mat_cs_ordenado.copy()
    for i,j in mat_cs_ordenado:
        if (depurando == True):
            print("{} {}, vec_excesoArcos[i-1]: {}, vec_excesoArcos[j-1]: {}".format(i, j, vec_excesoArcos[i-1], vec_excesoArcos[j-1]))
        if (vec_excesoArcos[i-1] > 0 and vec_excesoArcos[j-1] > 0):
            if (depurando == True):
                print("exceso > 0: {} {}, vec_excesoArcos[i-1]: {}, vec_excesoArcos[j-1]: {}".format(i, j, vec_excesoArcos[i-1], vec_excesoArcos[j-1]))
            mat_cs_removido.remove([i,j])
            vec_excesoArcos[i-1] -= 1
            vec_excesoArcos[j-1] -= 1
            ec -= 1

    return ec, vec_excesoArcos, mat_cs_removido

def calcularRecorridoFinal(cantCiu, mat_cs_ordenado, mat_costCiu):
    if (depurando == True):
        print("calcRecFinal: mat_cs_ordenado: {}".format(mat_cs_ordenado))
    origenes = []
    destinos = []
    costo = 0
    for a in mat_cs_ordenado:
        origenes.append(a[0])
        destinos.append(a[1])
        costo += mat_costCiu[a[0]-1][a[1]-1]
    if (depurando == True):
        print("calcRecFinal: origenes: {}".format(origenes))
        print("calcRecFinal: destinos: {}".format(destinos))

    p = 0
    dest = 0
    orig = 0
    recorrido_final = []
    proximo = 0
    for a in range(cantCiu):
        if (a == 0):
            orig = origenes[p]
            dest = destinos[p]
            recorrido_final.append(orig)
            recorrido_final.append(dest)
            destinos.remove(dest)
            origenes.remove(orig)
            proximo = dest
            if (depurando == True):
                print("calcRecFinal: recorrido_final: {}".format(recorrido_final))
                print("calcRecFinal: origenes: {}".format(origenes))
                print("calcRecFinal: destinos: {}".format(destinos))
        else:
            try:
                try:
                    p = destinos.index(proximo)
                    orig = origenes[p]
                    dest = destinos[p]
                    destinos.pop(p)
                    origenes.pop(p)
                except ValueError:
                    p = origenes.index(proximo) 
                    dest = origenes[p]
                    orig = destinos[p]
                    destinos.pop(p)
                    origenes.pop(p)
                if (proximo != orig):
                    proximo = orig
                else:
                    proximo = dest
                recorrido_final.append(proximo)
            except ValueError:
                if (recorrido_final[0] == proximo):
                    # Se tiene mas de un lazo
                    print("Se tiene mas de un lazo")
                    # lo que podria hacer es hacer un nuevo lazo llamando nuevamente al
                    # recorrido_final con el resto de caminos. Luego debere calcular el
                    # nodo donde conviene unir los lazos
                    print("calcRecFinal: origenes: {}".format(origenes))
                    print("calcRecFinal: destinos: {}".format(destinos))
                    return recorrido_final, costo
            if (depurando == True):
                print("calcRecFinal: recorrido_final: {}".format(recorrido_final))
                print("calcRecFinal: origenes: {}".format(origenes))
                print("calcRecFinal: destinos: {}".format(destinos))
    return recorrido_final, costo


#ec, vec_excesoArcos, mat_cs = removerCaminosCS(ec, vec_excesoArcos, mat_cs)
#vec_costos,mat_cs = ordenarCostosCS(mat_costCiu, mat_cs, reverse=True)
#print("desp removerCaminosCS: vec_costos: {}, mat_cs_ordenado: {}, vec_excesoArcos: {}".format(vec_costos,mat_cs,vec_excesoArcos))

#if (ec == 0):
#    recorrido_final, costo = calcularRecorridoFinal(cantCiu, mat_cs, mat_costCiu)
#    print("Terminó, recorrido final es: {} con costo: {}".format(recorrido_final, costo))


In [ ]:
## conceptualmente, la regla 5 la calculo como:
'''
obtener los nodos tal que EAi > 1 y EAi > 0
Por cada nodo se evalua su exceso de arcos
	si es mayor a 1
		hallar caminos que pasen por el nodo i
		tomar dichos caminos de a pares, por cada par evaluar:
			puentes_elegidos = []
			si el puente resultante del par:
				1. existe en la lista CS => descartar
				2. es un puente ya tomado => descartar (esta opción puede nunca ser verdadera)
				3. es un camino Cii => descartar (esta opcion nunca sera verdadera)
			sino:
				hallar su ahorro tal que Aii = Cij + Cik - Cjk
				agregar dicho puente a la lista de puentes con ahorro calculado
	si es mayor a 0
		tomar otro nodo que tenga EAj > 0, en caso de encontrarlo:
			hallar caminos que pasen por el nodo i y el nodo j
			tomar dichos caminos de a pares, por cada par evaluar:
				puentes_elegidos = []
				si el puente resultante del par:
					1. existe en la lista CS => descartar
					2. es un puente ya tomado => descartar
					3. es un camino Cii => descartar
				sino:
					hallar su ahorro tal que Aii = Cij + Cik - Cjk
					agregar dicho puente a la lista de puentes con ahorro calculado (para no tener que calcular el ahorro del mismo puente)
'''
def calcularListaA(cantCiu, mat_costCiu, ec, vec_excesoArcos, mat_cs_ordenado):
    vec_a_ahorro = []
    vec_a_puente = []
    if (ec == 0):
        return ec, vec_a_ahorro, vec_a_puente
    else:
        ## Armar una lista A con todos los Aii con EAi>1 y todos los Aij con EAi>0 y EAj>0
        # que tengan un camino puente que NO pertenezca a CS y el puente no es un Camino Cii.
        if (depurando == True):
            print("calc listaA: mat_cs_ordenado: {}".format(mat_cs_ordenado))
        ### AGREGAR ACA un vector que acumule los pares de ciudades que se tomaron con EAi > 0 y EAj > 0
        ### para evitar procesar por duplicado (ver línea 20 en Sublime) => luego poner un filtro al if que diga
        ### and [ij] not in mat_cs_ordenado and ji not in mat_cs_ordenado
        for i in range(1,cantCiu+1):
            # Para Aii con EAi > 1
            if (vec_excesoArcos[i-1] > 1):
                ## se deberán tomar los pares de caminos de CS tal que la ciudad 'a' figure en ambos caminos
                mat_cs_aux = mat_cs_ordenado.copy()
                mat_cs_ahorros = []
                ## buscar dentro de mat_cs_aux todos los caminos que contengan a la ciudad 'a'
                for b in mat_cs_aux:
                    if (b[0] == i or b[1] == i):
                        cs_aux = b
                        mat_cs_ahorros.append(cs_aux)
                        #print("{}: {}".format(i, cs_aux))
                if (depurando == True):
                    print("EAi > 1: mat_cs_ahorros: {}".format(mat_cs_ahorros))

                ## armar todas las combinaciones posibles de mat_cs_ahorros
                tup_cs_ahorros_combinado = combinations(mat_cs_ahorros,2)
                mat_cs_ahorros_combinado = []
                for r in tup_cs_ahorros_combinado:
                    mat_cs_ahorros_combinado.append([r[0],r[1]])
                
                # hallar los puentes correspondientes a los pares de caminos
                # seleccionados que NO pertenezcan a CS, no sea un puente ya tomado, no sea un camino Cii
                if (depurando == True):
                    print("EAi > 1: mat_cs_ahorros_combinado: {}".format(mat_cs_ahorros_combinado))
                vec_puente_aux = []
                comp_i = i
                comp_j = 0
                for o in mat_cs_ahorros_combinado:
                    #aux = [i,[0,0],0]
                    o_k = -1
                    o_l = -1
                    if(o[0][0] == comp_i):
                        o_k = o[0][1]
                    if(o[0][1] == comp_i):
                        o_k = o[0][0]
                    if(o[1][0] == comp_i):
                        o_l = o[1][1]
                    if(o[1][1] == comp_i):
                        o_l = o[1][0]
                    if (o_k == -1 or o_l == -1):
                        print("ERROR o:{}, o_k:{} o_l:{}".format(o, o_k, o_l))
                    aux = [comp_i, [o_k, o_l], comp_j]
                    if (([aux[1][1],aux[1][0]] not in mat_cs_ordenado )
                    and ([aux[1][0],aux[1][1]] not in mat_cs_ordenado )
                    and (o_k != -1)
                    and (o_l != -1)): #TODO: ver si aparece x,y en vez de y,x lo toma o no..
                        vec_puente_aux.append(aux)
                        if (depurando == True):
                            print("EAi > 1: vec_puente_aux: {}, vec_puente_aux.append({})".format(vec_puente_aux, aux))

                # calcular el ahorro de cada puente
                for e in vec_puente_aux:
                    costo_ij = mat_costCiu[e[0]-1][e[1][0]-1]
                    costo_ik = mat_costCiu[e[0]-1][e[1][1]-1]
                    costo_jk = mat_costCiu[e[1][0]-1][e[1][1]-1]
                    ahorro_ii = costo_ij + costo_ik - costo_jk
                    vec_a_ahorro.append(ahorro_ii)
                    vec_a_puente.append(e)
                    #print("vec_a_ahorro.append({})".format(ahorro_ii))
                    #print("vec_a_puente.append({})".format(e))
                if (depurando == True):
                    print("EAi > 1: vec_a_ahorro: {}".format(vec_a_ahorro))
                    print("EAi > 1: vec_a_puente: {}".format(vec_a_puente))
            
            # Para Aij con EAi>0 y EAj>0
            if (vec_excesoArcos[i-1] > 0 ):
                ciudades_aux = [r for r in range(1,cantCiu+1)]
                ciudades_aux.remove(i)
                mat_cs_ahorros_combinado = []
                for j in ciudades_aux:
                    if (depurando == True):
                        print("EAi>0 y EAj>0: i: {}, j: {}".format(i,j))
                    if (vec_excesoArcos[j-1] > 0
                        and [i,j] not in mat_cs_ordenado
                        and [j,i] not in mat_cs_ordenado):
                        
                        # hallar caminos que pasen por el nodo i y el nodo j
                        mat_cs_aux = mat_cs_ordenado.copy()
                        mat_cs_ahorros_i = []
                        mat_cs_ahorros_j = []
                        ## buscar dentro de mat_cs_aux todos los caminos que pasen por el nodo i
                        for b in mat_cs_aux:
                            if (b[0] == i or b[1] == i):
                                cs_aux = b
                                mat_cs_ahorros_i.append(cs_aux)
                            if (b[0] == j or b[1] == j):
                                cs_aux = b
                                mat_cs_ahorros_j.append(cs_aux)
                        if (depurando == True):
                            print("EAi>0 y EAj>0: mat_cs_ahorros_i: {}, mat_cs_ahorros_j: {}".format(mat_cs_ahorros_i, mat_cs_ahorros_j))

                        ## combinar elementos tomados de a 1 por cada grupo
                        tup_cs_ahorros_combinado_aux = product(mat_cs_ahorros_i, mat_cs_ahorros_j)
                        mat_cs_ahorros_combinado_aux = []
                        for r in tup_cs_ahorros_combinado_aux:
                            mat_cs_ahorros_combinado_aux.append([r[0],r[1]])
                
                        if (depurando == True):
                            print("EAi>0 y EAj>0: mat_cs_ahorro_aux: {}".format(mat_cs_ahorros_combinado_aux))

                        # hallar los puentes correspondientes a los pares de caminos
                        # seleccionados que:
                        #   NO pertenezcan a CS
                        #   no sea un puente ya tomado en la iteración,
                        #   no sea un puente ya tomado dentro de todos los calculados
                        #   no sea un camino Cii
                        vec_puente_aux = []
                        comp_i = i
                        comp_j = j
                        for o in mat_cs_ahorros_combinado_aux:
                            #aux = [comp_i,[0,0],comp_j]
                            o_k = -1
                            o_l = -1
                            if(o[0][0] == comp_i):
                                o_k = o[0][1]
                            if(o[0][1] == comp_i):
                                o_k = o[0][0]
                            if(o[1][0] == comp_j):
                                o_l = o[1][1]
                            if(o[1][1] == comp_j):
                                o_l = o[1][0]
                            if (o_k == -1 or o_l == -1):
                                print("ERROR o:{}, o_k:{} o_l:{}".format(o, o_k, o_l))
                            aux = [comp_i, [o_k, o_l], comp_j]
                            if (( [aux[1][1],aux[1][0]] not in mat_cs_ordenado )
                            and ( [aux[1][0],aux[1][1]] not in mat_cs_ordenado )
                            and ( [comp_i,[aux[1][1],aux[1][0]],comp_j] not in vec_puente_aux )
                            and ( [comp_i,[aux[1][0],aux[1][1]],comp_j] not in vec_puente_aux )
                            and ( [comp_j,[aux[1][0],aux[1][1]],comp_i] not in vec_puente_aux )
                            and ( [comp_j,[aux[1][1],aux[1][0]],comp_i] not in vec_puente_aux )  
                            and ( [comp_i,[aux[1][1],aux[1][0]],comp_j] not in vec_a_puente )
                            and ( [comp_i,[aux[1][0],aux[1][1]],comp_j] not in vec_a_puente ) 
                            and ( [comp_j,[aux[1][0],aux[1][1]],comp_i] not in vec_a_puente )
                            and ( [comp_j,[aux[1][1],aux[1][0]],comp_i] not in vec_a_puente ) 
                            and ( aux[1][0] != aux[1][1])
                            and (o_k != -1)
                            and (o_l != -1)):
                                vec_puente_aux.append(aux)
                                if (depurando == True):
                                    print("EAi>0 y EAj>0: vec_puente_aux: {}, vec_puente_aux.append({})".format(vec_puente_aux, aux))

                        # calcular el ahorro de cada puente
                        for e in vec_puente_aux:
                            costo_ik = mat_costCiu[e[0]-1][e[1][0]-1]
                            costo_jl = mat_costCiu[e[2]-1][e[1][1]-1]
                            costo_kl = mat_costCiu[e[1][0]-1][e[1][1]-1]
                            ahorro_ij = costo_ik + costo_jl - costo_kl
                            vec_a_ahorro.append(ahorro_ij)
                            vec_a_puente.append(e)
                        if (depurando == True):
                            print("EAi>0 y EAj>0: vec_a_ahorro: {}".format(vec_a_ahorro))
                            print("EAi>0 y EAj>0: vec_a_puente: {}".format(vec_a_puente))
        
        # Se procede a ordenar la lista A de mayor a menor en función a los ahorros (vec_a_ahorro)
        vec_a_ahorro,vec_a_puente = zip(*sorted(zip(vec_a_ahorro,vec_a_puente), reverse=True))
        vec_a_ahorro,vec_a_puente = list(vec_a_ahorro),list(vec_a_puente)
        ec = calcularEc(vec_excesoArcos)

    return ec, vec_a_ahorro, vec_a_puente

# ====================== REGLA 5 y 6 ===========================
#ec, vec_a_ahorro, vec_a_puente = calcularListaA(cantCiu, mat_costCiu, ec, vec_excesoArcos, mat_cs)
#print("desp calcularListaA: ec: {}, vec_a_ahorro: {}, vec_a_puente{}".format(ec, vec_a_ahorro, vec_a_puente))


## Regla 5
> Si EC > 0
> Armar una lista A con todos los Aii con EAi>1 y todos los Aij con EAi>0 y EAj>0 que tengan un camino puente que NO pertenezca a CS y el puente no es un Camino Cii.

## Regla 6
> Ordenar de mayor a menor la lista A


## Regla 7
> Remover el primer elemento de la lista A el Aij mayor<br>
Aplicar el ahorro removiendo de la Lista CS los caminos Cik y Cjl y agregando el puente Ckl.<br>
Actualizar EAi= EAi -1 y EAj= EAj -1, EC=EC-1 después de cada eliminación.<br>
Si EC=0 Terminar (verificar la existencia de más de un lazo).<br>
Actualizar la Lista A con el agregado de los ahorros válidos Akx y Alx<br>

Aplicar recursivamente las Regla 6 y 7 hasta que EC=0<br>

In [ ]:
def regla7(cantCiu, mat_costCiu, ec, vec_excesoArcos, mat_cs, vec_a_ahorro, vec_a_puente):
    while (ec > 0):
        # ====================== REGLA 7 ===========================
        # Remover el primer elemento de la lista A
        a_ahorro_aux = vec_a_ahorro.pop(0)
        a_puente_aux = vec_a_puente.pop(0) ## [i, [k,l], j]
        i = a_puente_aux[0]
        k = a_puente_aux[1][0]
        l = a_puente_aux[1][1]
        j = a_puente_aux[2]
        if (depurando == True):
            print("inic while: a_ahorro_aux: {}, a_puente_aux: {}".format(a_ahorro_aux,a_puente_aux))    

        mat_cs_aux = [a for a in mat_cs]
        if (depurando == True):
            print("inic while: mat_cs_aux: {}".format(mat_cs_aux))    

        # Aplicar el ahorro removiendo de la Lista CS los caminos Cik y Cjl y agregando el puente Ckl.
        if (j == 0):
            ## si se tiene un ahorro del tipo Aii (j = 0)
            # reemplazar ik e il por kl
            try:
                if (depurando == True):
                    print("Aii => j == 0: [i, [k, l], j]: [{}, [{}, {}], {}]".format(i, k, l, j))
                    print("antes de eliminar: mat_cs_aux: {}".format(mat_cs_aux))
                
                ai = []
                try:
                    mat_cs_aux.index([i,k])
                    ai = [i, k]
                except ValueError:
                    mat_cs_aux.index([k,i])
                    ai = [k, i]
                
                aj = []
                try:
                    mat_cs_aux.index([i,l])
                    aj = [i,l]
                except ValueError:
                    mat_cs_aux.index([l,i])
                    aj = [l,i]

                mat_cs_aux.remove(ai)
                mat_cs_aux.remove(aj)
                mat_cs_aux.append([k,l])
                if (depurando == True):
                    print("despues de eliminar mat_cs_aux: {}".format(mat_cs_aux))

                # Actualizar EAi= EAi -1 y EAj= EAj -1, EC=EC-1 después de cada eliminación.
                # EAi = EAi - 2
                vec_excesoArcos[i-1] -= 2
                ec -= 1
                if (depurando == True):
                    print("ec: {}, vec_excesoArcos: {}".format(ec, vec_excesoArcos))

            except ValueError:
                if (depurando == True):
                    print("Value Error")
        else:
            ## si se tiene un ahorro del tipo Aij
            # reemplazar ik y jl por kl
            try:
                if (depurando == True):
                    print("Aij => j != 0: [i, [k, l], j]: [{}, [{}, {}], {}]".format(i, k, l, j))
                    print("antes de eliminar: mat_cs_aux: {}".format(mat_cs_aux))
                
                ai = []
                try:
                    mat_cs_aux.index([i,k])
                    ai = [i, k]
                except ValueError:
                    mat_cs_aux.index([k,i])
                    ai = [k, i]
                
                aj = []
                try:
                    mat_cs_aux.index([j,l])
                    aj = [j,l]
                except ValueError:
                    mat_cs_aux.index([l,j])
                    aj = [l,j]

                mat_cs_aux.remove(ai)
                mat_cs_aux.remove(aj)
                mat_cs_aux.append([k,l])
                if (depurando == True):
                    print("despues de eliminar mat_cs_aux: {}".format(mat_cs_aux))
                
                # Actualizar EAi= EAi -1 y EAj= EAj -1, EC=EC-1 después de cada eliminación.
                # EAi = EAi - 1 y EAj = EAj - 1
                vec_excesoArcos[i-1] -= 1
                vec_excesoArcos[j-1] -= 1
                ec -= 1
                if (depurando == True):
                    print("ec: {}, vec_excesoArcos: {}".format(ec, vec_excesoArcos))

            except ValueError:
                if (depurando == True):
                    print("Value Error")

        if (depurando == True):
            print("mat_cs_aux: {}".format(mat_cs_aux))
        mat_cs = mat_cs_aux

        #Actualizar la Lista A con el agregado de los ahorros válidos Akx y Alx
            # a la lista de puentes hay que filtrar los caminos ya quitados
        # por simplicidad se vuelve a calcular lista A #TODO: ver como optimizar esto
        ec, vec_a_ahorro, vec_a_puente = calcularListaA(cantCiu, mat_costCiu, ec, vec_excesoArcos, mat_cs)
        if (depurando == True):
            print("desp calcularListaA: ec: {}, vec_excesoArcos: {}, vec_a_ahorro: {}, vec_a_puente{}".format(ec, vec_excesoArcos, vec_a_ahorro, vec_a_puente))
            print("EN EL WHILE")

    #Si EC=0 Terminar (verificar la existencia de más de un lazo).
    if (ec == 0):
        if (depurando == True):
            print("ec es 0")
        recorrido_final, costo = calcularRecorridoFinal(cantCiu, mat_cs, mat_costCiu)
        print("Terminó, recorrido final es: {} con costo: {}".format(recorrido_final, costo))
    return

#regla7(cantCiu, mat_costCiu, ec, vec_excesoArcos, mat_cs, vec_a_ahorro, vec_a_puente)

In [ ]:
def algoritmo(tipo, archivo):
    antes = datetime.now().timestamp()
    cantCiu, vec_costCiu, mat_costCiu, obj_costCiu = _leerArchConf(tipo, archivo) #tiene un doble lazo...
    if (depurando == True):
        print("mat_costCiu: {}".format(mat_costCiu))

    mat_cs = regla1(cantCiu, mat_costCiu)
    if (depurando == True):
        print("mat_cs: {}".format(mat_cs))

    vec_excesoArcos = calcularExcesoArcos(cantCiu, mat_cs)
    if (depurando == True):
        print("vec_excesoArcos: {}".format(vec_excesoArcos))

    ec = calcularEc(vec_excesoArcos)
    if (depurando == True):
        print("ec: {}".format(ec))
    if (ec == 0):
        recorrido_final, costo = calcularRecorridoFinal(cantCiu, mat_cs, mat_costCiu)
        print("Terminó, recorrido final es: {} con costo: {}".format(recorrido_final, costo))
        print("{} s".format(datetime.now().timestamp() - antes))
    else:
        ec, vec_excesoArcos, mat_cs = removerCaminosCS(ec, vec_excesoArcos, mat_cs)
        vec_costos,mat_cs = ordenarCostosCS(mat_costCiu, mat_cs, reverse=True)
        if (depurando == True):
            print("desp removerCaminosCS: vec_costos: {}, mat_cs_ordenado: {}, vec_excesoArcos: {}".format(vec_costos,mat_cs,vec_excesoArcos))

        if (ec == 0):
            recorrido_final, costo = calcularRecorridoFinal(cantCiu, mat_cs, mat_costCiu)
            print("Terminó, recorrido final es: {} con costo: {}".format(recorrido_final, costo))
            print("{} s".format(datetime.now().timestamp() - antes))
        else:
            ec, vec_a_ahorro, vec_a_puente = calcularListaA(cantCiu, mat_costCiu, ec, vec_excesoArcos, mat_cs)
            if (ec == 0):
                recorrido_final, costo = calcularRecorridoFinal(cantCiu, mat_cs, mat_costCiu)
                print("Terminó, recorrido final es: {} con costo: {}".format(recorrido_final, costo))
                print("{} s".format(datetime.now().timestamp() - antes))
            else:
                if (depurando == True):
                    print("desp calcularListaA: ec: {}, vec_excesoArcos: {}, vec_a_ahorro: {}, vec_a_puente{}".format(ec, vec_excesoArcos, vec_a_ahorro, vec_a_puente))
                regla7(cantCiu, mat_costCiu, ec, vec_excesoArcos, mat_cs, vec_a_ahorro, vec_a_puente)
                print("{} s".format(datetime.now().timestamp() - antes))
    return



In [ ]:
""" 
Resolver el TSP_IN10.txt > 130min
    Capaz acá encuentre una falencia en la implementación (o el algoritmo)
    Si no encuentro el problema, hago los tests
        Podría hacer tests:
        1. ahorros sean todos negativos
        2. tomar varios estados previos y compararlos
        Hacer impresiones de listas parciales

Tomar una circunferencia, de radio grande, e inventar ciudades => para tener un resultado conocido
    Generar un programa para crear mas y mas ciudades => guardarlas en un archivo con el formato de TSP_IN
Graficar las ciudades
¿si va bien con X ciudades, voy a 48?

Noto que los resultados dan como se espera

Graficar la resolucion del algoritmo



Verificar redondeos en costos de ciudades de TSPLIB

 """


depurando = False

""" algoritmo('TSP_IN','TSP/AGD_TSP_IN_04.txt') #[1, 2, 3, 4, 1] con costo: 7
algoritmo('TSP_IN','TSP/AGD_TSP_IN_05.txt') #[2, 5, 4, 1, 3, 2] con costo: 10
algoritmo('TSP_IN','TSP/AGD_TSP_IN_06.txt') #[4, 5, 2, 4] con costo: 11 y mas de un lazo...
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [1, 1, 6]
    #calcRecFinal: destinos: [6, 3, 3]
algoritmo('TSP_IN','TSP/AGD_TSP_IN_07.txt') #[1, 3, 5, 7, 6, 4, 2, 1] con costo: 25
algoritmo('TSP_IN','TSP/AGD_TSP_IN_08.txt') #[2, 5, 1, 6, 4, 2] con costo: 41 y mas de un lazo...
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [7, 8, 7]
    #calcRecFinal: destinos: [8, 3, 3]
algoritmo('TSP_IN','TSP/AGD_TSP_IN_09.txt') #[6, 7, 2, 5, 8, 4, 1, 3, 9, 6] con costo: 36
algoritmo('TSP_IN','TSP/AGD_TSP_IN_10.txt') #[6, 7, 2, 5, 8, 10, 4, 1, 3, 9, 6] con costo: 32
algoritmo('TSP_IN','TSP/AGD_TSP_IN_11.txt') #[2, 4, 1, 3, 9, 6, 7, 11, 10, 8, 5, 2] con costo: 35
algoritmo('TSP_IN','TSP/TSP_IN_01.txt') #[1, 2, 3, 4, 5, 1] con costo: 30
algoritmo('TSP_IN','TSP/TSP_IN_02.txt') #[1, 3, 2, 4, 5, 6, 7, 1] con costo: 35
algoritmo('TSP_IN','TSP/TSP_IN_03.txt') #[1, 5, 4, 3, 6, 2, 1] con costo: 31
algoritmo('TSP_IN','TSP/TSP_IN_04.txt') #[2, 4, 6, 5, 1, 3, 2] con costo: 30
algoritmo('TSP_IN','TSP/TSP_IN_05.txt') #[4, 12, 9, 1, 10, 11, 5, 3, 4] con costo: 84
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [2, 7, 6, 2]
    #calcRecFinal: destinos: [7, 8, 8, 6]
algoritmo('TSP_IN','TSP/TSP_IN_06.txt') #[1, 7, 2, 8, 3, 9, 1] con costo: 70
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [4, 4, 5]
    #calcRecFinal: destinos: [5, 6, 6]
algoritmo('TSP_IN','TSP/TSP_IN_07.txt') #[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1] con costo: 10
algoritmo('TSP_IN','TSP/TSP_IN_08.txt') #[4, 8, 6, 5, 9, 7, 10, 2, 1, 3, 4] con costo: 143
algoritmo('TSP_IN','TSP/TSP_IN_09.txt') #[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1] con costo: 10
algoritmo('TSP_IN','TSP/TSP_IN_10.txt') #[7, 9, 10, 8, 6, 4, 2, 1, 3, 5, 7] con costo: 17
algoritmo('TSP_IN','TSP/TSP_IN_11.txt') #[1, 2, 4, 6, 8, 10, 12, 14, 15, 3, 5, 7, 9, 11, 13, 1] con costo: 15
algoritmo('TSP_IN','TSP/TSP_IN_12.txt') #[1, 5, 10, 15, 2, 6, 11, 3, 7, 12, 4, 8, 13, 9, 14, 1] con costo: 15 """

#algoritmo('TSPLIB','ALL_tsp/euc2d/a280.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [193, 176, 149, 107, 34, 16, 14, 4, 192, 6, 31, 247, 168, 165, 129, 29, 191, 98, 33, 187, 4, 99, 92, 190, 166, 69, 67, 264, 263, 262, 261, 249, 248, 223, 205, 204, 203, 202, 185, 173, 140, 105, 74, 66, 28, 17, 12, 7, 276, 5, 278, 244, 241, 206, 201, 179, 177, 151, 139, 101, 89, 79, 36, 111, 108, 85, 83, 63, 62, 275, 274, 273, 272, 271, 269, 267, 260, 259, 258, 257, 255, 254, 253, 252, 250, 244, 242, 240, 240, 239, 238, 236, 235, 234, 233, 232, 232, 231, 231, 230, 229, 228, 227, 226, 225, 224, 221, 220, 219, 217, 217, 215, 215, 213, 213, 211, 211, 209, 209, 208, 207, 207, 199, 197, 196, 195, 194, 189, 188, 186, 184, 183, 182, 181, 180, 176, 170, 170, 167, 164, 163, 162, 161, 161, 160, 159, 158, 157, 154, 153, 153, 152, 151, 145, 145, 144, 143, 143, 142, 141, 141, 140, 138, 138, 137, 136, 136, 135, 134, 134, 130, 128, 127, 126, 124, 123, 122, 121, 120, 119, 119, 115, 114, 113, 110, 108, 106, 105, 103, 102, 100, 96, 95, 94, 94, 93, 91, 90, 89, 88, 87, 86, 84, 84, 82, 81, 80, 77, 75, 75, 73, 72, 71, 70, 68, 65, 62, 61, 60, 58, 57, 55, 53, 51, 50, 49, 49, 48, 47, 47, 46, 45, 45, 44, 44, 43, 42, 41, 40, 39, 38, 37, 35, 31, 30, 27, 26, 23, 23, 22, 22, 21, 20, 20, 19, 18, 18, 16, 13, 13, 11, 10, 8, 8, 171, 64, 278, 147, 78, 116, 219, 30, 193, 104, 243, 132]
    #calcRecFinal: destinos: [197, 179, 150, 174, 35, 17, 24, 5, 194, 7, 33, 248, 169, 166, 154, 32, 192, 99, 34, 188, 279, 100, 93, 191, 167, 70, 68, 265, 264, 263, 262, 250, 249, 224, 206, 205, 204, 203, 186, 174, 265, 107, 76, 67, 29, 133, 15, 9, 277, 6, 279, 247, 242, 208, 202, 180, 178, 178, 149, 169, 90, 80, 37, 112, 109, 86, 88, 64, 118, 276, 275, 274, 273, 272, 270, 268, 261, 260, 259, 258, 256, 257, 254, 255, 251, 245, 243, 245, 241, 246, 239, 237, 236, 235, 234, 237, 233, 246, 238, 251, 230, 229, 228, 227, 226, 225, 222, 221, 222, 220, 218, 218, 216, 216, 214, 214, 212, 252, 210, 253, 212, 210, 200, 198, 201, 196, 195, 190, 189, 187, 185, 184, 183, 182, 181, 177, 172, 171, 168, 165, 164, 163, 175, 162, 175, 160, 159, 158, 155, 156, 155, 156, 152, 199, 146, 200, 146, 144, 147, 148, 142, 266, 266, 139, 267, 268, 137, 269, 270, 135, 131, 129, 128, 127, 125, 124, 123, 122, 121, 157, 120, 117, 115, 114, 111, 110, 173, 106, 104, 103, 101, 97, 96, 97, 95, 98, 92, 91, 109, 112, 113, 116, 87, 85, 83, 82, 81, 78, 77, 76, 74, 73, 72, 71, 69, 66, 63, 118, 61, 59, 58, 56, 54, 52, 51, 52, 50, 53, 54, 48, 55, 56, 46, 59, 57, 60, 43, 42, 41, 40, 39, 38, 36, 32, 125, 28, 27, 25, 24, 26, 25, 130, 131, 21, 132, 133, 19, 271, 15, 14, 12, 11, 10, 9, 172, 65, 277, 148, 79, 117, 223, 126, 198, 102, 256, 150]
    #Terminó, recorrido final es: [1, 2, 3, 280, 1] con costo: 2660.9044066230367
    #4.090610027313232 s

#algoritmo('TSPLIB','ALL_tsp/euc2d/berlin52.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [33, 14, 27, 13, 17, 4, 3, 13, 16, 29, 20, 21, 26, 8, 23, 26, 16, 44, 46, 6, 18, 27, 4, 22, 2, 19, 1, 9, 18, 7, 19, 8, 1, 32, 2, 37, 39, 6, 38, 34, 5, 24, 35, 15, 50, 35, 10, 3, 39]
    #calcRecFinal: destinos: [43, 52, 47, 52, 21, 43, 17, 14, 29, 50, 30, 23, 47, 9, 30, 28, 44, 46, 48, 25, 22, 28, 25, 31, 42, 41, 32, 10, 31, 42, 45, 41, 49, 49, 7, 38, 40, 15, 40, 36, 24, 48, 36, 5, 20, 34, 33, 45, 37]
    #Terminó, recorrido final es: [11, 12, 51, 11] con costo: 7401.359020563809
    #0.06582522392272949 s

#algoritmo('TSPLIB','ALL_tsp/euc2d/bier127.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [83, 83, 63, 104, 96, 65, 65, 63, 88, 89, 85, 66, 92, 109, 92, 101, 69, 75, 81, 31, 82, 55, 77, 68, 78, 68, 55, 8, 70, 69, 76, 74, 18, 3, 26, 84, 35, 3, 2, 67, 12, 64, 59, 51, 40, 27, 18, 59, 58, 26, 5, 54, 44, 34, 86, 56, 50, 30, 44, 36, 35, 13, 12, 10, 9, 7, 5, 2, 1, 39, 38, 106, 85, 79, 52, 45, 17, 105, 86, 61, 61, 60, 34, 20, 19, 17, 13, 8, 6, 4, 1, 81, 21, 6, 90, 36, 23, 15, 9, 4, 58, 54, 90, 14, 109]
    #calcRecFinal: destinos: [101, 102, 82, 125, 119, 89, 113, 119, 96, 99, 110, 113, 125, 110, 99, 102, 75, 76, 126, 80, 126, 124, 79, 71, 117, 73, 66, 67, 71, 70, 78, 77, 74, 11, 27, 117, 40, 10, 16, 73, 31, 100, 60, 57, 43, 30, 21, 62, 100, 38, 52, 57, 103, 42, 88, 121, 115, 41, 45, 41, 37, 50, 14, 120, 24, 120, 121, 51, 7, 42, 39, 114, 87, 80, 124, 103, 20, 114, 87, 91, 62, 116, 43, 108, 72, 22, 115, 72, 15, 19, 16, 84, 22, 108, 116, 37, 24, 106, 11, 23, 64, 56, 91, 105, 104]
    #Terminó, recorrido final es: [98, 123, 95, 93, 127, 107, 111, 112, 94, 46, 49, 118, 48, 53, 47, 25, 29, 32, 33, 122, 28, 97, 98] con costo: 129250.12774098177
    #0.2858870029449463 s

#algoritmo('TSPLIB','ALL_tsp/euc2d/ch130.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [10, 6, 59, 28, 66, 56, 121, 22, 78, 57, 23, 62, 69, 30, 86, 64, 36, 84, 43, 66, 22, 28, 19, 3, 8, 37, 104, 101, 14, 27, 52, 79, 43, 31, 107, 24, 18, 8, 101, 17, 55, 56, 13, 15, 23, 108, 6, 15, 85, 52, 3, 30, 95, 19, 36, 32, 64, 17, 79, 37, 9, 21, 32, 67, 90, 74, 111, 10, 40, 9, 21]
    #calcRecFinal: destinos: [102, 55, 78, 62, 125, 82, 126, 33, 121, 82, 122, 115, 86, 83, 88, 88, 119, 119, 107, 85, 93, 115, 100, 83, 126, 93, 127, 111, 67, 100, 74, 116, 104, 34, 127, 29, 108, 18, 123, 34, 122, 65, 96, 29, 40, 114, 102, 24, 90, 75, 114, 59, 116, 27, 113, 84, 69, 31, 95, 47, 57, 33, 113, 96, 125, 75, 123, 14, 47, 65, 13]
    #Terminó, recorrido final es: [49, 91, 72, 58, 53, 120, 60, 51, 42, 44, 4, 20, 46, 80, 118, 35, 54, 2, 50, 130, 71, 41, 1, 39, 117, 112, 105, 128, 16, 45, 5, 11, 76, 109, 61, 129, 124, 26, 7, 97, 70, 63, 68, 48, 25, 98, 110, 89, 94, 77, 103, 81, 87, 12, 99, 73, 92, 38, 106, 49] con costo: 5928.085430533746
    #0.2982969284057617 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/ch150.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [21, 21, 132, 65, 31, 13, 64, 83, 50, 55, 97, 25, 5, 82, 91, 90, 56, 3, 70, 33, 18, 20, 26, 35, 4, 47, 33, 129, 16, 54, 13, 93, 56, 29, 3, 52, 43, 49, 32, 10, 2, 74, 44, 35, 23, 145, 27, 30, 85, 93, 59, 15, 27, 88, 64, 51, 1, 4, 6, 76, 68, 16, 22, 49, 45, 43, 2, 102, 7, 46, 30, 42, 6, 134, 47, 79, 48, 73, 37, 62, 86, 108, 20, 29, 19, 46, 59, 81, 82, 44, 87, 95, 26, 125, 89, 14, 112, 100, 77, 45, 48, 99, 88, 32, 1, 8, 15, 70, 91, 144, 7, 55, 18, 9, 23, 68, 105, 98, 14, 10, 92, 50, 5, 22, 80, 131, 75]
    #calcRecFinal: destinos: [115, 150, 137, 132, 123, 74, 136, 141, 137, 65, 100, 141, 143, 107, 106, 92, 83, 113, 102, 52, 85, 51, 75, 124, 150, 110, 105, 144, 111, 134, 106, 124, 146, 81, 62, 126, 67, 145, 67, 94, 19, 123, 115, 96, 131, 147, 129, 63, 142, 126, 78, 133, 31, 121, 112, 109, 34, 104, 28, 87, 128, 133, 125, 72, 128, 109, 37, 114, 8, 90, 89, 120, 9, 138, 120, 121, 73, 76, 42, 149, 135, 135, 25, 86, 99, 138, 79, 110, 95, 71, 98, 107, 146, 149, 96, 122, 136, 143, 122, 71, 63, 114, 94, 38, 103, 84, 78, 108, 119, 147, 84, 58, 142, 28, 38, 119, 111, 103, 80, 113, 54, 58, 97, 104, 77, 72, 34]
    #Terminó, recorrido final es: [60, 140, 117, 57, 39, 41, 101, 116, 12, 24, 118, 53, 40, 139, 127, 36, 69, 61, 11, 148, 130, 17, 66, 60] con costo: 6897.690980028125
    #0.8855581283569336 s

#algoritmo('TSPLIB', 'ALL_tsp/euc2d/d1291.tsp') # > 500min
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/d15112.tsp') #
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/d1655.tsp') #
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/d18512.tsp') #
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/d198.tsp') #
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [108, 108, 14, 167, 137, 128, 171, 138, 123, 20, 18, 15, 164, 40, 181, 182, 136, 124, 119, 197, 195, 192, 190, 188, 186, 196, 195, 189, 188, 187, 186, 174, 173, 157, 8, 164, 124, 19, 175, 173, 48, 35, 128, 151, 22, 102, 65, 35, 27, 177, 178, 153, 129, 52, 22, 18, 8, 179, 158, 154, 144, 127, 121, 118, 117, 104, 95, 85, 84, 62, 49, 45, 37, 28, 21, 11, 184, 150, 126, 160, 116, 115, 125, 107, 105, 90, 86, 76, 49, 30, 14, 144, 143, 133, 95, 89, 85, 75, 64, 48, 47, 46, 46, 42, 32, 31, 177, 44, 29, 24, 178, 165, 161, 157, 154, 152, 147, 146, 146, 141, 140, 135, 134, 131, 130, 127, 112, 110, 106, 103, 97, 94, 93, 93, 91, 89, 88, 87, 79, 74, 58, 57, 56, 55, 54, 53, 33, 31, 26, 16, 12, 11, 180, 179, 155, 149, 148, 133, 132, 119, 115, 113, 109, 106, 84, 83, 80, 79, 76, 72, 69, 68, 67, 66, 64, 59, 45, 44, 42, 36, 70, 67, 9, 39, 138, 43, 66, 26, 109, 40, 78]
    #calcRecFinal: destinos: [168, 167, 17, 168, 172, 171, 172, 139, 139, 21, 23, 16, 166, 41, 182, 183, 137, 169, 123, 198, 196, 193, 191, 189, 187, 197, 198, 190, 191, 192, 193, 175, 176, 159, 10, 165, 125, 20, 176, 174, 63, 47, 170, 163, 24, 103, 81, 39, 34, 183, 194, 160, 130, 65, 23, 19, 9, 185, 159, 156, 150, 129, 122, 120, 118, 105, 96, 100, 99, 75, 50, 61, 38, 29, 25, 13, 185, 151, 169, 161, 121, 122, 126, 111, 114, 96, 100, 81, 63, 33, 15, 145, 145, 136, 101, 97, 86, 82, 77, 50, 51, 53, 52, 57, 36, 38, 181, 56, 30, 25, 180, 166, 162, 158, 155, 162, 148, 149, 147, 143, 142, 141, 140, 132, 131, 170, 113, 111, 112, 104, 98, 101, 102, 94, 92, 90, 98, 99, 80, 83, 69, 58, 59, 60, 61, 62, 34, 32, 27, 17, 13, 12, 184, 194, 156, 152, 153, 135, 134, 120, 116, 114, 110, 107, 87, 88, 91, 92, 77, 73, 71, 72, 68, 74, 78, 60, 54, 55, 43, 37, 73, 70, 10, 51, 142, 41, 82, 28, 117, 71, 163]
    #Terminó, recorrido final es: [1, 3, 4, 5, 6, 7, 2, 1] con costo: 13205.132950251851
    #1.0430450439453125 s
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/d2103.tsp') #
#algoritmo('TSPLIB','ALL_tsp/euc2d/d493.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [8, 11, 226, 485, 35, 484, 109, 27, 103, 178, 178, 32, 30, 54, 467, 41, 467, 116, 141, 49, 139, 55, 111, 92, 27, 106, 48, 86, 124, 91, 13, 17, 4, 153, 95, 82, 81, 80, 37, 128, 102, 104, 473, 240, 22, 209, 298, 87, 64, 475, 184, 25, 429, 66, 132, 14, 123, 64, 39, 145, 89, 36, 457, 240, 84, 168, 143, 136, 164, 157, 18, 33, 129, 177, 38, 58, 99, 17, 5, 385, 335, 95, 22, 29, 266, 75, 53, 147, 127, 214, 215, 268, 117, 110, 26, 362, 88, 465, 34, 48, 215, 105, 268, 26, 115, 97, 334, 16, 77, 158, 149, 36, 165, 224, 210, 91, 56, 42, 210, 125, 113, 231, 121, 348, 232, 122, 103, 66, 269, 71, 155, 185, 203, 218, 154, 156, 130, 276, 474, 443, 228, 71, 55, 355, 100, 230, 162, 169, 7, 131, 297, 179, 190, 69, 207, 61, 206, 125, 97, 431, 183, 142, 200, 151, 180, 13, 191, 197, 209, 461, 491, 382, 148, 395, 59, 219, 78, 353, 186, 375, 130, 335, 44, 10, 385, 78, 140, 395, 121, 112, 169, 277, 202, 68, 15, 326, 477, 241, 223, 469, 172, 369, 239, 462, 174, 23, 229, 479, 237, 392, 199, 458, 463, 4, 49, 135, 72, 224, 181, 61, 444, 313, 397, 484, 114, 358, 79, 188, 485, 46, 58, 340, 392, 208, 30, 352, 334, 162, 486, 43, 431, 327, 206, 133, 468, 340, 267, 326, 212, 191, 120, 294, 451, 325, 75, 359, 161, 445, 295, 450, 293, 147, 163, 45, 193, 92, 238, 410, 400, 287, 279, 459, 171, 131, 456, 307, 390, 170, 186, 471, 98, 270, 442, 242, 425, 108, 469, 391, 174, 464, 232, 152, 428, 478, 427, 382, 175, 239, 296, 290, 380, 303, 386, 309, 387, 349, 188, 426, 421, 384, 306, 288, 441, 434, 433, 433, 285, 233, 408, 254, 253, 251, 244, 389, 299, 273, 271, 454, 452, 451, 449, 448, 447, 446, 440, 439, 438, 437, 436, 435, 434, 419, 418, 417, 416, 415, 414, 413, 409, 407, 406, 405, 404, 403, 402, 401, 388, 384, 381, 379, 378, 377, 376, 374, 368, 367, 366, 365, 364, 363, 361, 360, 357, 355, 354, 353, 351, 347, 346, 343, 339, 337, 336, 333, 332, 331, 330, 328, 327, 324, 323, 321, 320, 319, 318, 315, 311, 310, 307, 304, 303, 302, 300, 294, 292, 291, 290, 289, 286, 285, 283, 282, 281, 280, 279, 278, 275, 263, 262, 261, 260, 259, 258, 257, 250, 249, 248, 247, 246, 244, 242, 322, 430, 369, 166, 73, 370, 274, 442, 243, 424, 477, 272, 487, 212, 200, 179, 171, 349, 317, 213, 62, 140, 266, 341, 228, 227, 220, 88, 476, 9, 193, 52, 107, 156, 21, 236, 176, 67, 219, 159, 197, 9, 76, 473, 400, 454, 316, 20, 321, 409, 184, 45, 124, 246, 492, 310, 28, 98, 54]
    #calcRecFinal: destinos: [12, 12, 270, 493, 41, 493, 118, 29, 109, 183, 214, 33, 32, 65, 468, 42, 470, 128, 153, 52, 149, 65, 118, 94, 28, 107, 51, 87, 141, 94, 53, 20, 6, 164, 102, 83, 83, 82, 40, 139, 110, 111, 481, 255, 24, 222, 305, 116, 85, 476, 202, 422, 457, 69, 145, 15, 127, 86, 40, 152, 90, 37, 462, 277, 85, 187, 151, 146, 168, 158, 19, 34, 136, 196, 39, 60, 106, 19, 6, 429, 372, 100, 23, 31, 301, 77, 57, 150, 133, 226, 222, 272, 119, 114, 399, 491, 90, 466, 35, 50, 230, 108, 298, 423, 120, 99, 345, 18, 96, 165, 155, 38, 177, 225, 216, 93, 57, 43, 225, 132, 119, 241, 122, 372, 252, 126, 104, 67, 301, 73, 161, 198, 207, 223, 159, 160, 134, 297, 475, 458, 234, 72, 56, 356, 105, 236, 166, 182, 8, 137, 317, 189, 201, 70, 220, 63, 211, 129, 101, 432, 196, 144, 205, 157, 194, 14, 204, 198, 216, 463, 492, 398, 150, 427, 60, 221, 81, 359, 194, 386, 138, 362, 46, 11, 422, 80, 144, 428, 126, 113, 172, 305, 203, 70, 16, 342, 480, 267, 231, 472, 182, 390, 253, 464, 180, 24, 234, 482, 252, 423, 205, 461, 465, 5, 50, 137, 74, 235, 190, 62, 460, 322, 411, 487, 115, 364, 96, 195, 488, 47, 59, 344, 399, 211, 31, 371, 342, 167, 489, 44, 443, 329, 208, 134, 471, 356, 271, 345, 213, 192, 123, 312, 483, 341, 76, 375, 163, 459, 312, 483, 313, 148, 170, 47, 199, 93, 245, 420, 412, 300, 289, 460, 175, 135, 486, 316, 411, 173, 189, 472, 101, 274, 456, 256, 432, 112, 470, 398, 176, 466, 237, 154, 490, 480, 490, 391, 181, 254, 308, 309, 387, 308, 393, 314, 402, 350, 192, 430, 426, 394, 314, 302, 453, 446, 445, 444, 296, 238, 413, 265, 265, 264, 257, 396, 304, 278, 273, 455, 453, 452, 450, 449, 448, 447, 441, 440, 439, 438, 437, 436, 435, 420, 419, 418, 417, 416, 415, 414, 424, 408, 407, 406, 405, 404, 403, 421, 410, 401, 396, 394, 393, 389, 388, 383, 381, 380, 379, 378, 377, 376, 373, 374, 368, 367, 366, 365, 363, 361, 360, 357, 354, 350, 351, 346, 347, 344, 343, 339, 338, 336, 337, 333, 332, 331, 330, 328, 324, 323, 320, 318, 319, 315, 311, 295, 293, 292, 291, 306, 287, 286, 284, 283, 282, 281, 280, 299, 288, 264, 263, 262, 261, 260, 259, 258, 251, 250, 249, 248, 247, 245, 243, 329, 489, 370, 167, 74, 371, 275, 455, 256, 425, 478, 276, 488, 217, 204, 185, 173, 358, 325, 217, 63, 142, 269, 352, 229, 233, 221, 89, 479, 10, 195, 51, 117, 146, 235, 255, 201, 68, 227, 160, 218, 7, 79, 474, 383, 481, 348, 84, 338, 397, 187, 21, 143, 284, 482, 412, 25, 138, 373]
    #Terminó, recorrido final es: [1, 3, 2, 1] con costo: 36894.32975995228
    #167.26886987686157 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/d657.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [615, 580, 620, 205, 628, 567, 416, 562, 649, 512, 205, 574, 212, 542, 543, 613, 153, 212, 576, 648, 529, 338, 628, 378, 582, 132, 391, 431, 510, 13, 6, 458, 428, 398, 158, 563, 475, 574, 603, 384, 370, 587, 590, 516, 288, 216, 612, 648, 372, 594, 568, 514, 644, 211, 153, 476, 41, 17, 596, 615, 422, 246, 627, 606, 186, 234, 631, 161, 645, 211, 393, 614, 547, 372, 331, 418, 316, 412, 552, 557, 591, 468, 241, 250, 558, 190, 118, 575, 591, 438, 238, 578, 511, 434, 427, 258, 256, 201, 332, 8, 335, 10, 572, 8, 472, 235, 310, 595, 418, 86, 214, 409, 485, 277, 307, 157, 160, 113, 311, 141, 52, 71, 320, 266, 72, 263, 337, 283, 592, 272, 366, 143, 359, 81, 383, 322, 475, 222, 166, 622, 255, 58, 177, 350, 573, 586, 443, 41, 158, 351, 118, 224, 641, 15, 443, 601, 32, 586, 413, 399, 239, 382, 614, 477, 105, 309, 96, 314, 317, 235, 245, 371, 392, 207, 232, 328, 134, 224, 207, 156, 64, 483, 228, 223, 595, 121, 57, 206, 221, 262, 43, 373, 28, 230, 535, 58, 329, 381, 21, 315, 6, 461, 160, 596, 538, 267, 579, 126, 137, 68, 23, 390, 351, 276, 260, 43, 374, 175, 550, 472, 348, 128, 466, 341, 253, 522, 456, 135, 355, 376, 116, 102, 496, 625, 553, 148, 208, 248, 124, 49, 136, 89, 352, 218, 181, 410, 281, 349, 148, 32, 48, 642, 479, 297, 66, 39, 506, 103, 424, 402, 280, 177, 375, 316, 257, 209, 154, 12, 555, 289, 97, 83, 73, 549, 521, 410, 368, 311, 464, 208, 588, 523, 279, 249, 91, 492, 297, 533, 324, 16, 149, 230, 561, 527, 489, 411, 151, 52, 421, 379, 45, 47, 38, 324, 57, 56, 425, 336, 171, 524, 449, 87, 354, 268, 423, 240, 175, 503, 20, 367, 305, 252, 144, 367, 22, 226, 525, 454, 465, 577, 60, 478, 338, 314, 312, 111, 308, 325, 264, 225, 152, 138, 127, 114, 104, 99, 420, 395, 333, 650, 363, 460, 319, 570, 195, 27, 259, 353, 346, 337, 154, 54, 14, 343, 184, 414, 544, 306, 84, 77, 125, 107, 7, 485, 39, 561, 227, 130, 390, 69, 262, 530, 459, 342, 504, 365, 604, 592, 584, 532, 509, 416, 655, 634, 569, 554, 106, 576, 526, 247, 453, 545, 305, 537, 259, 231, 441, 336, 275, 18, 318, 353, 273, 291, 204, 490, 278, 101, 634, 24, 606, 493, 386, 92, 535, 447, 221, 320, 452, 210, 133, 309, 169, 72, 356, 424, 411, 289, 217, 169, 51, 444, 556, 531, 389, 75, 540, 360, 107, 131, 115, 651, 380, 75, 492, 607, 296, 294, 288, 582, 378, 94, 178, 452, 126, 28, 445, 129, 539, 369, 515, 84, 462, 19, 481, 189, 457, 640, 14, 308, 50, 486, 487, 406, 296, 26, 24, 134, 64, 271, 523, 132, 182, 473, 125, 93, 191, 145, 62, 497, 431, 430, 402, 345, 87, 15, 178, 243, 157, 517, 555, 339, 279, 646, 564, 35, 481, 607, 408, 407, 237, 162, 117, 61, 635, 501, 403, 377, 376, 213, 150, 467, 442, 92, 457, 395, 396, 352, 275, 209, 202, 451, 321, 174, 101, 88, 306, 233, 179, 108, 54, 644, 605, 604, 562, 396, 256, 33, 495, 159, 76, 502, 638, 463, 632, 78, 284, 488, 163, 45, 546, 225, 513, 198, 187, 40, 248, 199, 170, 550, 196, 181, 621, 617, 630, 548, 192, 180, 37, 33, 639, 637, 518, 300, 90, 88, 36, 619, 618, 599, 540, 623, 653, 183, 140, 290, 270, 394, 80, 251, 500, 282, 271, 494, 121, 147, 439, 49, 446, 109, 307, 482, 103, 474, 451, 146, 71, 534, 203, 548, 491, 164, 565, 642, 478, 42, 295, 630, 123, 228, 433, 358, 501, 373, 80, 609, 292, 23, 585, 200]
    #calcRecFinal: destinos: [624, 657, 626, 216, 654, 571, 440, 629, 653, 568, 226, 575, 234, 543, 544, 620, 176, 245, 657, 650, 560, 608, 655, 388, 583, 142, 400, 437, 529, 25, 79, 476, 625, 423, 170, 565, 520, 577, 612, 419, 388, 590, 599, 519, 295, 233, 613, 649, 400, 600, 571, 517, 646, 236, 161, 477, 53, 29, 597, 622, 440, 266, 656, 609, 197, 250, 632, 194, 647, 232, 412, 617, 553, 391, 332, 429, 317, 429, 557, 560, 594, 471, 265, 273, 567, 204, 142, 579, 600, 439, 270, 584, 512, 435, 428, 280, 285, 202, 356, 10, 347, 12, 573, 34, 473, 241, 326, 603, 430, 104, 581, 427, 515, 301, 593, 165, 183, 122, 327, 156, 471, 91, 344, 274, 76, 290, 355, 286, 598, 291, 387, 163, 374, 82, 393, 343, 521, 242, 188, 624, 276, 66, 189, 365, 627, 588, 463, 51, 166, 366, 139, 244, 643, 22, 450, 602, 35, 589, 434, 401, 263, 394, 616, 480, 106, 322, 110, 315, 330, 254, 257, 389, 399, 210, 238, 334, 509, 229, 219, 173, 74, 484, 253, 246, 598, 140, 68, 213, 240, 265, 59, 392, 30, 581, 537, 70, 340, 382, 29, 333, 7, 462, 511, 602, 539, 281, 583, 127, 508, 77, 30, 405, 370, 293, 285, 44, 379, 190, 552, 484, 362, 145, 468, 347, 269, 528, 460, 147, 361, 384, 133, 117, 500, 626, 629, 150, 220, 261, 135, 56, 507, 109, 359, 219, 188, 421, 298, 350, 155, 38, 62, 643, 480, 303, 73, 44, 507, 115, 433, 415, 593, 193, 385, 330, 268, 220, 168, 13, 559, 300, 111, 97, 86, 554, 522, 417, 383, 321, 467, 215, 589, 526, 286, 255, 105, 498, 298, 536, 325, 17, 162, 247, 566, 528, 494, 425, 167, 63, 438, 386, 50, 59, 470, 341, 63, 60, 432, 342, 172, 530, 450, 100, 363, 283, 437, 254, 194, 505, 445, 375, 312, 261, 151, 369, 25, 237, 531, 459, 469, 578, 70, 479, 357, 329, 328, 124, 323, 335, 272, 229, 168, 143, 505, 120, 120, 116, 426, 408, 339, 651, 368, 461, 334, 572, 201, 31, 267, 364, 362, 344, 159, 65, 19, 349, 195, 426, 551, 319, 95, 83, 506, 123, 79, 490, 47, 564, 243, 131, 401, 74, 264, 532, 465, 348, 508, 371, 610, 597, 585, 533, 510, 422, 656, 636, 570, 559, 122, 580, 527, 258, 455, 546, 313, 538, 274, 239, 442, 346, 287, 21, 323, 360, 278, 299, 206, 493, 292, 112, 635, 26, 611, 496, 397, 99, 536, 448, 231, 331, 453, 218, 141, 310, 179, 81, 361, 432, 420, 294, 223, 173, 55, 447, 558, 534, 398, 85, 542, 364, 113, 138, 119, 652, 385, 78, 495, 608, 302, 304, 293, 587, 387, 98, 180, 455, 136, 31, 446, 130, 541, 377, 516, 90, 466, 20, 486, 193, 474, 641, 18, 318, 55, 487, 491, 415, 303, 27, 441, 137, 69, 277, 525, 139, 187, 483, 128, 100, 197, 146, 67, 499, 436, 435, 406, 354, 93, 16, 185, 251, 164, 518, 556, 345, 282, 647, 566, 470, 482, 611, 413, 414, 249, 171, 129, 65, 636, 502, 407, 380, 381, 217, 155, 469, 444, 98, 458, 404, 403, 358, 284, 215, 203, 454, 327, 184, 108, 96, 313, 242, 186, 112, 61, 645, 610, 605, 563, 397, 260, 36, 498, 167, 82, 504, 639, 464, 633, 85, 287, 489, 165, 46, 547, 227, 514, 199, 192, 42, 252, 200, 172, 551, 198, 182, 623, 618, 631, 549, 196, 185, 40, 34, 640, 638, 519, 304, 95, 89, 37, 621, 619, 601, 541, 409, 654, 174, 149, 299, 236, 419, 102, 244, 503, 302, 269, 499, 114, 152, 417, 53, 448, 110, 357, 488, 119, 456, 449, 176, 67, 524, 214, 545, 497, 191, 569, 637, 520, 46, 326, 633, 144, 222, 436, 340, 513, 405, 94, 616, 301, 48, 652, 404]
    #Terminó, recorrido final es: [1, 3, 4, 5, 9, 11, 2, 1] con costo: 51753.2773389547
    #3065.6838319301605 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/dsj1000.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [233, 89, 542, 48, 89, 552, 238, 131, 423, 272, 188, 349, 765, 291, 101, 742, 305, 20, 261, 157, 730, 404, 568, 235, 543, 4, 330, 433, 240, 240, 189, 154, 193, 423, 394, 148, 157, 344, 154, 816, 155, 228, 860, 463, 792, 395, 119, 148, 29, 330, 394, 152, 4, 272, 87, 386, 270, 35, 333, 509, 106, 3, 641, 78, 543, 202, 266, 676, 106, 484, 587, 433, 171, 261, 699, 529, 119, 138, 644, 45, 277, 288, 523, 20, 189, 403, 459, 7, 332, 57, 464, 25, 152, 401, 219, 439, 181, 280, 798, 248, 48, 303, 428, 583, 193, 860, 306, 171, 385, 83, 99, 976, 335, 346, 208, 96, 419, 228, 46, 443, 137, 471, 389, 397, 401, 965, 393, 212, 445, 164, 808, 108, 258, 522, 302, 166, 402, 403, 188, 185, 725, 187, 368, 245, 42, 541, 19, 219, 205, 724, 392, 19, 694, 163, 790, 533, 255, 158, 294, 472, 2, 342, 650, 803, 134, 372, 537, 126, 612, 262, 236, 71, 236, 516, 8, 107, 141, 402, 237, 3, 677, 448, 266, 85, 760, 772, 643, 446, 528, 424, 169, 289, 288, 25, 562, 628, 547, 7, 55, 538, 503, 146, 370, 649, 206, 40, 27, 299, 447, 392, 761, 181, 367, 207, 384, 887, 652, 438, 35, 375, 141, 13, 310, 723, 574, 296, 325, 32, 44, 82, 267, 271, 297, 462, 292, 649, 243, 460, 111, 540, 67, 616, 625, 349, 183, 211, 615, 677, 306, 723, 59, 243, 512, 480, 397, 695, 195, 751, 710, 50, 559, 50, 45, 327, 503, 350, 966, 275, 340, 319, 426, 111, 120, 15, 343, 115, 8, 365, 742, 593, 133, 165, 667, 125, 494, 167, 9, 365, 881, 156, 178, 345, 253, 174, 67, 270, 264, 728, 374, 415, 258, 338, 467, 931, 12, 593, 242, 459, 640, 194, 99, 454, 629, 197, 24, 26, 492, 462, 120, 165, 180, 88, 482, 95, 52, 679, 68, 904, 560, 755, 43, 422, 777, 374, 410, 176, 504, 348, 368, 53, 178, 310, 539, 848, 717, 546, 340, 71, 103, 210, 317, 101, 588, 161, 387, 440, 521, 107, 864, 263, 247, 911, 100, 110, 850, 244, 498, 54, 634, 138, 234, 380, 784, 28, 612, 381, 38, 36, 382, 54, 672, 43, 262, 130, 73, 88, 452, 362, 252, 328, 74, 58, 493, 469, 84, 57, 639, 376, 17, 286, 282, 287, 84, 294, 318, 311, 358, 44, 187, 546, 447, 713, 662, 561, 21, 248, 137, 448, 104, 18, 75, 367, 377, 452, 296, 53, 17, 314, 9, 796, 118, 461, 97, 628, 520, 192, 42, 85, 142, 40, 24, 133, 103, 571, 536, 279, 167, 159, 114, 135, 569, 190, 364, 305, 336, 31, 345, 325, 848, 467, 321, 944, 567, 585, 130, 383, 623, 16, 905, 379, 563, 265, 199, 527, 158, 263, 395, 659, 486, 315, 94, 592, 30, 348, 30, 780, 224, 934, 135, 5, 329, 64, 232, 214, 259, 151, 312, 244, 220, 200, 91, 222, 123, 377, 934, 370, 334, 912, 204, 556, 689, 256, 346, 218, 146, 654, 196, 149, 79, 47, 854, 98, 23, 49, 214, 55, 28, 598, 182, 619, 321, 549, 256, 170, 82, 667, 226, 434, 139, 49, 572, 143, 888, 372, 825, 26, 73, 115, 314, 38, 192, 164, 21, 581, 155, 425, 366, 781, 98, 61, 495, 336, 32, 37, 287, 145, 803, 41, 127, 923, 373, 610, 511, 324, 381, 58, 225, 1, 333, 194, 478, 316, 594, 750, 894, 351, 533, 162, 494, 375, 566, 108, 213, 538, 514, 329, 195, 339, 827, 592, 920, 76, 646, 36, 92, 456, 398, 504, 299, 87, 626, 512, 150, 281, 405, 180, 66, 301, 173, 697, 318, 273, 575, 364, 16, 230, 23, 315, 142, 355, 39, 323, 34, 602, 122, 714, 482, 912, 696, 437, 425, 6, 15, 421, 76, 547, 150, 110, 196, 51, 620, 186, 629, 362, 656, 184, 821, 102, 391, 242, 410, 182, 65, 385, 252, 218, 488, 257, 434, 77, 209, 515, 74, 408, 406, 511, 75, 524, 70, 440, 268, 11, 557, 247, 646, 485, 625, 170, 172, 204, 307, 145, 273, 570, 357, 292, 673, 77, 116, 388, 59, 757, 12, 34, 275, 680, 31, 322, 453, 102, 117, 337, 819, 284, 60, 144, 356, 396, 466, 65, 458, 359, 497, 257, 481, 712, 125, 674, 332, 371, 245, 13, 127, 62, 121, 799, 121, 72, 320, 70, 230, 664, 95, 254, 557, 486, 285, 175, 86, 390, 400, 837, 390, 300, 811, 550, 153, 489, 769, 222, 241, 2, 373, 470, 169, 250, 60, 621, 662, 41, 274, 81, 309, 378, 500, 173, 162, 225, 66, 797, 556, 203, 229, 100, 232, 405, 112, 658, 33, 83, 636, 1, 749, 140, 112, 203, 711, 90, 149, 665, 601, 37, 707, 199, 539, 665, 79, 52, 239, 179, 117, 567, 5, 249, 260, 335, 417, 347, 97, 444, 469, 274, 143, 888, 936, 618, 618, 282, 697, 963, 47, 551, 728, 896, 33, 637, 522, 96, 342, 443, 768, 906, 455, 316, 278, 758, 136, 391, 344, 227, 754, 68, 657, 124, 224, 591, 437, 302, 679, 179, 695, 491, 259, 378, 386, 577, 731, 513, 341, 419, 989, 285, 339, 846, 123, 337, 501, 990, 289, 414, 851, 442, 853, 406, 64, 90, 18, 713, 400, 975, 139, 614, 94, 609, 409, 436, 660, 924, 237, 233, 680, 144, 561, 793, 161, 172, 901, 205, 678, 809]
    #calcRecFinal: destinos: [808, 464, 773, 407, 343, 765, 767, 607, 792, 845, 424, 404, 913, 991, 607, 913, 991, 502, 705, 407, 889, 595, 909, 949, 627, 569, 463, 455, 441, 671, 554, 238, 249, 766, 823, 338, 492, 683, 767, 845, 681, 986, 909, 476, 945, 513, 388, 746, 835, 922, 877, 260, 903, 444, 134, 741, 509, 475, 683, 602, 508, 818, 681, 572, 960, 255, 741, 730, 347, 644, 943, 603, 468, 303, 770, 568, 793, 411, 729, 996, 960, 758, 583, 584, 537, 468, 708, 267, 759, 202, 502, 551, 651, 445, 484, 980, 716, 651, 857, 884, 446, 705, 711, 685, 441, 923, 597, 846, 524, 213, 965, 998, 898, 393, 439, 980, 751, 874, 220, 542, 726, 863, 902, 811, 471, 995, 764, 927, 863, 528, 941, 627, 922, 816, 789, 933, 691, 716, 949, 234, 919, 941, 431, 603, 970, 648, 251, 384, 541, 809, 435, 61, 786, 174, 881, 776, 634, 595, 831, 788, 903, 518, 931, 968, 587, 387, 554, 962, 743, 702, 604, 750, 886, 804, 834, 124, 280, 709, 865, 328, 944, 773, 882, 615, 987, 993, 968, 898, 917, 766, 852, 487, 701, 29, 655, 709, 738, 761, 617, 892, 725, 304, 476, 729, 771, 207, 284, 953, 704, 708, 873, 653, 901, 769, 435, 889, 996, 953, 156, 674, 688, 212, 479, 910, 804, 776, 987, 798, 360, 304, 790, 617, 517, 919, 858, 862, 738, 483, 605, 582, 654, 670, 640, 921, 414, 483, 967, 740, 518, 877, 859, 623, 514, 938, 744, 868, 418, 966, 985, 415, 715, 109, 621, 588, 841, 411, 999, 591, 360, 834, 807, 527, 291, 807, 474, 472, 27, 489, 891, 844, 813, 458, 794, 907, 687, 815, 715, 722, 911, 950, 755, 658, 774, 519, 526, 847, 597, 871, 380, 529, 606, 823, 852, 977, 883, 925, 553, 488, 718, 768, 740, 578, 787, 327, 902, 784, 536, 480, 760, 351, 830, 782, 573, 552, 937, 882, 78, 940, 786, 822, 930, 659, 800, 985, 589, 475, 976, 633, 671, 460, 822, 431, 800, 870, 814, 973, 876, 652, 969, 560, 745, 578, 948, 264, 964, 910, 858, 293, 893, 293, 661, 927, 516, 271, 880, 736, 899, 159, 864, 350, 724, 613, 957, 478, 904, 382, 638, 875, 981, 736, 836, 532, 942, 894, 954, 323, 686, 684, 838, 818, 417, 626, 535, 710, 109, 614, 849, 579, 849, 684, 899, 604, 955, 689, 791, 596, 827, 876, 754, 673, 635, 764, 829, 952, 250, 422, 978, 988, 268, 642, 301, 954, 791, 832, 942, 211, 639, 398, 559, 819, 197, 933, 317, 669, 698, 900, 714, 136, 828, 632, 389, 176, 870, 749, 565, 900, 599, 701, 868, 772, 631, 727, 642, 977, 376, 600, 479, 744, 969, 495, 635, 995, 854, 986, 951, 735, 703, 140, 906, 859, 789, 963, 594, 582, 217, 307, 828, 871, 535, 782, 734, 885, 104, 521, 420, 916, 771, 947, 851, 11, 331, 690, 638, 783, 555, 326, 466, 850, 500, 510, 454, 972, 520, 924, 956, 574, 487, 956, 427, 579, 831, 655, 645, 616, 779, 962, 608, 855, 183, 756, 994, 153, 905, 994, 853, 221, 281, 645, 842, 840, 505, 841, 562, 756, 779, 783, 935, 770, 664, 200, 606, 416, 920, 967, 892, 548, 558, 908, 826, 265, 229, 979, 624, 948, 428, 613, 759, 907, 824, 999, 959, 531, 555, 548, 886, 532, 883, 839, 241, 978, 997, 727, 806, 914, 981, 693, 719, 817, 887, 685, 605, 506, 748, 836, 951, 692, 585, 660, 857, 584, 840, 277, 656, 830, 544, 449, 732, 862, 875, 788, 929, 690, 720, 653, 599, 837, 826, 998, 930, 943, 693, 610, 190, 921, 519, 829, 694, 477, 453, 810, 950, 322, 832, 636, 356, 311, 989, 805, 209, 416, 928, 474, 696, 847, 499, 970, 589, 947, 762, 525, 799, 558, 426, 661, 122, 703, 544, 221, 633, 269, 797, 570, 959, 549, 687, 678, 974, 581, 821, 526, 573, 964, 86, 757, 254, 670, 571, 940, 958, 300, 958, 598, 774, 481, 438, 600, 477, 534, 341, 814, 420, 545, 763, 421, 982, 611, 718, 720, 449, 925, 839, 747, 461, 990, 717, 620, 957, 971, 418, 880, 379, 971, 813, 762, 937, 935, 575, 775, 506, 286, 576, 566, 974, 501, 131, 915, 682, 878, 510, 890, 692, 456, 545, 269, 719, 945, 290, 897, 988, 383, 781, 873, 879, 982, 151, 815, 326, 427, 409, 553, 596, 979, 891, 838, 777, 493, 505, 928, 890, 540, 908, 975, 531, 324, 992, 916, 824, 722, 893, 973, 879, 631, 666, 731, 253, 624, 436, 865, 896, 116, 525, 319, 955, 787, 609, 357, 309, 408, 210, 952, 619, 611, 632, 371, 523, 442, 396, 734, 175, 743, 806, 914, 780, 682, 878, 485, 796, 775, 226, 1000, 884, 177, 794, 748, 992, 936, 577, 80, 491, 938, 657, 704, 497, 331, 601, 565, 550, 508, 745, 993, 669, 752, 355, 929, 1000, 983, 637, 498, 698, 972, 885, 835, 817, 997, 359, 983, 358, 208, 235, 366, 712, 217, 874, 334, 39, 746, 534, 855, 676, 166, 735, 185, 576, 897, 515, 80, 752, 563, 320, 163, 114, 470, 825, 290, 688, 184, 239, 699, 126, 251, 72, 312, 648, 643, 81, 62, 206, 186, 915, 118, 6, 297, 707, 747, 666, 227, 763, 177, 641, 278, 917, 279, 686, 46, 702, 91, 726, 92, 650, 805, 51, 691, 608, 517, 810, 672, 732, 842, 844, 499]
    #Terminó, recorrido final es: [22, 473, 843, 820, 668, 675, 128, 246, 105, 647, 580, 866, 586, 795, 223, 432, 430, 215, 276, 450, 507, 895, 961, 706, 132, 739, 147, 812, 191, 861, 353, 352, 56, 93, 465, 564, 363, 918, 946, 737, 984, 939, 833, 160, 10, 354, 129, 872, 429, 216, 490, 295, 69, 113, 802, 451, 663, 283, 231, 856, 168, 14, 399, 308, 733, 530, 778, 496, 412, 926, 369, 622, 867, 63, 869, 753, 457, 590, 700, 198, 785, 313, 298, 932, 413, 361, 201, 801, 721, 630, 22] con costo: 20122256.369692635
    #36125.7559068203 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/eil101.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [23, 39, 42, 23, 29, 32, 15, 15, 24, 2, 2, 63, 32, 21, 72, 21, 42, 68]
    #calcRecFinal: destinos: [67, 67, 43, 39, 68, 63, 57, 43, 29, 87, 57, 90, 90, 72, 73, 73, 87, 24]
    #Terminó, recorrido final es: [11, 64, 49, 36, 47, 46, 8, 45, 17, 84, 5, 60, 83, 18, 52, 89, 6, 94, 13, 95, 97, 92, 59, 96, 99, 93, 98, 37, 100, 91, 85, 61, 16, 86, 44, 38, 14, 22, 41, 74, 75, 56, 4, 25, 55, 54, 80, 12, 26, 40, 58, 53, 101, 28, 27, 69, 1, 50, 76, 77, 3, 79, 78, 34, 81, 33, 51, 9, 35, 65, 71, 66, 20, 30, 70, 31, 88, 7, 82, 48, 19, 10, 62, 11] con costo: 673.5107060922167
    #0.44196105003356934 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/eil51.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [3, 15, 3, 20, 33, 15, 35]
    #calcRecFinal: destinos: [36, 33, 20, 35, 45, 45, 36]
    #Terminó, recorrido final es: [13, 25, 18, 14, 6, 27, 48, 23, 24, 43, 7, 26, 8, 31, 28, 22, 1, 32, 11, 38, 5, 49, 10, 39, 30, 34, 50, 21, 29, 2, 16, 9, 51, 46, 12, 47, 4, 17, 37, 44, 42, 19, 40, 41, 13] con costo: 450.20710768865075
    #0.018424034118652344 s

#algoritmo('TSPLIB','ALL_tsp/euc2d/eil76.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [18, 18, 41, 21, 28, 41, 22, 25, 23, 16, 69, 42, 19, 16, 13, 25, 22, 21, 5, 13, 32, 8, 12, 36, 24, 5, 4, 2, 30, 17, 1, 36, 29, 27, 6, 47, 33, 26, 20, 20, 23, 26, 4, 1, 60, 42, 32, 12, 6, 62, 8, 60, 34, 29, 15, 27, 75, 3, 34, 2, 3]
    #calcRecFinal: destinos: [55, 24, 64, 61, 61, 56, 28, 55, 63, 49, 71, 64, 54, 51, 57, 50, 62, 74, 15, 54, 50, 19, 17, 69, 49, 37, 67, 30, 74, 40, 43, 47, 48, 45, 51, 48, 63, 76, 37, 70, 56, 67, 75, 73, 71, 43, 44, 40, 68, 73, 46, 70, 52, 45, 57, 52, 76, 44, 46, 68, 33]
    #Terminó, recorrido final es: [11, 59, 14, 53, 35, 7, 58, 72, 39, 9, 31, 10, 38, 65, 66, 11] con costo: 568.0244103487446
    #0.03404808044433594 s

#algoritmo('TSPLIB', 'ALL_tsp/euc2d/fl1400.tsp') #
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/fl1577.tsp') #
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/fl3795.tsp') #
#algoritmo('TSPLIB','ALL_tsp/euc2d/fl417.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [35, 34, 33, 36, 37, 49, 41, 52, 41, 55, 44, 45, 52, 102, 104, 43, 12, 23, 76, 68, 21, 100, 26, 1, 25, 24, 11, 2, 17, 307, 16, 378, 10, 20, 61, 23, 54, 29, 332, 106, 14, 378, 98, 250, 307, 3, 18, 64, 98, 394, 388, 68, 328, 19, 1, 13, 61, 373, 369, 81, 64, 358, 46, 302, 298, 306, 302, 67, 22, 373, 377, 369, 48, 12, 9, 358, 298, 346, 344, 305, 6, 410, 415, 413, 416, 412, 376, 371, 347, 343, 341, 300, 85, 18, 387, 386, 385, 384, 323, 322, 321, 320, 152, 44, 102, 376, 375, 371, 296, 250, 248, 304, 300, 106, 5, 215, 214, 221, 220, 211, 210, 34, 119, 109, 299, 125, 118, 115, 108, 124, 114, 5, 33, 271, 270, 375, 273, 272, 269, 268, 267, 266, 265, 264, 263, 263, 262, 262, 261, 260, 257, 257, 256, 256, 255, 254, 251, 395, 273, 272, 267, 266, 261, 260, 259, 258, 255, 254, 253, 252, 374, 370, 271, 270, 259, 258, 253, 252, 269, 268, 265, 264, 79, 79, 225, 224, 223, 223, 222, 222, 221, 220, 219, 218, 217, 216, 215, 214, 213, 212, 211, 210, 209, 209, 208, 208, 207, 75, 225, 224, 219, 218, 217, 216, 213, 212, 207, 99, 329, 129, 127, 127, 125, 123, 121, 119, 117, 115, 113, 113, 111, 107, 9, 303, 129, 128, 126, 126, 124, 123, 122, 121, 120, 118, 117, 116, 114, 112, 112, 111, 110, 109, 304, 128, 122, 120, 116, 110, 108, 46, 2, 8, 13, 321, 385, 92, 301, 372, 398, 229, 305, 130, 274]
    #calcRecFinal: destinos: [40, 38, 37, 38, 40, 51, 48, 57, 43, 57, 45, 47, 54, 105, 105, 50, 27, 30, 81, 76, 31, 101, 31, 10, 32, 32, 14, 3, 27, 333, 21, 399, 16, 28, 75, 29, 55, 30, 333, 308, 15, 379, 99, 379, 308, 4, 26, 67, 100, 409, 400, 71, 340, 24, 8, 20, 71, 412, 409, 92, 74, 400, 49, 343, 340, 332, 330, 74, 25, 396, 399, 394, 50, 17, 15, 388, 328, 347, 345, 306, 7, 411, 416, 414, 417, 413, 377, 372, 348, 344, 342, 301, 86, 19, 417, 415, 414, 410, 348, 346, 345, 341, 153, 47, 104, 398, 397, 395, 297, 251, 249, 331, 329, 107, 6, 239, 238, 245, 244, 233, 232, 36, 143, 133, 320, 149, 142, 137, 132, 148, 136, 7, 35, 293, 292, 387, 295, 294, 291, 290, 289, 288, 289, 288, 287, 285, 286, 284, 283, 282, 281, 279, 280, 278, 277, 276, 275, 411, 297, 296, 291, 290, 285, 284, 281, 280, 279, 278, 275, 274, 386, 384, 295, 294, 283, 282, 277, 276, 293, 292, 287, 286, 86, 85, 249, 248, 247, 245, 246, 244, 243, 242, 241, 240, 241, 240, 237, 236, 235, 234, 235, 234, 233, 230, 232, 230, 229, 80, 247, 246, 243, 242, 239, 238, 237, 236, 231, 101, 342, 153, 151, 149, 147, 145, 145, 141, 139, 139, 137, 135, 133, 131, 11, 322, 151, 152, 150, 148, 146, 147, 144, 143, 144, 140, 141, 138, 138, 136, 134, 135, 132, 131, 323, 150, 146, 142, 140, 134, 130, 51, 4, 22, 28, 303, 374, 80, 330, 396, 397, 231, 331, 299, 370]
    #Terminó, recorrido final es: [103, 206, 228, 204, 226, 205, 227, 203, 202, 368, 367, 393, 392, 366, 365, 391, 364, 383, 408, 407, 406, 382, 363, 381, 405, 404, 403, 362, 390, 361, 360, 389, 402, 401, 380, 359, 179, 157, 181, 159, 183, 161, 185, 163, 187, 165, 189, 167, 191, 169, 193, 171, 195, 173, 197, 175, 199, 177, 201, 200, 176, 198, 174, 196, 172, 194, 170, 192, 168, 190, 166, 188, 164, 186, 162, 184, 160, 182, 158, 180, 156, 178, 155, 154, 319, 318, 339, 317, 316, 338, 337, 315, 327, 357, 356, 355, 326, 314, 325, 354, 353, 352, 313, 336, 312, 311, 335, 351, 350, 324, 310, 334, 309, 349, 39, 42, 53, 56, 58, 59, 63, 66, 70, 97, 96, 88, 84, 78, 83, 91, 95, 82, 77, 73, 87, 94, 90, 93, 89, 65, 69, 62, 60, 72, 103] con costo: 12535.594693414077
    #91.29939699172974 s

#algoritmo('TSPLIB', 'ALL_tsp/euc2d/fnl4461.tsp') #
#algoritmo('TSPLIB','ALL_tsp/euc2d/gil262.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [53, 1, 77, 24, 45, 13, 105, 34, 106, 87, 28, 151, 154, 86, 92, 85, 76, 13, 40, 6, 49, 22, 110, 106, 29, 140, 167, 1, 160, 33, 45, 4, 178, 98, 88, 10, 108, 31, 96, 18, 111, 30, 104, 83, 74, 54, 14, 6, 165, 68, 94, 33, 128, 100, 27, 35, 199, 152, 131, 7, 198, 180, 126, 121, 48, 46, 129, 119, 81, 25, 21, 71, 144, 112, 100, 95, 44, 242, 50, 37, 25, 15, 7, 208, 35, 202, 173, 145, 102, 65, 60, 16, 142, 50, 89, 117, 98, 73, 34, 26, 208, 150, 119, 77, 70, 63, 56, 41, 11, 52, 42, 161, 137, 99, 60, 5, 124, 66, 12, 95, 56, 31, 10, 9, 92, 8, 203, 139, 83, 76, 191, 169, 84, 43, 174, 122, 103, 69, 66, 36, 234, 169, 144, 129, 24, 23, 20, 111, 86, 79, 192, 73, 53, 26, 239, 19, 225, 198, 84, 12, 135, 59, 48, 11, 252, 166, 85, 74, 72, 3, 103, 61, 36, 22, 4, 235, 63, 59, 3, 143, 109, 58, 177, 161, 115, 93, 42, 21, 18, 9, 90, 80, 23, 20, 188, 70, 97, 75, 69, 58, 43, 116, 68, 61, 57, 171, 118, 41, 19, 5, 217, 15, 65, 8, 112, 164, 46, 152, 57, 221, 80, 240, 16, 79, 94]
    #calcRecFinal: destinos: [124, 139, 212, 127, 212, 89, 132, 87, 189, 188, 156, 247, 244, 247, 255, 105, 155, 167, 127, 37, 130, 44, 158, 121, 97, 172, 248, 229, 164, 234, 93, 115, 210, 107, 151, 131, 155, 71, 166, 54, 252, 137, 215, 217, 141, 261, 107, 243, 222, 232, 149, 254, 143, 138, 173, 260, 251, 180, 238, 140, 207, 200, 215, 189, 128, 179, 130, 184, 186, 246, 186, 229, 184, 213, 209, 258, 199, 256, 91, 165, 141, 52, 160, 246, 239, 231, 261, 209, 219, 206, 158, 243, 232, 88, 248, 240, 117, 96, 182, 153, 251, 220, 218, 91, 244, 200, 242, 110, 225, 219, 194, 187, 257, 108, 109, 191, 257, 113, 133, 182, 256, 205, 238, 230, 178, 190, 214, 202, 206, 154, 262, 171, 146, 122, 196, 223, 168, 237, 168, 183, 254, 181, 192, 221, 40, 120, 29, 236, 149, 262, 218, 204, 228, 230, 260, 72, 241, 204, 135, 49, 146, 75, 196, 235, 255, 207, 222, 104, 185, 123, 113, 211, 220, 120, 81, 241, 216, 162, 118, 174, 227, 150, 237, 179, 145, 187, 197, 138, 30, 153, 211, 194, 236, 227, 258, 99, 132, 162, 177, 183, 223, 126, 142, 90, 203, 181, 123, 210, 185, 133, 27, 102, 228, 213, 190, 172, 216, 156, 214, 205, 197, 14, 116, 231, 28]
    #Terminó, recorrido final es: [147, 159, 134, 51, 67, 32, 226, 136, 176, 233, 39, 38, 62, 249, 157, 82, 224, 250, 163, 64, 125, 195, 2, 170, 148, 114, 253, 101, 193, 17, 55, 47, 175, 259, 78, 201, 245, 147] con costo: 2534.0096363967486
    #18.04486083984375 s

#algoritmo('TSPLIB','ALL_tsp/euc2d/kroA100.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [58, 26, 72, 58, 28, 14, 26, 10, 14, 48, 6, 62, 4, 16, 3, 49, 49, 43, 60, 22, 41, 27, 3, 34, 16, 6, 20, 12, 60, 21, 29, 4, 34, 35, 10, 12, 22, 11, 11, 13, 13, 9, 7, 33, 59, 41, 7, 28, 55, 15, 74, 94]
    #calcRecFinal: destinos: [67, 66, 84, 93, 67, 48, 65, 72, 71, 100, 90, 77, 66, 70, 29, 90, 63, 46, 77, 70, 100, 35, 43, 55, 94, 63, 86, 20, 62, 59, 46, 65, 83, 86, 84, 27, 88, 17, 15, 76, 33, 57, 57, 76, 74, 71, 9, 93, 83, 17, 21, 88]
    #Terminó, recorrido final es: [39, 85, 68, 73, 50, 44, 2, 54, 40, 64, 69, 81, 25, 61, 51, 87, 23, 98, 45, 91, 32, 47, 1, 92, 8, 42, 89, 31, 80, 56, 97, 75, 19, 53, 79, 18, 24, 38, 99, 36, 95, 82, 37, 5, 78, 52, 96, 30, 39] con costo: 23653.431315828893
    #0.226060152053833 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/kroA150.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [47, 5, 47, 28, 10, 69, 34, 1, 14, 92, 67, 19, 37, 72, 68, 128, 120, 82, 50, 2, 8, 56, 139, 115, 75, 93, 19, 58, 36, 3, 56, 4, 13, 16, 18, 43, 2, 49, 61, 8, 124, 90, 43, 26, 60, 22, 41, 26, 135, 105, 18, 54, 45, 27, 99, 31, 79, 88, 80, 25, 79, 44, 16, 118, 6, 125, 62, 119, 12, 142, 68, 86, 21, 60, 59, 82, 29, 4, 104, 51, 35, 10, 12, 21, 22, 99, 39, 83, 29, 71, 32, 101, 114, 112, 11, 65, 11, 95, 23, 52, 55, 40, 13, 9, 91, 24, 7, 33, 30, 59, 31, 102, 41, 40, 9, 1, 103, 7, 33, 74, 5, 105, 78, 25, 28, 53, 6, 52, 77, 51, 55, 87, 115, 30, 15, 84, 39, 3, 32, 69, 139, 112, 110, 34, 103, 88, 20]
    #calcRecFinal: destinos: [131, 147, 109, 58, 113, 73, 135, 63, 48, 130, 108, 53, 147, 113, 73, 136, 123, 116, 116, 44, 42, 119, 143, 123, 97, 131, 75, 108, 38, 14, 97, 66, 95, 70, 137, 136, 144, 90, 81, 92, 129, 106, 128, 118, 77, 70, 100, 129, 140, 148, 24, 64, 91, 35, 104, 122, 134, 137, 122, 61, 106, 50, 94, 124, 63, 145, 150, 143, 20, 148, 85, 150, 72, 62, 127, 126, 46, 65, 111, 87, 86, 84, 27, 141, 94, 102, 101, 149, 132, 100, 109, 121, 144, 132, 17, 66, 15, 126, 98, 96, 149, 54, 76, 57, 98, 38, 57, 76, 107, 74, 80, 111, 71, 64, 117, 130, 146, 117, 146, 141, 37, 142, 96, 81, 93, 134, 49, 78, 110, 125, 83, 145, 120, 121, 17, 36, 85, 46, 45, 114, 42, 107, 23, 140, 48, 127, 67]
    #Terminó, recorrido final es: [89, 138, 133, 89] con costo: 28421.557271547135
    #2.0702359676361084 s

#algoritmo('TSPLIB','ALL_tsp/euc2d/kroA200.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [87, 4, 11, 142, 12, 14, 104, 29, 103, 12, 149, 108, 88, 96, 19, 44, 45, 131, 70, 58, 88, 60, 69, 44, 92, 75, 40, 34, 84, 106, 8, 112, 121, 46, 87, 75, 129, 104, 17, 135, 20, 8, 103, 59, 37, 112, 22, 49, 31, 3, 81, 47, 171, 81, 64, 52, 70, 165, 21, 14, 42, 18, 58, 140, 36, 113, 17, 113, 19, 109, 25, 107, 133, 69, 34, 93, 131, 31, 141, 29, 127, 140, 21, 4, 43, 105, 10, 42, 33, 57, 43, 45, 98, 107, 33, 106, 36, 18, 100, 46, 11, 20, 66, 3, 22, 10, 102, 48, 67, 99, 190]
    #calcRecFinal: destinos: [96, 60, 48, 180, 40, 108, 166, 37, 129, 147, 190, 192, 114, 126, 119, 66, 97, 189, 102, 200, 98, 101, 189, 153, 119, 155, 147, 143, 170, 163, 155, 120, 172, 121, 126, 183, 146, 165, 143, 186, 162, 134, 146, 73, 184, 127, 183, 110, 67, 59, 197, 120, 200, 97, 162, 84, 76, 166, 164, 192, 55, 49, 141, 154, 74, 176, 25, 133, 99, 157, 90, 157, 176, 142, 90, 163, 180, 47, 171, 184, 186, 164, 154, 101, 137, 137, 92, 135, 156, 74, 105, 197, 114, 109, 100, 149, 57, 110, 156, 172, 52, 64, 153, 73, 134, 175, 76, 170, 55, 175, 93]
    #Terminó, recorrido final es: [7, 82, 78, 9, 138, 124, 118, 16, 179, 63, 51, 194, 122, 116, 188, 195, 182, 94, 95, 91, 150, 144, 23, 173, 168, 185, 50, 139, 86, 5, 196, 178, 56, 152, 148, 28, 39, 38, 71, 130, 72, 83, 62, 54, 6, 187, 151, 161, 125, 181, 2, 35, 169, 68, 30, 89, 41, 167, 128, 193, 158, 77, 80, 65, 177, 13, 79, 160, 15, 123, 198, 27, 191, 145, 85, 132, 1, 115, 117, 53, 111, 174, 159, 24, 32, 136, 61, 26, 199, 7] con costo: 31783.981337077916
    #3.8240959644317627 s

#algoritmo('TSPLIB','ALL_tsp/euc2d/kroB100.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [50, 66, 43, 66, 53, 19, 27, 44, 40, 5, 7, 68, 47, 61, 28, 47, 42, 27, 6, 18, 30, 52, 34, 3, 2, 60, 62, 14, 53, 5, 64, 4, 4, 36, 75, 43, 95, 91, 12, 65, 24, 3, 52, 12, 15, 2, 79, 29, 13, 17, 39, 57, 32, 26, 18, 21, 19, 26, 1, 17, 59, 92, 10, 23, 76, 14, 22, 60, 37, 35, 8, 1, 73, 20, 36, 9, 28, 56, 25, 30, 11, 58, 22, 20, 7, 9, 49, 70, 54, 21, 8, 33, 78, 37, 55, 85]
    #calcRecFinal: destinos: [55, 87, 50, 74, 73, 44, 35, 92, 67, 62, 34, 86, 81, 94, 97, 65, 82, 71, 15, 77, 49, 58, 57, 93, 16, 87, 100, 42, 70, 67, 82, 6, 83, 41, 80, 89, 98, 97, 71, 79, 77, 11, 54, 98, 33, 13, 81, 32, 78, 41, 40, 94, 59, 69, 24, 90, 96, 56, 90, 45, 76, 96, 68, 88, 99, 64, 88, 74, 72, 61, 29, 95, 85, 38, 45, 46, 91, 100, 46, 75, 93, 84, 23, 80, 84, 25, 86, 39, 72, 10, 99, 83, 16, 38, 89, 69]
    #Terminó, recorrido final es: [31, 51, 48, 63, 31] con costo: 21659.270043351462
    #0.34236907958984375 s

#algoritmo('TSPLIB','ALL_tsp/euc2d/kroB150.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [7, 76, 7, 24, 125, 12, 21, 89, 27, 125, 23, 43, 27, 50, 13, 5, 80, 15, 13, 113, 46, 12, 39, 17, 1, 41, 2, 58, 86, 65, 83, 108, 77, 114, 6, 62, 9, 25, 58, 108, 55, 26, 5, 71, 47, 22, 20, 24, 109, 39, 8, 61, 98, 104, 79, 99, 32, 30, 59, 56, 54, 107, 31, 3, 95, 91, 121, 94, 38, 33, 50, 32, 91, 42, 6, 28, 33, 73, 77, 75, 10, 25, 34, 31, 21, 72, 56, 9, 76, 131, 45, 70, 2, 8, 10, 19, 22, 62, 65, 23, 1, 88, 20, 99, 3, 70, 30, 104, 129, 88, 92, 42, 26]
    #calcRecFinal: destinos: [78, 129, 82, 137, 137, 46, 102, 122, 40, 142, 35, 119, 85, 72, 107, 86, 115, 79, 15, 126, 142, 40, 126, 106, 85, 89, 68, 98, 133, 130, 138, 114, 80, 145, 134, 138, 82, 34, 109, 113, 64, 61, 43, 124, 67, 75, 110, 117, 145, 71, 106, 132, 148, 149, 110, 105, 117, 68, 118, 101, 149, 130, 67, 59, 123, 94, 139, 95, 101, 81, 133, 132, 123, 55, 54, 38, 45, 118, 115, 134, 141, 90, 90, 47, 122, 124, 119, 78, 111, 139, 81, 121, 35, 112, 92, 105, 112, 83, 147, 102, 53, 148, 64, 141, 73, 111, 41, 147, 131, 28, 19, 17, 53]
    #Terminó, recorrido final es: [69, 140, 60, 116, 4, 14, 143, 93, 49, 18, 29, 37, 120, 16, 135, 146, 127, 51, 63, 44, 66, 48, 103, 128, 84, 11, 52, 150, 87, 96, 97, 144, 136, 100, 74, 36, 57, 69] con costo: 27131.585709581985
    #2.6911630630493164 s

#algoritmo('TSPLIB','ALL_tsp/euc2d/kroB200.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [51, 34, 72, 47, 128, 16, 34, 27, 159, 51, 121, 135, 2, 10, 19, 5, 64, 4, 4, 21, 43, 106, 123, 43, 47, 44, 44, 53, 82, 95, 109, 102, 93, 111, 77, 41, 144, 189, 3, 50, 50, 15, 117, 52, 55, 26, 112, 132, 104, 33, 5, 137, 68, 48, 2, 25, 13, 42, 96, 87, 31, 133, 157, 3, 65, 57, 42, 8, 55, 18, 21, 26, 98, 1, 63, 159, 75, 87, 116, 13, 18, 15, 17, 27, 36, 141, 176, 152, 78, 127, 130, 10, 73, 23, 46, 100, 41, 38, 31, 76, 76, 97, 14, 178, 103, 77, 32, 66, 12, 71, 37, 22, 60, 81, 92, 40, 39, 69, 35, 35, 96, 29, 137, 8, 131, 59, 30, 1, 14, 101, 66, 59, 83, 60, 52, 108, 73, 24, 108, 40, 20, 12, 149, 7, 79, 79, 32, 36, 9, 85, 28, 56, 124, 106, 186, 28, 131, 30, 17, 70, 67, 11, 58, 53, 134, 22, 20, 49, 120, 140, 7, 9, 94, 74, 49, 86, 126, 91, 37, 92, 182, 186, 71, 114, 65, 69, 170, 193, 125, 107, 132]
    #calcRecFinal: destinos: [195, 169, 169, 188, 147, 103, 57, 61, 164, 156, 156, 151, 16, 142, 135, 67, 82, 6, 83, 123, 196, 118, 142, 89, 179, 130, 102, 149, 121, 98, 110, 148, 158, 189, 153, 148, 160, 190, 11, 185, 164, 33, 194, 163, 144, 62, 128, 194, 193, 147, 197, 162, 111, 195, 155, 146, 78, 110, 162, 168, 63, 145, 172, 158, 141, 94, 109, 97, 104, 24, 90, 56, 174, 90, 126, 185, 181, 114, 163, 187, 105, 112, 45, 176, 119, 184, 180, 166, 155, 188, 200, 68, 122, 88, 146, 101, 192, 80, 48, 136, 99, 99, 64, 199, 140, 105, 175, 133, 161, 161, 184, 88, 74, 113, 200, 190, 70, 150, 172, 61, 119, 136, 151, 29, 178, 175, 134, 95, 120, 150, 145, 167, 182, 165, 116, 166, 85, 153, 152, 117, 38, 174, 198, 58, 179, 113, 167, 45, 46, 122, 91, 100, 127, 183, 196, 129, 199, 75, 192, 198, 197, 93, 84, 170, 181, 23, 80, 138, 125, 183, 84, 25, 180, 165, 86, 138, 187, 129, 72, 19, 6, 89, 107, 168, 124, 62, 39, 160, 118, 157, 81]
    #Terminó, recorrido final es: [115, 173, 139, 191, 171, 143, 177, 154, 54, 115] con costo: 30316.916743005284
    #3.652919054031372 s

#algoritmo('TSPLIB','ALL_tsp/euc2d/lin105.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [90, 87, 63, 61, 60, 35, 34, 5, 4, 92, 93, 62, 81, 45, 40, 64, 64, 59, 94, 84, 95, 79, 56, 97, 91, 40, 48, 97, 84, 98, 89, 69, 62, 41, 69, 42, 30, 83, 78, 19, 65, 60, 13, 4, 87, 38, 34, 74, 73, 41, 55, 54, 53, 52, 51, 50, 48, 47, 46, 44, 21, 30, 68, 74, 73, 72, 19, 31, 67, 101, 24, 22, 91, 78, 77, 76, 21, 57, 71, 45, 88, 46]
    #calcRecFinal: destinos: [99, 94, 70, 66, 65, 39, 38, 14, 13, 93, 98, 105, 89, 104, 104, 72, 67, 105, 100, 86, 100, 86, 59, 102, 96, 49, 49, 101, 85, 99, 90, 70, 63, 42, 75, 43, 32, 85, 83, 27, 66, 61, 14, 5, 88, 39, 35, 81, 80, 43, 56, 57, 58, 53, 54, 55, 50, 51, 52, 47, 29, 31, 71, 75, 76, 77, 24, 32, 68, 102, 27, 29, 92, 82, 79, 80, 22, 58, 82, 44, 95, 96]
    #Terminó, recorrido final es: [9, 17, 16, 25, 18, 26, 37, 36, 33, 28, 23, 20, 12, 103, 15, 11, 10, 7, 6, 1, 2, 3, 8, 9] con costo: 15209.165267167711
    #0.15945076942443848 s

#algoritmo('TSPLIB','ALL_tsp/euc2d/lin318.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [114, 9, 245, 244, 215, 214, 195, 168, 90, 63, 35, 107, 2, 197, 92, 26, 198, 108, 93, 3, 62, 186, 81, 131, 61, 66, 38, 88, 60, 49, 271, 34, 117, 12, 150, 45, 87, 145, 40, 117, 12, 169, 64, 169, 64, 164, 59, 161, 304, 189, 135, 84, 270, 305, 95, 184, 79, 133, 28, 56, 13, 106, 1, 97, 91, 94, 40, 100, 4, 150, 254, 149, 197, 153, 202, 112, 97, 7, 84, 203, 194, 174, 167, 146, 141, 121, 98, 89, 69, 62, 41, 36, 16, 113, 8, 120, 15, 174, 69, 121, 16, 42, 30, 187, 188, 83, 316, 183, 78, 229, 124, 123, 19, 18, 275, 270, 223, 214, 65, 60, 13, 4, 304, 297, 248, 244, 94, 87, 38, 34, 283, 179, 178, 74, 73, 146, 264, 261, 254, 231, 160, 159, 158, 157, 156, 155, 153, 151, 149, 148, 126, 55, 54, 53, 52, 51, 50, 48, 47, 46, 44, 43, 21, 135, 30, 173, 68, 125, 20, 283, 229, 179, 178, 177, 124, 123, 74, 73, 72, 19, 18, 136, 31, 172, 67, 234, 232, 206, 129, 128, 127, 115, 101, 24, 23, 22, 10, 286, 231, 201, 183, 182, 181, 126, 106, 91, 78, 77, 76, 21, 6, 1, 276, 45, 36, 5, 275, 163, 11, 57, 142, 14, 71, 41, 267, 162, 317]
    #calcRecFinal: destinos: [122, 17, 249, 248, 224, 223, 204, 175, 99, 70, 39, 108, 3, 198, 93, 37, 203, 113, 98, 8, 105, 194, 89, 141, 65, 176, 145, 196, 164, 316, 318, 137, 208, 103, 209, 104, 190, 209, 104, 125, 20, 177, 72, 172, 67, 210, 105, 210, 310, 191, 138, 86, 318, 310, 100, 191, 86, 138, 33, 59, 116, 111, 6, 102, 96, 196, 49, 207, 111, 154, 261, 156, 201, 154, 206, 115, 101, 10, 85, 204, 195, 175, 168, 147, 142, 122, 99, 90, 70, 63, 42, 37, 17, 114, 9, 208, 103, 180, 75, 130, 25, 43, 32, 189, 190, 85, 317, 188, 83, 237, 132, 131, 27, 26, 276, 271, 224, 215, 66, 61, 14, 5, 305, 298, 249, 245, 95, 88, 39, 35, 290, 186, 185, 81, 80, 148, 267, 264, 257, 239, 161, 162, 163, 158, 159, 160, 155, 157, 152, 151, 134, 56, 57, 58, 53, 54, 55, 50, 51, 52, 47, 46, 29, 136, 31, 176, 71, 128, 23, 286, 234, 180, 181, 182, 129, 130, 75, 76, 77, 24, 25, 137, 32, 173, 68, 237, 239, 207, 132, 133, 134, 116, 102, 27, 28, 29, 11, 290, 232, 202, 187, 184, 185, 127, 107, 92, 82, 79, 80, 22, 7, 2, 297, 48, 33, 112, 298, 167, 15, 58, 147, 120, 82, 44, 257, 152, 96]
    #Terminó, recorrido final es: [219, 227, 226, 235, 228, 236, 246, 247, 252, 251, 253, 256, 262, 263, 268, 272, 273, 280, 279, 285, 284, 291, 299, 300, 309, 308, 303, 302, 306, 307, 311, 312, 205, 200, 199, 301, 193, 192, 295, 293, 288, 292, 294, 296, 289, 287, 282, 274, 277, 278, 281, 171, 170, 166, 165, 269, 315, 266, 265, 260, 258, 259, 255, 314, 250, 143, 144, 140, 139, 242, 241, 240, 243, 238, 233, 230, 222, 313, 225, 119, 118, 221, 220, 217, 110, 109, 216, 211, 212, 213, 218, 219] con costo: 42592.18724086247
    #16.23972797393799 s

#algoritmo('TSPLIB', 'ALL_tsp/euc2d/nrw1379.tsp') #
#algoritmo('TSPLIB','ALL_tsp/euc2d/p654.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [3, 1, 106, 106, 16, 16, 1, 103, 2, 104, 4, 18, 19, 13, 12, 63, 64, 71, 11, 14, 9, 68, 65, 67, 66, 81, 93, 74, 65, 8, 18, 75, 7, 74, 82, 92, 80, 97, 90, 38, 101, 86, 78, 76, 98, 96, 88, 75, 70, 91, 85, 100, 39, 630, 622, 588, 573, 548, 540, 506, 491, 95, 87, 63, 602, 601, 596, 587, 586, 581, 520, 519, 514, 505, 504, 499, 83, 595, 580, 513, 498, 11, 629, 628, 627, 623, 621, 620, 619, 600, 597, 595, 591, 585, 582, 580, 576, 547, 546, 545, 541, 539, 538, 537, 518, 515, 513, 509, 503, 500, 498, 494, 100, 650, 649, 647, 646, 644, 638, 637, 635, 634, 632, 612, 611, 606, 605, 601, 599, 599, 598, 593, 592, 588, 586, 584, 584, 583, 578, 577, 574, 568, 567, 565, 564, 562, 556, 555, 553, 552, 550, 530, 529, 524, 523, 519, 517, 517, 516, 511, 510, 506, 504, 502, 502, 501, 496, 495, 492, 80, 38, 6, 625, 624, 617, 616, 598, 596, 583, 581, 575, 543, 542, 535, 534, 516, 514, 501, 499, 493, 103, 83, 2, 626, 623, 618, 615, 614, 608, 594, 591, 589, 579, 576, 574, 544, 541, 536, 533, 532, 526, 512, 509, 507, 497, 494, 492, 489, 487, 486, 485, 484, 483, 482, 481, 480, 479, 478, 477, 476, 475, 474, 473, 472, 471, 470, 469, 468, 467, 466, 465, 464, 463, 462, 461, 460, 459, 458, 457, 456, 455, 454, 453, 452, 451, 450, 449, 448, 447, 446, 445, 444, 443, 442, 441, 440, 439, 438, 437, 436, 435, 434, 433, 432, 431, 430, 429, 428, 427, 426, 425, 424, 423, 422, 421, 420, 419, 418, 417, 416, 415, 414, 413, 412, 411, 410, 409, 408, 407, 406, 405, 404, 403, 402, 401, 400, 399, 398, 397, 397, 396, 395, 395, 393, 391, 390, 389, 388, 387, 386, 385, 384, 383, 382, 381, 380, 379, 378, 377, 376, 375, 374, 373, 372, 371, 370, 369, 368, 367, 366, 365, 364, 363, 362, 361, 360, 359, 358, 357, 356, 355, 354, 353, 352, 351, 350, 349, 348, 347, 346, 345, 344, 343, 342, 341, 340, 339, 338, 337, 336, 335, 334, 333, 332, 331, 330, 329, 328, 327, 326, 325, 324, 323, 322, 321, 320, 319, 318, 317, 316, 315, 314, 313, 312, 311, 310, 309, 308, 307, 306, 305, 304, 303, 302, 301, 301, 300, 299, 299, 295, 294, 293, 292, 291, 290, 289, 288, 287, 286, 285, 284, 283, 282, 281, 280, 279, 278, 277, 276, 275, 274, 273, 272, 271, 270, 269, 268, 267, 266, 265, 264, 263, 262, 261, 260, 259, 258, 257, 256, 255, 254, 253, 252, 251, 250, 249, 248, 247, 246, 245, 244, 243, 242, 241, 240, 239, 238, 237, 236, 235, 234, 233, 232, 231, 230, 229, 228, 227, 226, 225, 224, 223, 222, 221, 220, 219, 218, 217, 216, 215, 214, 213, 212, 211, 210, 209, 208, 207, 206, 205, 205, 204, 203, 203, 201, 199, 198, 197, 196, 195, 194, 193, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 182, 181, 180, 179, 178, 177, 176, 175, 174, 173, 172, 171, 170, 169, 168, 167, 166, 165, 164, 163, 162, 161, 160, 159, 158, 157, 156, 155, 154, 153, 152, 151, 150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 113, 112, 111, 110, 109, 109, 108, 107, 107, 70, 91, 589, 507, 651, 639, 569, 557, 593, 578, 511, 496, 10, 90]
    #calcRecFinal: destinos: [5, 491, 297, 298, 631, 573, 549, 105, 4, 105, 5, 50, 50, 14, 62, 72, 72, 73, 13, 62, 10, 69, 67, 69, 68, 82, 95, 76, 66, 9, 19, 79, 8, 77, 86, 94, 81, 98, 93, 42, 102, 87, 79, 77, 99, 97, 89, 78, 71, 94, 89, 102, 42, 654, 642, 643, 631, 572, 560, 561, 549, 96, 88, 64, 654, 602, 597, 642, 587, 582, 572, 520, 515, 560, 505, 500, 85, 626, 618, 544, 536, 12, 653, 653, 652, 643, 641, 641, 640, 614, 613, 611, 609, 608, 607, 605, 603, 571, 571, 570, 561, 559, 559, 558, 532, 531, 529, 527, 526, 525, 523, 521, 101, 651, 650, 648, 647, 645, 639, 638, 636, 635, 633, 649, 648, 637, 636, 630, 628, 600, 627, 594, 625, 590, 622, 620, 585, 619, 579, 617, 615, 569, 568, 566, 565, 563, 557, 556, 554, 553, 551, 567, 566, 555, 554, 548, 546, 518, 545, 512, 543, 508, 540, 538, 503, 537, 497, 535, 533, 84, 39, 7, 645, 644, 633, 632, 613, 612, 607, 606, 603, 563, 562, 551, 550, 531, 530, 525, 524, 521, 104, 84, 3, 646, 624, 634, 616, 629, 621, 610, 592, 590, 604, 577, 575, 564, 542, 552, 534, 547, 539, 528, 510, 508, 522, 495, 493, 490, 488, 490, 489, 488, 487, 486, 485, 484, 483, 482, 481, 480, 479, 478, 477, 476, 475, 474, 473, 472, 471, 470, 469, 468, 467, 466, 465, 464, 463, 462, 461, 460, 459, 458, 457, 456, 455, 454, 453, 452, 451, 450, 449, 448, 447, 446, 445, 444, 443, 442, 441, 440, 439, 438, 437, 436, 435, 434, 433, 432, 431, 430, 429, 428, 427, 426, 425, 424, 423, 422, 421, 420, 419, 418, 417, 416, 415, 414, 413, 412, 411, 410, 409, 408, 407, 406, 405, 404, 403, 402, 401, 398, 400, 399, 396, 394, 392, 394, 393, 392, 391, 390, 389, 388, 387, 386, 385, 384, 383, 382, 381, 380, 379, 378, 377, 376, 375, 374, 373, 372, 371, 370, 369, 368, 367, 366, 365, 364, 363, 362, 361, 360, 359, 358, 357, 356, 355, 354, 353, 352, 351, 350, 349, 348, 347, 346, 345, 344, 343, 342, 341, 340, 339, 338, 337, 336, 335, 334, 333, 332, 331, 330, 329, 328, 327, 326, 325, 324, 323, 322, 321, 320, 319, 318, 317, 316, 315, 314, 313, 312, 311, 310, 309, 308, 307, 306, 305, 302, 304, 303, 300, 296, 298, 297, 296, 295, 294, 293, 292, 291, 290, 289, 288, 287, 286, 285, 284, 283, 282, 281, 280, 279, 278, 277, 276, 275, 274, 273, 272, 271, 270, 269, 268, 267, 266, 265, 264, 263, 262, 261, 260, 259, 258, 257, 256, 255, 254, 253, 252, 251, 250, 249, 248, 247, 246, 245, 244, 243, 242, 241, 240, 239, 238, 237, 236, 235, 234, 233, 232, 231, 230, 229, 228, 227, 226, 225, 224, 223, 222, 221, 220, 219, 218, 217, 216, 215, 214, 213, 212, 211, 210, 209, 206, 208, 207, 204, 202, 200, 202, 201, 200, 199, 198, 197, 196, 195, 194, 193, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 182, 181, 180, 179, 178, 177, 176, 175, 174, 173, 172, 171, 170, 169, 168, 167, 166, 165, 164, 163, 162, 161, 160, 159, 158, 157, 156, 155, 154, 153, 152, 151, 150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 113, 110, 112, 111, 108, 73, 92, 609, 527, 652, 640, 570, 558, 610, 604, 528, 522, 6, 99]
    #Terminó, recorrido final es: [17, 58, 56, 54, 53, 55, 57, 52, 51, 49, 48, 46, 44, 45, 47, 43, 41, 40, 36, 37, 35, 30, 29, 22, 23, 20, 21, 24, 26, 25, 27, 28, 31, 32, 33, 34, 59, 60, 61, 15, 17] con costo: 30034.93571997329
    #166.67824006080627 s

#algoritmo('TSPLIB', 'ALL_tsp/euc2d/pcb1173.tsp') #
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/pcb3038.tsp') #
#algoritmo('TSPLIB','ALL_tsp/euc2d/pcb442.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [116, 278, 175, 261, 235, 424, 116, 346, 278, 240, 211, 151, 106, 341, 226, 187, 186, 207, 261, 338, 252, 108, 174, 267, 82, 128, 401, 400, 334, 333, 228, 163, 153, 109, 342, 275, 327, 185, 127, 421, 345, 344, 344, 343, 343, 283, 279, 236, 212, 180, 178, 154, 137, 110, 102, 76, 65, 378, 339, 113, 270, 272, 260, 130, 120, 139, 228, 410, 408, 167, 81, 419, 313, 234, 104, 94, 417, 388, 189, 177, 115, 253, 423, 410, 421, 386, 312, 221, 80, 280, 280, 396, 237, 101, 132, 426, 412, 381, 374, 373, 372, 371, 370, 369, 368, 367, 365, 364, 363, 362, 360, 359, 357, 356, 354, 353, 351, 350, 349, 347, 346, 345, 341, 336, 335, 332, 331, 328, 325, 322, 320, 318, 316, 314, 311, 310, 308, 307, 306, 305, 305, 304, 303, 303, 302, 301, 301, 300, 299, 298, 297, 297, 296, 295, 294, 293, 293, 292, 291, 291, 290, 289, 289, 288, 287, 286, 285, 284, 284, 283, 281, 276, 275, 274, 273, 271, 269, 266, 263, 259, 258, 257, 256, 255, 254, 253, 251, 250, 248, 247, 246, 245, 244, 243, 242, 241, 238, 236, 235, 227, 225, 224, 223, 222, 220, 216, 215, 214, 213, 210, 209, 208, 205, 204, 203, 201, 200, 199, 198, 196, 194, 192, 191, 190, 189, 188, 184, 183, 182, 181, 180, 176, 173, 172, 171, 170, 169, 168, 166, 165, 164, 163, 162, 161, 160, 159, 158, 157, 156, 155, 150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 139, 137, 136, 135, 134, 133, 131, 129, 126, 125, 124, 123, 122, 120, 119, 118, 117, 114, 113, 112, 111, 110, 108, 107, 105, 105, 104, 103, 97, 92, 90, 88, 85, 78, 77, 74, 72, 70, 68, 66, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 32, 31, 30, 30, 29, 28, 28, 27, 26, 26, 25, 24, 24, 23, 22, 22, 21, 20, 20, 19, 18, 18, 17, 16, 16, 15, 14, 14, 13, 12, 12, 11, 10, 10, 9, 8, 8, 7, 6, 6, 5, 4, 4, 3, 2, 2, 237, 387, 337, 127, 217, 227, 144, 394, 349, 97, 96, 96, 95, 87, 324, 179, 195, 250, 249, 179, 408, 404, 359, 358, 356, 355, 272, 268, 268, 264, 214, 211, 202, 138, 121, 33, 238, 206, 383, 376, 32, 230, 84, 265, 175, 429, 286, 206, 307, 231, 111, 66, 324, 381, 277, 338, 204, 208, 330, 142, 174, 101, 262]
    #calcRecFinal: destinos: [392, 295, 187, 422, 407, 425, 138, 348, 296, 241, 212, 152, 107, 428, 411, 396, 401, 218, 413, 427, 417, 439, 399, 270, 439, 140, 406, 405, 432, 432, 407, 393, 393, 385, 428, 423, 431, 400, 389, 424, 366, 362, 361, 353, 352, 308, 281, 262, 221, 193, 190, 164, 151, 121, 103, 99, 98, 382, 433, 384, 271, 438, 419, 390, 388, 140, 406, 414, 412, 394, 100, 422, 340, 239, 441, 436, 418, 390, 397, 397, 387, 418, 438, 411, 425, 441, 340, 229, 100, 440, 426, 399, 265, 436, 391, 440, 413, 382, 375, 374, 373, 372, 371, 370, 369, 368, 366, 365, 364, 363, 361, 360, 358, 357, 355, 354, 352, 351, 350, 348, 347, 367, 342, 337, 336, 333, 332, 329, 326, 323, 321, 319, 317, 315, 312, 339, 309, 334, 331, 330, 306, 329, 328, 304, 327, 326, 302, 325, 300, 299, 323, 298, 322, 321, 320, 319, 294, 318, 317, 292, 316, 315, 290, 314, 313, 287, 311, 310, 285, 309, 282, 279, 437, 277, 276, 274, 273, 269, 264, 260, 259, 258, 257, 256, 255, 254, 252, 251, 249, 248, 247, 246, 245, 244, 243, 242, 266, 263, 240, 234, 233, 232, 231, 230, 226, 225, 224, 223, 222, 220, 219, 218, 217, 216, 215, 202, 213, 210, 209, 197, 205, 193, 203, 191, 201, 200, 199, 198, 197, 194, 192, 188, 186, 185, 184, 183, 182, 181, 178, 177, 165, 176, 173, 172, 171, 170, 169, 168, 167, 166, 162, 161, 160, 159, 158, 157, 145, 156, 155, 154, 153, 150, 149, 148, 147, 146, 143, 141, 136, 135, 134, 133, 132, 131, 130, 129, 128, 126, 125, 124, 123, 122, 119, 118, 117, 106, 115, 114, 98, 93, 91, 89, 86, 79, 99, 75, 73, 71, 69, 67, 65, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 33, 64, 63, 31, 62, 61, 29, 60, 59, 27, 58, 57, 25, 56, 55, 23, 54, 53, 21, 52, 51, 19, 50, 49, 17, 48, 47, 15, 46, 45, 13, 44, 43, 11, 42, 41, 9, 40, 39, 7, 38, 37, 5, 36, 35, 3, 414, 389, 427, 386, 404, 405, 391, 395, 433, 380, 380, 379, 379, 378, 429, 395, 196, 415, 415, 398, 409, 409, 435, 435, 434, 434, 420, 420, 416, 416, 402, 403, 402, 392, 385, 377, 239, 207, 384, 377, 376, 229, 83, 437, 152, 430, 288, 195, 335, 232, 112, 102, 430, 109, 282, 375, 233, 219, 431, 398, 403, 383, 267]
    #Terminó, recorrido final es: [34, 442, 1, 34] con costo: 54291.85149779473
    #59.987797021865845 s
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/pr1002.tsp') #
#algoritmo('TSPLIB','ALL_tsp/euc2d/pr107.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [100, 95, 90, 85, 80, 75, 69, 65, 63, 62, 61, 60, 59, 58, 57, 56, 50, 49, 44, 43, 38, 37, 32, 31, 26, 25, 20, 19, 14, 13, 8, 7, 2, 1, 102, 100, 97, 95, 94, 92, 90, 89, 87, 85, 84, 82, 80, 79, 77, 75, 74, 71, 69, 67, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 53, 51, 50, 49, 47, 45, 44, 41, 39, 38, 37, 35, 33, 32, 31, 29, 27, 26, 25, 23, 21, 20, 19, 17, 15, 14, 13, 11, 9, 7, 5, 3, 2, 1, 99, 70, 43, 8]
    #calcRecFinal: destinos: [102, 97, 92, 87, 82, 77, 71, 67, 64, 70, 74, 79, 84, 89, 94, 99, 54, 52, 48, 46, 42, 40, 36, 34, 30, 28, 24, 22, 18, 16, 12, 10, 6, 4, 103, 103, 98, 98, 96, 93, 93, 91, 88, 88, 86, 83, 83, 81, 78, 78, 76, 72, 72, 68, 68, 66, 66, 73, 76, 81, 86, 91, 96, 101, 54, 52, 53, 51, 48, 46, 47, 42, 40, 41, 39, 36, 34, 35, 33, 30, 28, 29, 27, 24, 22, 23, 21, 18, 16, 17, 15, 12, 10, 9, 6, 4, 5, 3, 101, 73, 45, 11]
    #Terminó, recorrido final es: [104, 105, 107, 106, 55, 104] con costo: 24582.337649086294
    #0.09443998336791992 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/pr124.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [46, 64, 46, 67, 5, 61, 56, 60, 59, 57, 55, 5, 81, 82, 44, 45, 30, 19, 18, 10, 7, 6, 9, 29, 25, 23, 22, 26, 2, 24, 59, 55, 16, 14, 41, 105, 63, 3, 1, 67, 65, 82, 50, 49, 48, 47, 33, 31, 30, 29, 27, 12, 11, 9, 7, 1, 39, 37, 24, 22, 44, 42, 35, 20, 18, 123, 122, 121, 120, 119, 118, 117, 116, 115, 106, 105, 99, 98, 97, 96, 95, 94, 93, 92, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 63, 52, 41, 13, 51, 17, 4, 100, 66]
    #calcRecFinal: destinos: [53, 65, 52, 81, 32, 62, 58, 62, 60, 58, 57, 6, 84, 83, 54, 54, 33, 21, 20, 11, 28, 31, 16, 36, 38, 40, 39, 37, 3, 26, 61, 56, 17, 15, 43, 107, 68, 4, 2, 83, 66, 84, 51, 50, 49, 48, 34, 32, 34, 35, 28, 13, 12, 10, 27, 8, 40, 38, 25, 23, 45, 43, 36, 21, 19, 124, 123, 122, 121, 120, 119, 118, 117, 116, 107, 106, 100, 99, 98, 97, 96, 95, 94, 93, 92, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 64, 53, 42, 14, 15, 47, 8, 124, 115]
    #Terminó, recorrido final es: [101, 102, 114, 113, 112, 111, 110, 109, 108, 104, 103, 89, 88, 87, 85, 86, 90, 91, 101] con costo: 57039.836579278315
    #0.2867310047149658 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/pr136.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [119, 107, 88, 85, 77, 73, 52, 49, 43, 39, 18, 15, 9, 5, 129, 125, 105, 103, 95, 91, 71, 69, 61, 57, 37, 35, 27, 23, 4, 2, 131, 106, 97, 93, 72, 70, 63, 59, 38, 36, 29, 25, 3, 1, 131, 129, 125, 124, 121, 120, 113, 109, 107, 106, 105, 103, 97, 95, 93, 91, 90, 89, 87, 86, 79, 77, 75, 73, 72, 71, 70, 69, 63, 61, 59, 57, 56, 55, 54, 53, 45, 43, 41, 39, 38, 37, 36, 35, 29, 27, 25, 23, 22, 21, 20, 19, 11, 9, 7, 5, 4, 3, 2, 1, 118, 117, 115, 84, 83, 82, 81, 51, 50, 48, 47, 17, 16, 14, 13, 109, 113, 75, 79, 41, 45, 11, 7]
    #calcRecFinal: destinos: [132, 119, 94, 98, 88, 85, 64, 60, 49, 52, 30, 26, 15, 18, 135, 133, 110, 114, 100, 102, 76, 80, 66, 68, 42, 46, 32, 34, 12, 8, 136, 108, 99, 101, 74, 78, 65, 67, 40, 44, 31, 33, 10, 6, 132, 130, 126, 133, 135, 136, 114, 110, 108, 118, 117, 115, 98, 96, 94, 92, 102, 101, 100, 99, 80, 78, 76, 74, 84, 83, 82, 81, 64, 62, 60, 58, 68, 67, 66, 65, 46, 44, 42, 40, 51, 50, 48, 47, 30, 28, 26, 24, 34, 33, 32, 31, 12, 10, 8, 6, 13, 14, 16, 17, 120, 121, 124, 86, 87, 89, 90, 53, 54, 55, 56, 19, 20, 21, 22, 130, 126, 96, 92, 62, 58, 24, 28]
    #Terminó, recorrido final es: [122, 128, 127, 134, 123, 116, 104, 112, 111, 122] con costo: 89288.50656859964
    #0.808323860168457 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/pr144.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [14, 87, 22, 88, 23, 84, 8, 2, 7, 1, 112, 111, 100, 86, 85, 38, 32, 31, 29, 28, 20, 19, 27, 18, 80, 79, 10, 9, 4, 107, 104, 142, 135, 128, 121, 116, 115, 87, 68, 61, 56, 49, 46, 43, 22, 13, 107, 105, 26, 25, 17, 16, 113, 111, 100, 83, 39, 38, 33, 31, 7, 1, 10, 4, 143, 142, 140, 139, 138, 136, 135, 133, 132, 129, 128, 126, 125, 124, 122, 121, 119, 118, 116, 115, 108, 104, 81, 79, 71, 69, 68, 66, 65, 64, 62, 61, 59, 57, 56, 54, 53, 52, 50, 49, 48, 46, 45, 43, 41, 35, 26, 24, 17, 15, 5, 101, 12, 134, 120, 60, 72, 138, 124, 64, 52, 83, 3]
    #calcRecFinal: destinos: [15, 103, 37, 103, 37, 85, 19, 28, 9, 3, 114, 113, 102, 110, 86, 40, 34, 33, 30, 29, 21, 20, 30, 21, 82, 81, 12, 11, 6, 109, 106, 144, 137, 130, 123, 118, 132, 88, 70, 63, 58, 51, 48, 45, 23, 14, 108, 106, 35, 36, 41, 42, 114, 112, 101, 84, 40, 39, 34, 32, 8, 2, 11, 5, 144, 143, 141, 140, 139, 137, 136, 134, 133, 130, 129, 127, 126, 125, 123, 122, 120, 119, 117, 131, 109, 105, 82, 80, 72, 70, 69, 67, 66, 65, 63, 62, 60, 58, 57, 55, 54, 53, 51, 50, 59, 47, 71, 44, 42, 36, 27, 25, 18, 16, 6, 102, 13, 131, 117, 47, 44, 141, 127, 67, 55, 110, 24]
    #Terminó, recorrido final es: [92, 93, 94, 95, 96, 97, 98, 99, 90, 89, 78, 77, 76, 75, 73, 74, 91, 92] con costo: 35495.966890277276
    #3.8026139736175537 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/pr152.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [108, 104, 100, 96, 32, 27, 22, 123, 121, 119, 117, 45, 43, 41, 115, 113, 111, 109, 33, 28, 23, 105, 101, 97, 93, 29, 24, 19, 123, 121, 119, 117, 45, 43, 41, 108, 104, 100, 96, 32, 27, 22, 124, 122, 120, 118, 48, 46, 44, 42, 106, 102, 98, 94, 29, 24, 19, 150, 147, 144, 141, 124, 122, 120, 118, 48, 46, 44, 42, 139, 135, 131, 127, 60, 57, 54, 137, 133, 129, 125, 58, 55, 52, 49, 105, 101, 97, 93, 30, 25, 20, 150, 148, 145, 142, 143, 146, 149, 152]
    #calcRecFinal: destinos: [110, 112, 114, 116, 37, 38, 39, 127, 131, 135, 139, 69, 65, 61, 116, 114, 112, 110, 37, 38, 39, 107, 103, 99, 95, 31, 26, 20, 128, 132, 136, 140, 54, 57, 60, 109, 111, 113, 115, 33, 28, 23, 126, 130, 134, 138, 50, 53, 56, 59, 107, 103, 99, 95, 30, 25, 21, 151, 148, 145, 142, 125, 129, 133, 137, 49, 52, 55, 58, 140, 136, 132, 128, 61, 65, 69, 138, 134, 130, 126, 59, 56, 53, 50, 106, 102, 98, 94, 31, 26, 21, 152, 149, 146, 143, 141, 144, 147, 151]
    #Terminó, recorrido final es: [84, 86, 83, 64, 63, 62, 87, 82, 88, 81, 68, 67, 66, 89, 80, 90, 79, 72, 71, 70, 91, 78, 92, 77, 76, 74, 75, 73, 51, 47, 34, 36, 35, 1, 16, 2, 15, 3, 14, 4, 13, 5, 12, 6, 11, 7, 10, 8, 9, 18, 17, 40, 85, 84] con costo: 62023.40531880582
    #13.692190885543823 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/pr226.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [20, 20, 65, 66, 62, 21, 1, 73, 50, 17, 19, 18, 33, 13, 71, 48, 69, 67, 17, 31, 11, 32, 12, 68, 27, 26, 7, 6, 32, 31, 12, 11, 224, 194, 191, 188, 158, 155, 152, 151, 134, 133, 84, 74, 61, 51, 25, 22, 5, 2, 79, 56, 34, 30, 29, 14, 10, 9, 225, 224, 222, 221, 220, 219, 218, 217, 215, 214, 213, 212, 211, 210, 209, 208, 207, 205, 204, 203, 202, 200, 199, 198, 197, 195, 194, 192, 191, 189, 188, 186, 185, 184, 183, 182, 181, 179, 178, 177, 176, 175, 174, 173, 172, 171, 169, 168, 167, 166, 164, 163, 162, 161, 159, 158, 156, 155, 153, 151, 148, 148, 145, 144, 143, 142, 141, 141, 140, 139, 139, 138, 137, 137, 135, 133, 130, 130, 127, 126, 125, 124, 123, 123, 122, 121, 121, 120, 119, 119, 117, 115, 113, 111, 110, 109, 109, 108, 107, 107, 106, 105, 104, 104, 103, 103, 101, 99, 97, 95, 94, 93, 93, 92, 91, 91, 90, 89, 88, 88, 87, 87, 85, 84, 82, 81, 80, 78, 77, 76, 74, 62, 60, 59, 58, 57, 55, 54, 53, 25, 23, 22, 5, 3, 2, 201, 165, 29, 9, 206, 170, 216, 180, 21, 1, 197, 161]
    #calcRecFinal: destinos: [52, 51, 66, 70, 64, 50, 73, 75, 52, 18, 65, 19, 49, 72, 72, 49, 70, 68, 67, 36, 16, 36, 16, 69, 28, 28, 8, 8, 48, 33, 71, 13, 226, 196, 193, 190, 160, 157, 154, 153, 136, 135, 86, 76, 63, 53, 27, 24, 7, 4, 80, 57, 35, 34, 30, 15, 14, 10, 226, 225, 223, 222, 221, 220, 219, 218, 216, 215, 214, 213, 212, 211, 210, 209, 208, 206, 205, 204, 203, 201, 200, 199, 198, 196, 195, 193, 192, 190, 189, 187, 186, 185, 184, 183, 182, 180, 179, 178, 177, 176, 175, 174, 173, 172, 170, 169, 168, 167, 165, 164, 163, 162, 160, 159, 157, 156, 154, 152, 150, 149, 147, 146, 150, 143, 149, 142, 147, 145, 140, 146, 144, 138, 136, 134, 132, 131, 129, 128, 132, 125, 131, 124, 129, 127, 122, 128, 126, 120, 118, 116, 114, 112, 118, 117, 110, 116, 115, 108, 114, 112, 113, 106, 111, 105, 102, 100, 98, 96, 102, 101, 94, 100, 99, 92, 98, 96, 97, 90, 95, 89, 86, 85, 83, 82, 81, 79, 78, 77, 75, 63, 61, 60, 59, 58, 56, 55, 54, 26, 24, 23, 6, 4, 3, 202, 166, 35, 15, 207, 171, 217, 181, 64, 83, 223, 187]
    #Terminó, recorrido final es: [41, 43, 47, 46, 45, 44, 37, 38, 42, 39, 40, 41] con costo: 66425.9464588144
    #7.147763013839722 s
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/pr2392.tsp') #
#algoritmo('TSPLIB','ALL_tsp/euc2d/pr264.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [118, 119, 162, 42, 161, 42, 261, 130, 232, 126, 161, 43, 232, 155, 125, 46, 146, 1, 229, 228, 220, 203, 158, 150, 149, 147, 145, 141, 139, 137, 135, 125, 122, 113, 107, 101, 95, 89, 86, 50, 47, 45, 40, 37, 33, 27, 21, 15, 9, 201, 200, 200, 198, 85, 85, 83, 82, 262, 261, 230, 196, 159, 151, 133, 131, 130, 54, 53, 44, 3, 2, 259, 257, 255, 253, 251, 249, 247, 245, 243, 241, 235, 229, 228, 220, 219, 218, 217, 216, 214, 214, 213, 212, 210, 210, 209, 208, 206, 206, 204, 204, 197, 194, 193, 193, 191, 190, 190, 189, 188, 187, 185, 184, 184, 183, 182, 181, 179, 178, 178, 177, 176, 175, 174, 173, 172, 164, 157, 156, 153, 152, 151, 150, 149, 146, 145, 142, 141, 140, 139, 138, 137, 136, 135, 134, 133, 126, 123, 122, 120, 119, 116, 114, 113, 111, 110, 108, 107, 105, 104, 102, 101, 99, 98, 96, 95, 93, 92, 90, 89, 87, 80, 80, 79, 78, 76, 76, 75, 74, 72, 72, 71, 70, 68, 68, 67, 66, 64, 64, 63, 62, 60, 60, 57, 55, 53, 51, 50, 49, 47, 38, 37, 36, 35, 34, 32, 31, 31, 30, 29, 28, 26, 25, 25, 24, 23, 22, 20, 19, 19, 18, 17, 16, 14, 13, 13, 12, 11, 10, 8, 7, 7, 6, 4, 2, 1, 246, 250, 254, 257, 197, 6, 61, 160, 44]
    #calcRecFinal: destinos: [129, 129, 226, 59, 162, 43, 263, 132, 234, 128, 226, 59, 233, 158, 128, 48, 148, 41, 233, 236, 241, 260, 159, 153, 156, 148, 164, 142, 140, 138, 136, 127, 124, 118, 112, 106, 100, 94, 88, 52, 49, 46, 41, 39, 34, 28, 22, 16, 10, 202, 259, 203, 201, 87, 86, 84, 84, 263, 262, 231, 199, 160, 152, 134, 132, 131, 55, 56, 45, 4, 5, 260, 258, 256, 254, 252, 250, 248, 246, 244, 242, 236, 234, 235, 242, 243, 219, 245, 217, 248, 215, 249, 213, 252, 211, 253, 209, 256, 207, 258, 205, 198, 195, 205, 194, 192, 207, 191, 208, 189, 188, 186, 211, 185, 212, 183, 182, 180, 215, 179, 216, 177, 176, 218, 174, 173, 165, 227, 157, 154, 230, 231, 154, 155, 147, 165, 172, 175, 180, 181, 186, 187, 192, 195, 196, 199, 127, 124, 123, 121, 120, 117, 115, 114, 112, 111, 109, 108, 106, 105, 103, 102, 100, 99, 97, 96, 94, 93, 91, 90, 88, 91, 81, 92, 79, 97, 77, 98, 75, 103, 73, 104, 71, 109, 69, 110, 67, 115, 65, 116, 63, 121, 61, 58, 56, 54, 52, 51, 57, 48, 39, 38, 62, 36, 35, 33, 65, 32, 66, 30, 29, 27, 69, 26, 70, 24, 23, 21, 73, 20, 74, 18, 17, 15, 77, 14, 78, 12, 11, 9, 81, 8, 82, 5, 3, 40, 244, 247, 251, 255, 202, 83, 117, 227, 58]
    #Terminó, recorrido final es: [240, 264, 237, 238, 224, 225, 163, 221, 169, 170, 171, 143, 144, 166, 167, 168, 222, 223, 239, 240] con costo: 38813.948575412694
    #3.6406099796295166 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/pr299.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [143, 142, 7, 54, 66, 93, 92, 29, 146, 65, 234, 236, 238, 106, 72, 30, 109, 150, 226, 198, 196, 67, 12, 2, 297, 272, 257, 180, 173, 148, 141, 139, 134, 67, 62, 54, 35, 32, 1, 236, 235, 36, 239, 192, 111, 197, 168, 114, 270, 128, 22, 113, 260, 248, 225, 184, 126, 118, 106, 63, 59, 7, 13, 44, 40, 265, 262, 261, 254, 249, 248, 242, 227, 193, 183, 170, 169, 128, 123, 121, 82, 72, 71, 64, 63, 57, 41, 39, 38, 32, 31, 30, 28, 27, 26, 25, 24, 23, 22, 258, 257, 252, 245, 240, 237, 213, 163, 134, 117, 97, 89, 85, 75, 62, 61, 59, 225, 172, 18, 114, 124, 119, 94, 88, 79, 14, 11, 269, 267, 263, 254, 246, 244, 186, 185, 178, 176, 175, 171, 171, 115, 108, 87, 86, 85, 76, 76, 75, 44, 39, 16, 15, 269, 99, 55, 191, 81, 69, 151, 153, 150, 5, 1, 296, 296, 265, 252, 250, 244, 212, 212, 194, 187, 181, 180, 179, 175, 167, 147, 147, 145, 140, 124, 121, 111, 100, 96, 95, 89, 84, 83, 83, 73, 52, 51, 50, 49, 48, 47, 19, 15, 10, 9, 8, 2, 154, 3, 18, 40, 37, 45, 41, 274, 193, 188, 240, 169, 118, 190, 102, 145, 141, 58, 261, 20, 33, 183, 53, 78, 177]
    #calcRecFinal: destinos: [144, 143, 8, 55, 68, 144, 93, 71, 155, 105, 235, 237, 272, 107, 77, 34, 110, 152, 299, 234, 242, 70, 13, 5, 298, 273, 259, 182, 190, 149, 142, 140, 164, 69, 107, 56, 36, 35, 3, 238, 274, 37, 243, 194, 112, 198, 197, 115, 271, 172, 23, 123, 262, 268, 227, 185, 127, 120, 108, 64, 60, 92, 14, 46, 43, 267, 264, 263, 255, 251, 250, 243, 299, 247, 184, 195, 170, 174, 127, 125, 102, 73, 74, 105, 66, 58, 45, 60, 42, 33, 34, 31, 29, 28, 27, 26, 25, 24, 79, 259, 258, 255, 246, 241, 241, 218, 164, 163, 119, 139, 90, 87, 80, 65, 110, 61, 226, 174, 21, 116, 126, 120, 96, 90, 81, 88, 12, 271, 268, 266, 260, 247, 245, 256, 256, 187, 178, 176, 179, 173, 117, 109, 95, 94, 86, 80, 78, 77, 47, 57, 17, 17, 270, 100, 56, 192, 82, 70, 152, 154, 151, 6, 4, 298, 297, 266, 253, 251, 249, 218, 213, 195, 189, 182, 181, 189, 177, 168, 149, 148, 146, 161, 125, 122, 113, 101, 97, 98, 91, 99, 98, 84, 74, 53, 52, 51, 50, 49, 48, 20, 16, 11, 10, 9, 6, 155, 4, 19, 42, 38, 46, 43, 273, 196, 253, 239, 191, 122, 188, 101, 153, 161, 112, 264, 21, 68, 186, 116, 91, 167]
    #Terminó, recorrido final es: [156, 157, 158, 159, 160, 211, 210, 209, 222, 221, 220, 208, 207, 224, 223, 204, 205, 206, 162, 137, 138, 136, 135, 133, 103, 132, 129, 104, 130, 131, 165, 166, 201, 199, 200, 202, 228, 203, 229, 231, 230, 233, 232, 277, 276, 275, 279, 278, 281, 282, 283, 280, 284, 285, 286, 219, 216, 217, 287, 288, 289, 291, 290, 292, 293, 295, 294, 215, 214, 156] con costo: 49080.71718689763
    #83.33115410804749 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/pr439.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [399, 400, 149, 396, 430, 229, 430, 402, 403, 189, 18, 31, 377, 376, 341, 380, 187, 379, 83, 148, 24, 419, 225, 414, 190, 145, 27, 428, 406, 403, 343, 283, 281, 232, 190, 188, 109, 108, 32, 23, 18, 417, 404, 284, 29, 230, 424, 149, 374, 408, 385, 57, 26, 62, 113, 361, 367, 413, 412, 291, 289, 57, 287, 424, 63, 368, 317, 409, 254, 366, 96, 29, 425, 376, 360, 326, 231, 228, 226, 171, 131, 35, 1, 285, 56, 385, 222, 38, 375, 280, 223, 120, 267, 400, 388, 383, 268, 222, 221, 84, 83, 184, 159, 103, 67, 26, 12, 384, 143, 301, 166, 101, 95, 363, 264, 19, 429, 418, 415, 402, 396, 393, 392, 389, 360, 300, 294, 273, 257, 246, 159, 138, 135, 134, 110, 105, 104, 100, 86, 78, 77, 77, 73, 68, 53, 17, 228, 175, 161, 285, 263, 255, 249, 241, 240, 138, 121, 330, 162, 2, 1, 99, 69, 44, 328, 209, 72, 21, 174, 355, 350, 345, 322, 321, 311, 310, 254, 252, 216, 215, 198, 197, 182, 179, 178, 177, 175, 160, 157, 156, 153, 128, 46, 4, 368, 318, 314, 313, 301, 210, 205, 152, 143, 71, 65, 47, 123, 61, 432, 422, 419, 414, 412, 410, 386, 357, 352, 347, 331, 292, 290, 288, 81, 80, 79, 51, 42, 39, 38, 393, 371, 342, 180, 155, 103, 99, 97, 92, 27, 5, 302, 172, 168, 145, 33, 16, 391, 374, 372, 358, 353, 348, 317, 315, 209, 207, 184, 183, 150, 113, 30, 21, 9, 382, 181, 154, 70, 45, 397, 327, 325, 324, 323, 320, 312, 309, 308, 307, 305, 250, 220, 219, 218, 217, 214, 213, 212, 211, 210, 206, 203, 202, 201, 200, 199, 196, 195, 194, 193, 165, 164, 163, 132, 131, 117, 115, 108, 95, 93, 92, 66, 36, 35, 15, 14, 13, 365, 364, 363, 361, 356, 355, 351, 350, 346, 345, 338, 337, 336, 335, 334, 333, 330, 303, 299, 298, 297, 296, 295, 294, 278, 277, 276, 274, 273, 271, 270, 269, 268, 266, 265, 262, 261, 260, 258, 257, 247, 246, 244, 243, 242, 239, 238, 236, 235, 234, 173, 169, 168, 147, 141, 140, 139, 136, 134, 127, 126, 124, 119, 118, 116, 111, 110, 106, 104, 90, 89, 88, 75, 60, 59, 55, 53, 50, 48, 43, 41, 40, 22, 11, 8, 7, 6, 2, 394, 371, 230, 186, 10, 144, 59, 191, 172, 129, 94, 142, 245, 233, 340, 249, 415, 48, 276, 65, 98, 238, 382, 410, 166, 417, 320, 34, 341]
    #calcRecFinal: destinos: [406, 416, 226, 407, 432, 282, 433, 405, 409, 225, 20, 32, 379, 380, 342, 390, 188, 389, 85, 185, 25, 421, 231, 420, 224, 185, 64, 429, 407, 408, 344, 284, 282, 280, 223, 189, 146, 144, 62, 25, 19, 418, 405, 344, 30, 281, 427, 186, 381, 411, 387, 79, 64, 75, 116, 370, 378, 423, 422, 292, 290, 58, 340, 426, 73, 378, 319, 411, 316, 367, 97, 63, 426, 377, 370, 327, 232, 229, 227, 208, 133, 37, 4, 287, 82, 386, 233, 40, 381, 286, 224, 122, 306, 401, 401, 384, 305, 234, 237, 85, 84, 191, 160, 114, 68, 28, 13, 394, 150, 303, 167, 123, 100, 369, 307, 20, 431, 428, 416, 404, 397, 398, 399, 390, 362, 333, 339, 275, 259, 248, 177, 158, 137, 136, 112, 107, 106, 101, 87, 86, 87, 78, 74, 69, 55, 33, 283, 204, 162, 286, 264, 256, 252, 242, 241, 139, 122, 332, 163, 52, 3, 125, 70, 47, 329, 253, 88, 23, 204, 357, 352, 347, 323, 322, 312, 311, 255, 253, 217, 216, 199, 198, 183, 180, 179, 178, 176, 161, 158, 157, 154, 130, 49, 5, 369, 319, 315, 314, 302, 250, 206, 153, 151, 72, 66, 49, 124, 76, 433, 423, 420, 421, 413, 425, 387, 359, 354, 349, 332, 293, 291, 289, 82, 81, 80, 52, 44, 42, 39, 395, 373, 343, 181, 156, 115, 128, 98, 102, 28, 6, 304, 173, 169, 147, 34, 17, 392, 375, 373, 359, 354, 349, 318, 316, 251, 208, 192, 192, 151, 114, 31, 22, 10, 383, 182, 155, 91, 46, 398, 328, 326, 325, 324, 321, 313, 310, 309, 308, 306, 251, 221, 220, 219, 218, 215, 214, 213, 212, 211, 207, 205, 203, 202, 201, 200, 197, 196, 195, 194, 167, 165, 164, 133, 132, 120, 117, 109, 96, 94, 93, 67, 37, 36, 16, 15, 14, 366, 365, 364, 362, 358, 356, 353, 351, 348, 346, 339, 338, 337, 336, 335, 334, 331, 304, 300, 299, 298, 297, 296, 295, 279, 278, 277, 275, 274, 272, 271, 270, 269, 267, 266, 263, 262, 261, 259, 258, 248, 247, 245, 244, 243, 240, 239, 237, 236, 235, 174, 170, 170, 148, 142, 141, 140, 137, 135, 129, 127, 126, 121, 119, 118, 112, 111, 107, 105, 91, 90, 89, 76, 61, 60, 56, 54, 51, 50, 45, 43, 41, 24, 12, 9, 8, 7, 3, 395, 372, 187, 227, 11, 146, 58, 193, 171, 130, 102, 152, 260, 272, 288, 256, 388, 54, 279, 71, 125, 265, 391, 427, 176, 431, 329, 74, 293]
    #Terminó, recorrido final es: [434, 437, 435, 439, 436, 438, 434] con costo: 103655.70558478309
    #161.14152908325195 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/pr76.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [1, 13, 1, 14, 71, 57, 71, 51, 12, 62, 39, 16, 13, 27, 4, 26, 3, 29, 28, 5, 34, 11, 57, 34, 11, 41, 18, 51, 6, 65, 42, 19, 61, 38, 32, 26, 15, 9, 3, 63, 53, 30, 7, 42, 41, 19, 18, 59, 36, 75, 72, 61, 38, 8, 43, 20]
    #calcRecFinal: destinos: [76, 74, 75, 74, 73, 64, 72, 65, 14, 63, 40, 17, 15, 28, 5, 29, 6, 32, 33, 10, 40, 17, 58, 35, 12, 59, 36, 66, 7, 66, 43, 20, 62, 39, 33, 27, 16, 10, 4, 64, 54, 31, 8, 54, 60, 31, 37, 60, 37, 76, 73, 58, 35, 9, 53, 30]
    #Terminó, recorrido final es: [68, 70, 67, 50, 49, 52, 55, 56, 2, 23, 22, 21, 25, 24, 46, 45, 44, 48, 47, 69, 68] con costo: 115639.44139024527
    #0.16555309295654297 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/rat195.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [132, 157, 182, 171, 190, 188, 127, 158, 164, 160, 132, 130, 194, 170, 129, 179, 177, 175, 172, 146, 146, 151, 149, 114, 110, 142, 111, 192, 187, 183, 180, 159, 148, 123, 161, 191, 181, 122, 112, 108, 124, 116, 108, 186, 178, 193, 185, 109, 153, 157, 152, 150, 184, 147, 174, 125, 126, 173, 165, 163, 162, 115, 113, 121, 176, 144, 128, 189, 117]
    #calcRecFinal: destinos: [144, 170, 195, 183, 191, 189, 128, 159, 165, 161, 145, 143, 195, 171, 142, 180, 178, 176, 173, 160, 147, 152, 150, 115, 111, 143, 112, 193, 188, 184, 181, 172, 149, 124, 162, 192, 182, 123, 113, 121, 125, 117, 109, 187, 179, 194, 186, 110, 166, 158, 153, 151, 185, 148, 175, 126, 127, 174, 166, 164, 163, 116, 114, 122, 177, 145, 129, 190, 130]
    #Terminó, recorrido final es: [1, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 167, 168, 169, 156, 155, 154, 141, 140, 139, 138, 137, 136, 135, 134, 133, 120, 119, 118, 131, 107, 106, 105, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1] con costo: 2568.5177061689496
    #1.012138843536377 s

#algoritmo('TSPLIB','ALL_tsp/euc2d/rat575.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [300, 112, 87, 506, 327, 348, 85, 17, 477, 369, 63, 499, 281, 114, 107, 16, 109, 87, 56, 329, 67, 329, 15, 5, 553, 39, 415, 115, 81, 13, 130, 68, 514, 28, 305, 279, 114, 90, 12, 478, 242, 111, 66, 60, 21, 23, 543, 301, 394, 392, 107, 45, 21, 257, 432, 351, 300, 12, 396, 303, 65, 278, 506, 475, 302, 88, 15, 13, 5, 281, 22, 551, 38, 62, 57, 49, 136, 128, 432, 60, 50, 40, 18, 517, 510, 487, 487, 82, 497, 370, 105, 151, 474, 415, 323, 265, 105, 49, 534, 372, 324, 68, 102, 64, 535, 492, 492, 388, 518, 103, 28, 540, 72, 474, 37, 488, 349, 83, 551, 409, 278, 373, 108, 552, 79, 476, 152, 135, 36, 44, 396, 56, 326, 305, 554, 346, 257, 553, 528, 498, 352, 84, 34, 20, 19, 542, 534, 393, 348, 153, 91, 58, 418, 73, 387, 416, 388, 493, 125, 509, 242, 108, 489, 128, 29, 111, 89, 41, 36, 51, 411, 102, 158, 135, 326, 26, 387, 433, 62, 372, 79, 544, 351, 106, 512, 92, 20, 369, 518, 323, 493]
    #calcRecFinal: destinos: [324, 134, 110, 528, 328, 371, 86, 18, 478, 392, 86, 500, 304, 138, 129, 17, 110, 88, 80, 353, 69, 330, 39, 26, 555, 40, 417, 138, 104, 14, 154, 92, 515, 52, 330, 280, 115, 113, 14, 500, 266, 112, 67, 82, 22, 46, 544, 325, 395, 393, 131, 46, 44, 282, 434, 352, 301, 34, 418, 304, 66, 302, 529, 476, 303, 89, 16, 35, 27, 282, 23, 552, 61, 64, 58, 50, 159, 130, 433, 61, 51, 41, 42, 540, 511, 511, 509, 83, 498, 371, 129, 152, 475, 416, 346, 266, 106, 72, 535, 373, 325, 69, 126, 65, 557, 516, 514, 412, 541, 125, 29, 541, 73, 497, 38, 489, 350, 84, 575, 410, 279, 374, 109, 575, 80, 477, 153, 158, 37, 45, 419, 57, 350, 328, 555, 347, 280, 554, 529, 499, 353, 85, 35, 43, 43, 543, 557, 394, 349, 154, 113, 59, 419, 74, 409, 417, 411, 516, 126, 510, 265, 131, 512, 151, 52, 134, 90, 42, 59, 74, 412, 103, 159, 136, 327, 27, 410, 434, 63, 395, 81, 542, 374, 104, 488, 91, 19, 370, 517, 347, 515]
    #Terminó, recorrido final es: [485, 507, 508, 530, 531, 532, 533, 556, 558, 559, 560, 536, 513, 537, 538, 539, 561, 562, 563, 564, 565, 566, 567, 545, 568, 546, 574, 573, 572, 571, 570, 569, 547, 548, 549, 550, 527, 525, 526, 504, 503, 502, 524, 501, 523, 522, 521, 520, 519, 496, 495, 494, 470, 469, 491, 490, 466, 467, 443, 444, 96, 119, 120, 121, 144, 143, 142, 164, 165, 166, 167, 145, 168, 169, 170, 171, 172, 173, 150, 127, 149, 148, 147, 146, 122, 124, 123, 101, 100, 99, 98, 97, 75, 76, 77, 78, 54, 55, 53, 31, 32, 33, 11, 10, 9, 8, 30, 7, 6, 4, 3, 2, 1, 25, 24, 48, 47, 70, 71, 93, 94, 116, 117, 95, 118, 141, 139, 140, 163, 162, 185, 208, 209, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 217, 216, 215, 214, 213, 236, 237, 238, 239, 240, 241, 218, 219, 220, 243, 244, 245, 267, 268, 269, 246, 223, 247, 248, 249, 227, 226, 225, 224, 200, 222, 199, 198, 221, 197, 196, 174, 175, 176, 177, 201, 202, 203, 179, 180, 157, 156, 178, 155, 133, 132, 137, 161, 160, 183, 182, 181, 204, 205, 206, 184, 207, 229, 228, 230, 253, 252, 275, 276, 299, 298, 321, 322, 344, 345, 368, 366, 367, 390, 391, 413, 389, 435, 436, 414, 437, 460, 459, 458, 482, 483, 505, 481, 480, 479, 457, 456, 455, 454, 453, 452, 451, 450, 473, 472, 471, 448, 447, 446, 468, 445, 423, 424, 425, 426, 449, 427, 428, 429, 430, 431, 408, 407, 406, 405, 382, 384, 383, 359, 360, 313, 336, 337, 338, 339, 340, 362, 361, 385, 363, 386, 364, 341, 365, 342, 343, 319, 320, 296, 297, 273, 274, 251, 250, 272, 271, 270, 292, 293, 294, 295, 318, 317, 316, 315, 314, 291, 290, 289, 288, 287, 286, 263, 264, 262, 212, 211, 210, 233, 232, 231, 254, 277, 255, 256, 234, 235, 258, 259, 260, 261, 283, 306, 284, 285, 307, 308, 331, 332, 309, 310, 311, 312, 334, 335, 358, 357, 356, 333, 355, 354, 378, 379, 380, 381, 404, 403, 402, 401, 400, 377, 376, 375, 397, 398, 399, 422, 421, 420, 441, 442, 465, 464, 486, 463, 440, 439, 438, 462, 461, 484, 485] con costo: 7297.6648057753255
    #660.0972800254822 s

#algoritmo('TSPLIB','ALL_tsp/euc2d/rat783.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [761, 747, 498, 185, 172, 538, 472, 328, 758, 759, 712, 366, 683, 218, 748, 658, 770, 682, 420, 728, 714, 499, 421, 400, 601, 576, 737, 542, 459, 376, 371, 686, 405, 715, 580, 254, 120, 689, 719, 687, 662, 529, 372, 164, 550, 218, 112, 773, 407, 337, 240, 173, 345, 471, 463, 414, 295, 288, 284, 127, 264, 120, 765, 466, 574, 457, 661, 571, 476, 647, 522, 506, 479, 377, 224, 751, 444, 403, 378, 127, 494, 780, 303, 768, 728, 646, 520, 467, 320, 291, 255, 207, 532, 427, 332, 14, 699, 616, 753, 747, 515, 492, 707, 618, 565, 515, 374, 364, 260, 238, 234, 605, 446, 757, 685, 643, 589, 482, 288, 281, 263, 62, 370, 355, 203, 51, 692, 586, 488, 450, 368, 237, 143, 722, 490, 364, 201, 190, 724, 686, 664, 588, 569, 536, 472, 444, 402, 323, 309, 73, 702, 690, 651, 237, 116, 607, 581, 328, 325, 253, 250, 207, 655, 637, 447, 210, 743, 727, 705, 511, 435, 263, 5, 727, 720, 675, 391, 360, 323, 306, 293, 217, 190, 179, 61, 1, 509, 339, 267, 255, 232, 24, 669, 487, 365, 766, 382, 317, 750, 721, 619, 500, 389, 314, 204, 69, 8, 693, 660, 563, 436, 432, 342, 301, 220, 656, 590, 454, 427, 290, 573, 369, 256, 732, 688, 664, 642, 630, 614, 245, 208, 90, 26, 611, 513, 495, 425, 394, 189, 742, 705, 649, 645, 506, 474, 419, 384, 163, 113, 749, 456, 314, 183, 771, 722, 609, 541, 368, 354, 44, 31, 28, 741, 480, 252, 20, 718, 711, 671, 615, 612, 553, 530, 522, 504, 477, 455, 435, 373, 326, 307, 261, 112, 622, 584, 229, 135, 733, 723, 700, 514, 498, 422, 403, 317, 311, 287, 257, 241, 187, 151, 47, 567, 321, 52, 475, 286, 552, 526, 358, 73, 53, 654, 545, 503, 461, 320, 152, 91, 692, 607, 606, 580, 559, 534, 471, 397, 344, 214, 165, 714, 464, 434, 353, 333, 767, 593, 514, 390, 282, 269, 752, 620, 587, 583, 331, 653, 571, 505, 417, 351, 337, 72, 20, 756, 749, 735, 595, 558, 541, 533, 527, 484, 411, 380, 361, 357, 282, 258, 243, 225, 188, 134, 8, 437, 206, 157, 769, 691, 681, 680, 631, 566, 565, 458, 420, 396, 331, 329, 68, 635, 590, 438, 82, 744, 725, 707, 679, 775, 617, 525, 449, 421, 398, 389, 321, 295, 262, 14, 777, 624, 623, 564, 505, 338, 258, 198, 55, 615, 572, 232, 627, 624, 585, 554, 42, 740, 694, 648, 531, 422, 241, 69, 671, 528, 455, 451, 428, 423, 341, 163, 47, 3, 560, 486, 343, 327, 50, 708, 592, 516, 348, 726, 703, 702, 678, 646, 528, 449, 445, 300, 251, 213, 202, 711, 587, 539, 352, 92, 736, 694, 512, 477, 431, 298, 42, 621, 501, 464, 424, 404, 352, 318, 211, 1, 479, 417, 406, 630, 613, 546, 531, 380, 290, 19, 15, 667, 641, 593, 563, 562, 452, 370, 340, 330, 308, 296, 280, 104, 102, 665, 408, 22, 649, 468, 426, 172, 128, 603, 530, 453, 212, 763, 734, 695, 610, 583, 349, 725, 674, 480, 156, 49, 700, 597, 518, 409, 292, 264, 221, 151, 381, 369, 176, 570, 356, 347, 49, 582, 561, 276, 78, 3, 652, 600, 535, 502, 491, 434, 429, 297, 296, 183, 21, 739, 568, 465, 382, 584, 532, 279, 209, 586, 137, 53, 672, 180, 164, 105, 26, 666, 648, 629, 228, 44, 667, 638, 540, 346, 102, 754, 469, 385, 209, 107, 16, 782, 773, 760, 496, 414, 134, 697, 656, 620, 189, 59, 24, 730, 677, 408, 404, 271, 757, 402, 72, 285, 473, 50, 634, 612, 111, 762, 140, 197, 670, 760, 639, 450, 161, 608, 226, 319, 548, 378, 657, 104, 557, 521, 367, 363, 554, 523, 546, 32, 387, 395, 746, 781, 267, 626, 476, 704, 537, 107, 330, 473, 631, 405, 250, 567]
    #calcRecFinal: destinos: [778, 769, 510, 186, 185, 555, 499, 343, 776, 776, 729, 371, 697, 234, 755, 677, 780, 687, 430, 744, 729, 511, 442, 416, 605, 596, 754, 558, 474, 393, 392, 710, 426, 724, 596, 265, 137, 693, 733, 696, 673, 552, 391, 182, 555, 238, 128, 777, 425, 349, 259, 182, 347, 478, 486, 432, 312, 310, 298, 136, 280, 133, 772, 485, 582, 467, 684, 589, 494, 670, 543, 512, 490, 393, 243, 752, 462, 412, 398, 153, 497, 781, 322, 779, 751, 653, 524, 483, 334, 299, 277, 214, 549, 440, 350, 35, 720, 637, 767, 761, 529, 508, 719, 642, 575, 519, 379, 383, 283, 247, 254, 614, 459, 768, 706, 663, 610, 484, 308, 292, 272, 82, 372, 357, 220, 54, 713, 597, 493, 465, 388, 257, 152, 732, 504, 373, 210, 200, 742, 695, 673, 599, 579, 556, 482, 461, 416, 339, 324, 96, 716, 706, 666, 251, 133, 621, 604, 344, 332, 262, 269, 223, 660, 647, 451, 213, 758, 731, 718, 526, 439, 271, 10, 730, 721, 680, 411, 375, 325, 316, 306, 225, 206, 198, 68, 25, 525, 346, 286, 276, 240, 34, 684, 496, 376, 774, 390, 327, 763, 726, 641, 501, 397, 329, 208, 93, 30, 701, 675, 578, 445, 446, 362, 312, 228, 663, 598, 468, 447, 293, 581, 379, 268, 736, 709, 668, 650, 633, 633, 247, 224, 93, 32, 626, 518, 513, 443, 399, 203, 756, 709, 661, 655, 517, 489, 431, 395, 176, 119, 762, 457, 315, 201, 775, 739, 622, 556, 385, 360, 60, 36, 34, 746, 493, 261, 31, 735, 717, 678, 632, 627, 559, 551, 527, 516, 491, 460, 441, 383, 336, 316, 279, 116, 636, 600, 244, 149, 737, 741, 710, 524, 502, 438, 415, 319, 318, 301, 275, 245, 202, 162, 65, 574, 322, 56, 488, 299, 557, 533, 366, 91, 65, 662, 549, 510, 462, 324, 161, 96, 698, 617, 616, 594, 572, 551, 481, 407, 358, 223, 173, 717, 469, 448, 365, 345, 771, 604, 520, 401, 287, 281, 764, 625, 591, 602, 334, 654, 575, 521, 418, 354, 338, 92, 36, 764, 750, 745, 608, 570, 542, 548, 543, 485, 418, 381, 362, 367, 283, 268, 256, 233, 197, 149, 19, 448, 217, 169, 778, 698, 691, 690, 639, 578, 577, 466, 424, 399, 342, 341, 86, 643, 599, 443, 86, 753, 738, 713, 685, 783, 628, 544, 460, 428, 412, 392, 333, 311, 275, 22, 779, 640, 625, 566, 507, 340, 259, 200, 60, 629, 573, 233, 634, 638, 602, 562, 51, 748, 696, 659, 539, 433, 252, 78, 679, 544, 463, 454, 442, 429, 355, 169, 58, 10, 568, 492, 356, 335, 62, 716, 595, 523, 351, 740, 715, 708, 688, 651, 537, 456, 452, 307, 253, 221, 204, 712, 603, 540, 363, 103, 745, 703, 517, 483, 439, 309, 54, 628, 507, 470, 430, 413, 359, 326, 219, 15, 481, 419, 413, 636, 619, 550, 547, 386, 300, 30, 25, 674, 644, 606, 564, 579, 453, 377, 348, 336, 310, 304, 285, 113, 108, 672, 410, 35, 657, 478, 433, 180, 140, 609, 534, 458, 219, 770, 738, 699, 618, 585, 361, 734, 676, 489, 162, 52, 701, 598, 519, 410, 303, 265, 226, 156, 386, 374, 179, 577, 359, 350, 56, 592, 569, 277, 90, 5, 659, 601, 536, 509, 500, 437, 436, 304, 297, 188, 27, 743, 576, 475, 387, 591, 545, 284, 212, 594, 143, 58, 683, 186, 165, 108, 27, 669, 652, 632, 229, 55, 668, 640, 547, 353, 105, 755, 470, 388, 211, 111, 21, 783, 774, 765, 497, 415, 135, 704, 658, 623, 191, 61, 28, 731, 682, 409, 406, 272, 759, 400, 103, 315, 495, 59, 644, 613, 136, 766, 157, 191, 676, 772, 650, 440, 187, 635, 244, 335, 560, 396, 681, 119, 588, 535, 394, 384, 561, 503, 538, 16, 401, 423, 723, 782, 291, 645, 487, 665, 553, 153, 375, 508, 689, 441, 260, 611]
    #Terminó, recorrido final es: [6, 40, 38, 43, 66, 75, 87, 84, 79, 101, 106, 109, 114, 122, 142, 148, 145, 166, 158, 178, 192, 174, 168, 170, 181, 194, 199, 193, 171, 177, 131, 132, 141, 130, 129, 154, 167, 159, 150, 144, 124, 121, 126, 138, 118, 99, 85, 71, 64, 48, 77, 74, 89, 95, 98, 94, 83, 76, 63, 39, 29, 33, 9, 12, 18, 7, 2, 23, 17, 37, 57, 45, 41, 46, 70, 81, 67, 80, 100, 97, 88, 110, 115, 123, 117, 125, 146, 139, 147, 155, 160, 175, 184, 196, 195, 205, 216, 215, 235, 236, 242, 266, 270, 289, 305, 313, 302, 294, 278, 273, 274, 249, 248, 227, 222, 231, 230, 246, 239, 4, 11, 13, 6] con costo: 9553.54759243741
    #6511.678591012955 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/rat99.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [56, 54, 73, 59, 47, 73, 1, 62, 83, 49, 46, 95, 49, 53, 1, 79, 91, 64, 74, 93, 71, 66, 57, 80, 72, 75, 97, 69, 50, 82, 86, 65, 2, 96, 38, 76, 84, 90, 77, 71, 69, 38, 98, 61, 51, 40, 87, 58, 46, 89, 86, 56, 52, 91, 39, 67, 77, 61, 92, 79]
    #calcRecFinal: destinos: [64, 63, 82, 60, 55, 74, 10, 63, 84, 60, 55, 96, 50, 54, 2, 88, 93, 65, 75, 94, 72, 67, 58, 81, 81, 76, 98, 78, 51, 83, 87, 66, 10, 97, 48, 85, 85, 99, 78, 80, 70, 39, 99, 62, 52, 48, 88, 59, 47, 90, 95, 57, 53, 92, 40, 68, 68, 70, 94, 89]
    #Terminó, recorrido final es: [9, 18, 17, 16, 15, 23, 24, 25, 26, 27, 35, 36, 44, 45, 43, 34, 33, 42, 41, 32, 31, 30, 29, 37, 28, 19, 20, 21, 22, 13, 12, 11, 3, 4, 14, 5, 6, 7, 8, 9] con costo: 1225.778247874619
    #0.10485196113586426 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/rd100.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [36, 3, 81, 60, 45, 8, 15, 65, 55, 58, 3, 39, 51, 16, 36, 47, 65, 35, 34, 58, 29, 19, 32, 5, 13, 1, 38, 62, 30, 60, 50, 68, 11, 54, 30, 48, 59, 94, 17, 44, 83, 66, 7, 79, 24, 66, 18, 42, 24, 13, 6, 29, 7, 70, 1, 31, 17, 77, 22, 5, 53, 12, 6, 77, 25, 4, 9, 51, 10, 14, 12, 39, 20, 26, 63, 15, 27, 75, 28, 21, 46, 50, 46, 21, 59, 20, 84, 63, 86, 94, 11, 54, 4, 47, 27, 69, 40]
    #calcRecFinal: destinos: [64, 10, 91, 71, 52, 71, 87, 91, 96, 88, 33, 55, 68, 45, 57, 57, 81, 44, 41, 76, 100, 37, 78, 22, 49, 18, 70, 87, 80, 69, 92, 83, 61, 73, 48, 80, 93, 98, 38, 100, 90, 98, 40, 99, 42, 76, 62, 43, 25, 67, 53, 35, 34, 72, 8, 84, 72, 95, 41, 61, 99, 14, 79, 93, 43, 32, 26, 90, 33, 85, 85, 96, 78, 74, 67, 97, 92, 82, 37, 49, 73, 56, 56, 75, 95, 74, 88, 86, 97, 16, 52, 19, 82, 64, 28, 9, 31]
    #Terminó, recorrido final es: [23, 89, 2, 23] con costo: 8555.325188426214
    #0.4091479778289795 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/rd400.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [156, 290, 168, 48, 16, 277, 89, 6, 277, 210, 112, 116, 97, 56, 133, 180, 49, 42, 365, 132, 292, 300, 112, 300, 175, 55, 50, 118, 150, 74, 23, 92, 259, 98, 174, 24, 58, 258, 57, 166, 180, 156, 193, 90, 157, 209, 65, 147, 83, 203, 80, 138, 38, 18, 20, 184, 39, 213, 134, 105, 60, 34, 29, 287, 35, 122, 8, 272, 3, 29, 86, 98, 337, 7, 46, 61, 187, 11, 33, 37, 46, 84, 172, 187, 331, 45, 126, 314, 230, 10, 82, 140, 21, 82, 217, 15, 76, 8, 118, 72, 182, 243, 105, 205, 236, 7, 245, 66, 229, 119, 110, 303, 36, 37, 162, 114, 338, 138, 171, 40, 155, 108, 146, 33, 147, 273, 25, 31, 79, 229, 35, 303, 63, 70, 160, 25, 110, 62, 203, 202, 44, 356, 257, 68, 4, 125, 149, 351, 320, 102, 276, 199, 232, 330, 161, 117, 109, 11, 50, 145, 107, 89, 114, 346, 155, 104, 19, 54, 207, 178, 143, 57, 193, 148, 1, 162, 202, 75, 69, 164, 261, 14, 320, 255, 113, 170, 144, 13, 291, 1, 103, 135, 251, 32, 92, 158, 12, 90, 165, 27, 111, 117, 27, 69, 164, 153, 59, 226, 5, 52, 85, 47, 56, 52, 292, 67, 131, 129, 99, 101, 32, 17, 270, 234, 220, 183, 54, 43, 225, 268, 299, 40, 65, 116, 231, 183, 36, 78, 93, 299, 93, 254, 286, 222, 178, 159, 42, 318, 22, 22, 158, 195, 113, 5, 224, 60, 63, 200, 244, 51, 13, 78, 106, 81, 254, 130, 373, 20, 94, 21, 154, 198, 226, 9, 2, 70, 132, 134, 283, 234, 127, 77, 109, 96, 6, 3, 212, 214, 95, 71, 327, 278, 204, 26, 152, 139, 14, 248, 235, 298, 127, 12, 235, 15, 87, 216, 48, 31, 166, 85, 237, 102, 197, 41, 2, 39, 274, 251, 51, 53, 74, 87, 88, 289, 167, 230, 53, 44, 120, 215, 120, 73, 213, 133, 115, 91, 45, 121, 88, 77, 182, 145, 47, 354, 26, 190, 101, 130, 302, 41, 10, 73, 310, 18, 80, 248, 260, 107, 172, 191, 380, 323, 151, 62, 309, 305, 317, 79, 282, 324, 24, 373, 284, 377, 34, 297, 192, 143, 257, 38, 206, 137, 348, 197, 323, 209, 169, 64, 280, 340, 336]
    #calcRecFinal: destinos: [192, 352, 380, 191, 393, 351, 268, 58, 322, 319, 246, 314, 219, 137, 352, 204, 136, 298, 399, 272, 322, 307, 264, 339, 319, 239, 152, 391, 294, 140, 124, 157, 313, 288, 381, 240, 136, 286, 96, 371, 296, 366, 335, 121, 273, 265, 366, 371, 394, 275, 337, 150, 239, 394, 227, 271, 304, 379, 364, 171, 329, 161, 163, 392, 271, 265, 128, 332, 361, 174, 302, 240, 377, 64, 325, 94, 243, 363, 324, 326, 86, 381, 188, 356, 389, 169, 131, 390, 346, 146, 270, 188, 30, 106, 233, 30, 341, 91, 316, 154, 217, 340, 194, 357, 263, 341, 363, 215, 259, 279, 148, 310, 119, 75, 225, 315, 399, 177, 294, 55, 236, 354, 189, 194, 290, 339, 163, 125, 375, 349, 123, 334, 297, 364, 196, 200, 293, 142, 358, 214, 181, 389, 367, 253, 201, 242, 335, 374, 397, 261, 397, 395, 383, 387, 256, 186, 264, 100, 83, 175, 160, 306, 392, 379, 211, 111, 384, 289, 361, 309, 370, 179, 338, 293, 4, 313, 285, 220, 316, 186, 315, 353, 396, 266, 142, 218, 177, 19, 370, 9, 179, 266, 276, 362, 358, 327, 126, 232, 238, 321, 386, 282, 218, 76, 173, 185, 224, 281, 255, 353, 359, 206, 275, 141, 374, 205, 376, 247, 104, 350, 59, 67, 386, 357, 231, 283, 400, 376, 287, 317, 385, 168, 84, 390, 291, 372, 279, 208, 330, 360, 135, 345, 347, 304, 347, 308, 391, 355, 367, 66, 344, 387, 350, 281, 245, 345, 321, 331, 252, 165, 173, 198, 326, 241, 362, 307, 393, 274, 333, 301, 184, 208, 228, 201, 139, 81, 332, 267, 372, 395, 185, 284, 246, 103, 151, 207, 382, 285, 369, 343, 344, 348, 210, 167, 296, 385, 141, 318, 241, 384, 153, 43, 333, 301, 312, 336, 124, 242, 368, 159, 258, 383, 369, 343, 122, 295, 368, 311, 238, 95, 219, 278, 252, 400, 328, 342, 398, 128, 247, 227, 129, 212, 342, 249, 222, 149, 256, 375, 244, 199, 233, 181, 123, 360, 328, 308, 396, 311, 325, 71, 189, 382, 334, 305, 228, 355, 280, 196, 267, 23, 97, 72, 288, 329, 263, 365, 17, 349, 216, 61, 306, 16, 211, 260, 295, 170, 359, 99, 108, 249, 253, 115, 312, 398, 49, 100, 237, 68, 195, 190, 144]
    #Terminó, recorrido final es: [221, 262, 388, 223, 250, 269, 28, 378, 176, 221] con costo: 16394.5321452711
    #266.9941339492798 s
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/rl11849.tsp') #
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/rl1304.tsp') #
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/rl1323.tsp') #
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/rl1889.tsp') #
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/rl5915.tsp') #
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/rl5934.tsp') #
#algoritmo('TSPLIB','ALL_tsp/euc2d/st70.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [33, 26, 30, 43, 17, 40, 31, 9, 2, 35, 3, 54, 25, 23, 7, 33, 7, 37, 39, 26, 23, 40, 50, 52, 5, 6, 1, 2, 49, 12, 22, 12, 38, 25, 20, 14, 24, 15, 27, 18, 6, 3, 59, 22, 13, 16, 29, 5, 63, 10, 17, 8, 4, 1, 44, 15, 31, 13, 37, 21, 27, 28, 53]
    #calcRecFinal: destinos: [62, 55, 44, 68, 41, 46, 35, 61, 19, 69, 8, 62, 39, 36, 19, 54, 32, 50, 61, 49, 47, 45, 58, 60, 53, 42, 16, 4, 55, 34, 38, 21, 59, 45, 30, 20, 57, 57, 46, 42, 41, 32, 63, 66, 70, 47, 70, 10, 66, 52, 43, 28, 18, 36, 68, 24, 69, 29, 58, 34, 9, 14, 60]
    #Terminó, recorrido final es: [11, 64, 65, 51, 56, 67, 48, 11] con costo: 643.1226501326508
    #0.02602696418762207 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/ts225.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [214, 213, 212, 211, 209, 208, 207, 206, 204, 203, 202, 201, 199, 198, 197, 196, 194, 193, 192, 191, 189, 188, 187, 186, 184, 183, 182, 181, 179, 178, 177, 176, 174, 173, 172, 171, 169, 168, 167, 166, 164, 163, 162, 161, 159, 158, 157, 156, 154, 153, 152, 151, 149, 148, 147, 146, 144, 143, 142, 141, 139, 138, 137, 136, 134, 133, 132, 131, 129, 128, 127, 126, 117, 116, 115, 114, 113, 112, 111, 110, 109, 108, 107, 106, 105, 104, 103, 102, 101, 101, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 26, 25, 24, 23, 22, 21, 20, 19, 19, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 1, 211, 206, 191, 186, 181, 166, 161, 156, 114, 196, 176, 14, 131, 171]
    #calcRecFinal: destinos: [215, 214, 213, 212, 210, 209, 208, 207, 205, 204, 203, 202, 200, 199, 198, 197, 195, 194, 193, 192, 190, 189, 188, 187, 185, 184, 183, 182, 180, 179, 178, 177, 175, 174, 173, 172, 170, 169, 168, 167, 165, 164, 163, 162, 160, 159, 158, 157, 155, 154, 153, 152, 150, 149, 148, 147, 145, 144, 143, 142, 140, 139, 138, 137, 135, 134, 133, 132, 130, 129, 128, 127, 118, 117, 116, 115, 215, 113, 112, 111, 110, 109, 210, 107, 106, 105, 104, 103, 205, 102, 195, 94, 93, 92, 91, 90, 190, 88, 87, 86, 85, 84, 185, 82, 81, 80, 79, 78, 201, 77, 175, 75, 74, 73, 72, 71, 170, 69, 68, 67, 66, 65, 165, 63, 62, 61, 60, 59, 160, 57, 56, 55, 54, 53, 155, 52, 150, 50, 49, 48, 47, 46, 145, 44, 43, 42, 41, 40, 140, 38, 37, 36, 35, 34, 135, 32, 31, 30, 29, 28, 151, 130, 146, 25, 24, 23, 22, 21, 141, 20, 18, 17, 16, 15, 136, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 126, 2, 89, 83, 70, 64, 58, 45, 39, 33, 118, 200, 180, 18, 27, 108]
    #Terminó, recorrido final es: [224, 225, 125, 124, 123, 122, 121, 120, 119, 220, 219, 218, 217, 216, 95, 96, 97, 98, 99, 100, 221, 222, 223, 224] con costo: 134920.310216783
    #5.135664939880371 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/tsp225.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [124, 41, 21, 20, 10, 8, 125, 39, 9, 46, 18, 110, 176, 32, 127, 128, 15, 13, 209, 194, 43, 197, 184, 181, 171, 170, 169, 164, 164, 145, 144, 142, 123, 122, 110, 102, 100, 89, 69, 54, 52, 42, 40, 37, 36, 26, 23, 18, 17, 12, 11, 81, 179, 176, 172, 143, 134, 130, 129, 126, 121, 58, 44, 38, 31, 28, 25, 7, 5, 94, 174, 173, 4, 167, 120, 119, 118, 84, 111, 65, 49, 93, 168, 77, 74, 166, 151, 31, 150, 194, 168, 116, 27, 27, 53, 118, 14, 2, 174, 119, 84, 78, 64, 60, 48, 4, 117, 159, 192, 191, 189, 77, 25, 177, 165, 162, 158, 152, 148, 140, 137, 135, 134, 133, 106, 88, 87, 85, 78, 74, 19, 14, 6, 82, 193, 159, 158, 156, 155, 147, 143, 141, 136, 136, 129, 105, 101, 99, 98, 96, 90, 66, 65, 63, 62, 56, 30, 29, 24, 16, 2, 150, 149, 105, 51, 51, 93, 45, 114, 80, 131, 121, 82, 80, 75, 34, 192, 130, 120, 116, 115, 71, 133, 199, 49, 216, 153, 113, 103, 68, 33, 33, 217, 161, 154, 139, 138, 108, 107, 104, 91, 72, 71, 60, 58, 55, 28, 146, 190, 30, 225, 215, 68, 45, 88, 83, 50, 96, 157, 188, 5, 109]
    #calcRecFinal: destinos: [125, 42, 22, 21, 11, 9, 126, 40, 10, 195, 203, 112, 178, 206, 128, 222, 22, 23, 221, 195, 44, 198, 185, 182, 172, 171, 170, 213, 165, 146, 145, 201, 124, 123, 111, 103, 101, 90, 70, 55, 53, 43, 41, 38, 37, 204, 24, 19, 20, 13, 12, 95, 180, 177, 173, 201, 135, 131, 132, 127, 122, 61, 46, 39, 32, 29, 26, 8, 6, 221, 181, 182, 198, 214, 184, 185, 187, 86, 113, 67, 50, 209, 169, 79, 76, 167, 214, 216, 212, 218, 212, 117, 189, 188, 54, 186, 16, 59, 180, 186, 210, 97, 112, 62, 193, 197, 187, 161, 224, 205, 205, 217, 208, 178, 166, 163, 163, 153, 149, 141, 138, 183, 215, 190, 107, 89, 210, 86, 79, 75, 203, 15, 7, 85, 196, 160, 213, 157, 156, 148, 144, 142, 183, 137, 222, 220, 102, 100, 99, 97, 91, 67, 66, 64, 63, 57, 206, 202, 208, 17, 47, 151, 152, 106, 57, 207, 94, 48, 115, 95, 211, 175, 83, 81, 76, 35, 196, 211, 175, 223, 223, 73, 199, 224, 207, 219, 154, 114, 104, 69, 35, 34, 219, 162, 155, 140, 139, 109, 108, 220, 92, 73, 72, 61, 59, 56, 204, 147, 225, 202, 47, 132, 70, 218, 87, 92, 52, 98, 160, 191, 36, 179]
    #Terminó, recorrido final es: [3, 200, 1, 3] con costo: 4152.467909607948
    #3.2796378135681152 s
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/u1060.tsp') #
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/u1432.tsp') #
#algoritmo('TSPLIB','ALL_tsp/euc2d/u159.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [28, 1, 32, 30, 59, 52, 2, 29, 25, 108, 91, 51, 111, 106, 58, 54, 19, 105, 27, 76, 74, 40, 88, 75, 71, 62, 61, 60, 56, 55, 49, 35, 33, 18, 44, 43, 42, 41, 24, 23, 21, 3, 1, 149, 148, 108, 90, 89, 87, 53, 36, 26, 18, 16, 15, 6, 122, 121, 120, 119, 117, 116, 115, 114, 112, 111, 109, 92, 89, 87, 70, 50, 47, 46, 45, 15, 6, 2, 146, 143, 95, 57, 152, 146, 144, 143, 104, 103, 102, 101, 100, 99, 98, 97, 85, 84, 83, 82, 81, 80, 79, 78, 71, 69, 68, 67, 66, 65, 64, 39, 36, 34, 22, 20, 93, 48, 73, 38, 147, 118, 114, 77, 86, 55, 32]
    #calcRecFinal: destinos: [29, 3, 33, 31, 60, 53, 159, 30, 26, 110, 92, 52, 113, 107, 59, 63, 31, 106, 28, 77, 75, 41, 95, 76, 73, 63, 62, 61, 57, 56, 50, 37, 34, 20, 45, 44, 43, 42, 25, 24, 22, 4, 159, 150, 150, 109, 94, 94, 96, 54, 38, 27, 19, 17, 16, 152, 123, 122, 121, 120, 118, 117, 116, 115, 113, 112, 110, 93, 90, 88, 72, 51, 48, 47, 46, 17, 153, 4, 149, 145, 96, 58, 153, 147, 145, 144, 105, 104, 103, 102, 101, 100, 99, 98, 86, 85, 84, 83, 82, 81, 80, 79, 72, 70, 69, 68, 67, 66, 65, 40, 37, 35, 23, 21, 107, 49, 74, 39, 148, 119, 123, 78, 97, 64, 91]
    #Terminó, recorrido final es: [125, 127, 129, 124, 130, 128, 131, 132, 133, 134, 135, 136, 137, 138, 140, 139, 141, 142, 14, 13, 11, 12, 10, 9, 8, 7, 151, 155, 154, 5, 156, 157, 158, 126, 125] con costo: 45636.673810316475
    #4.971535921096802 s
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/u1817.tsp') #
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/u2152.tsp') #
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/u2319.tsp') #
#algoritmo('TSPLIB','ALL_tsp/euc2d/u574.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [58, 435, 11, 279, 128, 66, 280, 436, 59, 57, 327, 262, 14, 497, 299, 524, 145, 497, 128, 26, 53, 144, 61, 111, 112, 48, 56, 25, 23, 12, 14, 466, 94, 215, 189, 529, 327, 180, 55, 525, 329, 36, 298, 15, 181, 60, 96, 134, 291, 186, 356, 520, 473, 167, 153, 148, 168, 29, 13, 186, 44, 470, 129, 481, 207, 204, 233, 60, 5, 189, 62, 16, 555, 41, 17, 266, 263, 86, 26, 412, 472, 141, 230, 217, 212, 282, 471, 82, 356, 96, 310, 18, 520, 37, 483, 510, 309, 19, 174, 173, 480, 437, 571, 503, 48, 565, 564, 487, 464, 229, 319, 568, 154, 97, 28, 315, 470, 306, 457, 143, 134, 465, 166, 232, 494, 297, 21, 569, 185, 87, 6, 32, 140, 31, 218, 527, 253, 208, 243, 206, 552, 556, 18, 498, 79, 495, 534, 296, 33, 559, 318, 426, 252, 499, 290, 227, 457, 557, 262, 482, 425, 249, 204, 555, 61, 135, 146, 242, 170, 166, 522, 196, 406, 199, 40, 237, 205, 406, 425, 195, 113, 286, 198, 467, 542, 561, 108, 477, 412, 65, 49, 509, 210, 75, 357, 192, 184, 47, 102, 99, 72, 71, 70, 68, 201, 183, 24, 468, 39, 547, 82, 118, 546, 155, 89, 107, 69, 324, 310, 178, 138, 322, 532, 287, 274, 530, 124, 63, 462, 136, 353, 530, 550, 460, 484, 131, 541, 147, 501, 488, 486, 309, 302, 301, 300, 202, 100, 536, 552, 293, 248, 238, 228, 223, 220, 187, 182, 181, 179, 175, 157, 105, 93, 90, 88, 85, 76, 73, 36, 2, 460, 132, 45, 150, 203, 315, 501, 131, 266, 119, 22, 540, 539, 114, 202, 91, 277, 517, 226, 338, 551, 200, 31, 282, 44, 85, 115, 506, 571, 437, 321, 275, 557, 191, 467, 283, 482, 438, 572, 569, 523, 171, 435, 407, 246, 1, 518, 190, 288, 8, 548, 154, 114, 111, 413, 235, 267, 160, 479, 142, 289, 5, 544, 67, 515, 338, 235, 160, 103, 506, 478, 115, 97, 533, 325, 513, 52, 295, 28, 274, 232, 328, 119, 339, 562, 251, 39, 125, 109, 547, 510, 138, 561, 516, 221, 42, 226, 354, 289, 477, 486, 461, 458, 353, 198, 193, 536, 339, 321, 223, 216, 178, 67, 116, 73, 8, 3, 526, 316, 286, 228, 175, 161, 158, 148, 121, 105, 23, 15, 303, 478, 170, 162, 157, 102, 92, 83, 34, 539, 537, 2, 508, 475, 324, 313, 307, 251, 245, 224, 195, 173, 51, 305, 300, 54, 213, 9, 318, 493, 490, 491, 1, 566, 564, 504, 493, 490, 211, 188, 65, 151, 56, 414, 295, 526, 240, 153, 244, 98, 280, 95, 41, 505, 464, 11, 150, 511, 220, 164, 47]
    #calcRecFinal: destinos: [59, 440, 12, 281, 130, 127, 281, 440, 78, 58, 328, 278, 529, 499, 303, 525, 146, 498, 129, 165, 54, 145, 63, 121, 113, 50, 80, 165, 27, 13, 545, 475, 95, 216, 194, 531, 330, 285, 57, 528, 330, 38, 299, 17, 285, 78, 139, 136, 292, 208, 357, 522, 474, 213, 168, 151, 169, 30, 546, 187, 46, 474, 130, 507, 238, 206, 234, 76, 7, 190, 64, 222, 558, 80, 21, 268, 278, 87, 27, 414, 473, 142, 231, 218, 215, 308, 472, 84, 358, 100, 312, 19, 521, 38, 485, 511, 313, 20, 177, 176, 481, 439, 573, 504, 49, 567, 566, 489, 465, 231, 320, 570, 156, 99, 30, 317, 471, 307, 458, 144, 135, 466, 214, 234, 496, 298, 22, 570, 209, 88, 7, 33, 141, 34, 219, 528, 254, 209, 244, 207, 554, 559, 20, 500, 81, 496, 535, 305, 35, 560, 319, 427, 254, 500, 291, 241, 459, 560, 263, 484, 426, 250, 205, 556, 62, 137, 147, 243, 172, 167, 523, 197, 408, 200, 43, 240, 239, 407, 427, 197, 120, 294, 250, 468, 543, 563, 109, 513, 413, 66, 51, 512, 211, 77, 358, 193, 185, 50, 104, 101, 74, 72, 126, 71, 248, 184, 25, 469, 42, 549, 83, 123, 550, 164, 90, 108, 70, 326, 311, 210, 140, 323, 533, 288, 276, 532, 125, 64, 463, 137, 355, 531, 551, 461, 485, 132, 543, 149, 503, 489, 488, 312, 304, 302, 311, 203, 101, 538, 553, 294, 249, 239, 230, 225, 221, 188, 183, 182, 180, 177, 159, 107, 94, 91, 89, 86, 77, 75, 37, 4, 463, 133, 46, 152, 245, 316, 502, 133, 267, 123, 219, 541, 542, 120, 247, 92, 279, 518, 241, 372, 553, 201, 32, 283, 45, 110, 117, 507, 572, 438, 323, 276, 558, 192, 469, 284, 483, 439, 573, 574, 524, 172, 436, 408, 247, 568, 519, 191, 293, 10, 549, 155, 118, 112, 415, 236, 268, 161, 480, 143, 290, 6, 545, 69, 516, 340, 237, 163, 104, 508, 514, 116, 98, 535, 326, 514, 53, 296, 29, 275, 233, 329, 122, 372, 563, 252, 40, 126, 110, 548, 512, 139, 562, 517, 222, 43, 227, 355, 292, 515, 487, 462, 459, 354, 199, 194, 537, 340, 322, 224, 217, 179, 68, 117, 74, 9, 4, 527, 317, 287, 229, 176, 162, 159, 149, 122, 106, 24, 16, 304, 479, 171, 163, 158, 103, 93, 84, 35, 540, 538, 3, 509, 476, 325, 314, 308, 253, 246, 225, 196, 174, 52, 306, 301, 55, 214, 10, 320, 495, 491, 492, 574, 567, 565, 505, 494, 492, 212, 284, 127, 152, 79, 415, 297, 534, 242, 156, 521, 124, 277, 106, 81, 502, 476, 554, 169, 519, 236, 544, 314]
    #Terminó, recorrido final es: [341, 343, 344, 371, 370, 345, 346, 347, 369, 368, 367, 350, 352, 351, 349, 348, 403, 404, 416, 417, 418, 419, 420, 421, 422, 423, 424, 428, 429, 430, 432, 431, 434, 433, 446, 445, 447, 443, 444, 441, 442, 453, 452, 455, 454, 451, 456, 450, 448, 449, 401, 402, 400, 399, 398, 397, 396, 395, 256, 255, 257, 258, 259, 260, 261, 391, 392, 393, 394, 405, 409, 410, 411, 359, 360, 361, 362, 363, 364, 365, 366, 376, 377, 378, 375, 374, 373, 379, 380, 381, 382, 385, 383, 384, 386, 387, 388, 390, 389, 265, 264, 273, 272, 271, 270, 269, 334, 333, 332, 331, 335, 336, 337, 342, 341] con costo: 39544.5875571151
#253.6720838546753 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/u724.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [3, 396, 119, 699, 85, 14, 134, 254, 15, 365, 708, 182, 84, 293, 129, 135, 204, 31, 670, 465, 472, 649, 118, 578, 707, 35, 208, 202, 207, 352, 292, 190, 671, 2, 128, 693, 577, 307, 212, 171, 5, 3, 259, 581, 343, 17, 216, 293, 691, 18, 166, 34, 419, 271, 37, 30, 395, 165, 66, 694, 643, 562, 300, 222, 137, 188, 695, 493, 340, 226, 191, 161, 90, 570, 35, 143, 220, 244, 203, 494, 297, 659, 586, 205, 498, 412, 384, 196, 292, 270, 297, 163, 64, 335, 557, 654, 626, 582, 38, 673, 9, 476, 670, 702, 562, 196, 491, 221, 655, 309, 143, 568, 556, 294, 68, 6, 161, 78, 193, 640, 185, 97, 320, 582, 326, 131, 234, 672, 332, 400, 327, 359, 602, 245, 63, 664, 550, 542, 480, 471, 471, 405, 365, 309, 258, 216, 215, 211, 209, 134, 106, 69, 37, 32, 715, 618, 154, 147, 41, 33, 360, 311, 712, 647, 644, 640, 573, 569, 567, 521, 486, 420, 358, 347, 344, 307, 306, 304, 300, 229, 210, 193, 159, 154, 153, 152, 122, 118, 63, 22, 6, 94, 593, 405, 173, 11, 189, 188, 259, 181, 322, 93, 130, 560, 328, 341, 217, 139, 132, 316, 138, 459, 288, 629, 455, 314, 296, 232, 28, 700, 693, 26, 698, 590, 353, 157, 149, 136, 107, 65, 51, 10, 530, 480, 159, 123, 347, 16, 479, 16, 346, 395, 558, 599, 335, 529, 383, 214, 130, 228, 461, 692, 288, 213, 230, 296, 413, 227, 45, 142, 403, 343, 409, 250, 211, 529, 451, 205, 127, 645, 606, 32, 555, 312, 88, 217, 667, 576, 478, 303, 230, 621, 508, 418, 302, 298, 285, 274, 21, 588, 158, 146, 87, 29, 80, 27, 624, 619, 612, 248, 238, 223, 219, 481, 694, 526, 295, 398, 111, 469, 516, 353, 215, 194, 83, 663, 661, 626, 622, 615, 612, 559, 310, 167, 94, 66, 21, 23, 5, 704, 579, 568, 566, 466, 399, 233, 129, 29, 528, 468, 210, 639, 273, 416, 142, 200, 237, 411, 239, 546, 48, 324, 414, 413, 241, 386, 591, 364, 295, 70, 68, 567, 546, 523, 416, 410, 361, 355, 312, 273, 267, 266, 248, 235, 223, 190, 88, 52, 697, 614, 175, 145, 722, 707, 669, 669, 657, 657, 651, 624, 615, 611, 610, 593, 592, 583, 518, 498, 484, 455, 454, 341, 324, 314, 306, 303, 277, 276, 269, 266, 225, 220, 189, 179, 177, 162, 158, 157, 150, 137, 136, 121, 97, 87, 79, 71, 52, 43, 40, 31, 24, 19, 8, 542, 507, 492, 350, 342, 329, 660, 642, 641, 628, 199, 8, 7, 40, 327, 611, 688, 674, 552, 411, 140, 197, 547, 14, 668, 33, 478, 181, 625, 338, 656, 369, 289, 323, 393, 186, 75, 56, 530, 235, 178, 174, 202, 176, 42, 655, 558, 139, 155, 93, 450, 144, 701, 621, 502, 285, 351, 199, 616, 249, 50, 668, 22, 595, 586, 579, 540, 516, 464, 454, 305, 245, 192, 654, 156, 501, 493, 276, 213, 141, 128, 126, 120, 89, 700, 50, 566, 329, 280, 279, 78, 26, 23, 96, 187, 554, 110, 73, 57, 125, 366, 100, 675, 198, 561, 531, 403, 451, 392, 383, 446, 429, 369, 378, 543, 517, 460, 418, 316, 311, 627, 559, 192, 71, 708, 617, 81, 43, 41, 9, 642, 354, 354, 338, 336, 318, 304, 268, 206, 195, 74, 64, 30, 2, 444, 431, 376, 250, 209, 666, 634, 146, 120, 706, 617, 233, 598, 580, 496, 362, 360, 287, 286, 283, 249, 247, 156, 123, 368, 447, 438, 379, 422, 430, 370, 436, 421, 390, 388, 439, 425, 449, 440, 386, 381, 434, 426, 387, 53, 260, 61, 54, 75, 76, 185, 260, 533, 62, 532, 262, 437, 433, 7, 390, 375, 367, 443, 434, 440, 425, 427, 423, 387, 261, 56, 124, 53, 169, 76, 716, 535, 533, 532, 263, 186, 72, 125, 113, 58, 553, 367, 421, 380, 448, 377, 368, 432, 445, 208, 325, 270, 321, 486, 402, 92, 59, 476, 184, 675, 570, 177, 712, 400, 544, 506, 629, 246, 147, 552, 525, 200, 212, 290, 653, 677, 109, 484, 253, 705, 697, 594, 537, 457, 272, 366, 42, 442, 415, 631, 326, 310, 349, 72, 372, 98, 625, 340, 127]
    #calcRecFinal: destinos: [713, 401, 651, 711, 86, 99, 135, 513, 17, 410, 718, 560, 86, 408, 603, 148, 243, 34, 684, 466, 474, 650, 649, 585, 718, 36, 519, 203, 254, 514, 404, 257, 682, 724, 131, 703, 585, 318, 506, 650, 714, 4, 274, 589, 345, 100, 346, 409, 696, 99, 605, 65, 420, 275, 39, 38, 401, 605, 67, 723, 645, 596, 485, 515, 141, 550, 699, 496, 495, 519, 538, 620, 101, 588, 666, 163, 228, 255, 534, 500, 488, 662, 589, 207, 500, 458, 385, 257, 294, 538, 477, 164, 83, 339, 572, 665, 676, 583, 47, 682, 702, 477, 680, 711, 563, 201, 492, 514, 683, 315, 155, 572, 557, 408, 69, 15, 610, 117, 194, 643, 575, 695, 321, 587, 355, 282, 330, 681, 333, 407, 328, 483, 604, 255, 104, 679, 569, 548, 490, 473, 472, 406, 404, 402, 265, 224, 229, 522, 521, 148, 116, 90, 44, 46, 724, 622, 160, 602, 80, 36, 364, 357, 713, 652, 646, 647, 574, 571, 578, 526, 487, 459, 483, 497, 345, 319, 317, 356, 301, 231, 252, 267, 638, 165, 608, 153, 171, 122, 84, 691, 720, 687, 596, 473, 175, 704, 544, 545, 272, 182, 331, 689, 603, 565, 334, 342, 227, 601, 133, 362, 607, 460, 505, 634, 464, 363, 487, 320, 95, 709, 698, 95, 703, 606, 504, 636, 168, 151, 117, 82, 59, 710, 565, 488, 619, 124, 352, 20, 482, 19, 350, 396, 595, 601, 336, 541, 385, 510, 284, 331, 462, 696, 289, 214, 322, 417, 461, 241, 686, 144, 474, 344, 414, 253, 251, 540, 453, 236, 132, 652, 608, 684, 556, 313, 89, 218, 676, 581, 481, 305, 231, 635, 509, 419, 501, 299, 286, 278, 24, 590, 609, 149, 98, 39, 671, 91, 660, 638, 628, 515, 239, 224, 244, 482, 722, 527, 298, 406, 115, 470, 518, 508, 218, 195, 85, 664, 665, 667, 623, 618, 614, 575, 315, 172, 96, 108, 25, 690, 705, 721, 584, 576, 574, 467, 407, 234, 283, 44, 531, 469, 522, 648, 275, 417, 166, 237, 238, 453, 240, 547, 81, 325, 462, 458, 243, 394, 598, 489, 299, 104, 109, 577, 549, 524, 457, 415, 363, 356, 398, 277, 268, 269, 512, 242, 225, 545, 102, 112, 721, 630, 184, 173, 723, 719, 681, 677, 661, 658, 653, 633, 616, 613, 632, 600, 594, 607, 520, 507, 485, 456, 456, 349, 348, 361, 308, 317, 279, 281, 524, 523, 226, 513, 258, 180, 179, 636, 160, 609, 151, 138, 150, 641, 103, 102, 672, 116, 60, 690, 92, 82, 25, 20, 701, 543, 510, 497, 351, 495, 333, 663, 644, 648, 656, 236, 717, 710, 48, 334, 631, 689, 683, 561, 412, 162, 198, 564, 18, 678, 686, 479, 183, 627, 339, 659, 371, 302, 337, 394, 551, 110, 57, 534, 240, 183, 176, 204, 178, 49, 674, 597, 140, 164, 688, 452, 145, 709, 637, 503, 287, 499, 201, 633, 511, 51, 680, 692, 597, 587, 580, 541, 517, 465, 470, 308, 246, 247, 662, 620, 504, 494, 280, 509, 152, 133, 167, 639, 103, 720, 108, 573, 332, 281, 282, 107, 27, 28, 687, 551, 564, 114, 74, 58, 126, 375, 101, 678, 527, 563, 537, 475, 452, 393, 384, 447, 430, 370, 379, 548, 520, 463, 467, 358, 313, 658, 592, 256, 106, 719, 623, 685, 46, 49, 10, 646, 503, 502, 348, 337, 319, 357, 271, 242, 197, 111, 115, 47, 715, 445, 432, 377, 251, 222, 679, 635, 172, 121, 714, 630, 330, 600, 584, 499, 475, 489, 290, 291, 284, 252, 256, 632, 168, 373, 448, 439, 380, 424, 431, 371, 437, 423, 392, 389, 441, 427, 450, 442, 391, 382, 436, 428, 388, 55, 264, 105, 55, 77, 77, 553, 261, 539, 105, 539, 263, 438, 435, 716, 391, 376, 374, 444, 435, 441, 426, 428, 424, 389, 525, 62, 169, 54, 170, 113, 717, 536, 535, 536, 264, 187, 73, 170, 114, 61, 554, 373, 422, 381, 449, 378, 372, 433, 446, 221, 323, 191, 232, 490, 397, 91, 60, 468, 180, 673, 571, 174, 4, 399, 555, 505, 637, 219, 604, 549, 262, 206, 511, 291, 119, 79, 112, 301, 512, 706, 11, 599, 528, 463, 278, 374, 685, 443, 429, 613, 359, 397, 491, 70, 382, 67, 45, 265, 591]
    #Terminó, recorrido final es: [1, 12, 13, 1] con costo: 44462.30033532347
    #4031.8656690120697 s

#algoritmo('TSPLIB', 'ALL_tsp/euc2d/usa13509.tsp') #
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/vm1084.tsp') #
#algoritmo('TSPLIB', 'ALL_tsp/euc2d/vm1748.tsp') #






#algoritmo('TSPLIB','ALL_tsp/euc2d/iia5.tsp')
    #Terminó, recorrido final es: [3, 4, 2, 5, 1, 3] con costo: 5318.742722343164
    #0.0060231685638427734 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/iia10.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [9, 2, 7, 6, 2]
    #calcRecFinal: destinos: [10, 10, 9, 7, 6]
    #Terminó, recorrido final es: [4, 5, 3, 8, 1, 4] con costo: 5693.816444107245
    #0.005176067352294922 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/iia15.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [1, 2, 13, 1, 2, 14, 3, 11, 5, 10, 5]
    #calcRecFinal: destinos: [13, 3, 15, 11, 9, 15, 9, 14, 10, 12, 12]
    #Terminó, recorrido final es: [4, 6, 7, 8, 4] con costo: 7004.4387982425815
    #0.006062030792236328 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/iia20.tsp')
    #Terminó, recorrido final es: [7, 11, 19, 20, 10, 17, 16, 5, 2, 3, 14, 4, 13, 15, 18, 12, 6, 9, 1, 8, 7] con costo: 6389.3293634306465
    #0.007912874221801758 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/iia30.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [5, 16, 19, 11, 8, 2, 3, 18, 4, 15, 10, 9, 6, 5, 4, 12, 2, 13, 1, 12, 24, 8, 19, 7, 1, 30, 7]
    #calcRecFinal: destinos: [25, 24, 26, 16, 17, 6, 21, 29, 17, 25, 28, 27, 28, 15, 11, 13, 10, 30, 21, 27, 29, 20, 20, 18, 3, 9, 26]
    #Terminó, recorrido final es: [14, 23, 22, 14] con costo: 7545.265532304834
    #0.009985923767089844 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/iia50.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [29, 16, 14, 13, 3, 1, 15, 24, 2, 36, 8, 19, 7, 9, 15, 13, 11, 32, 25, 21, 43, 25, 5, 17, 1, 4, 32, 16, 8, 5, 18, 4, 23, 19, 30, 33, 2, 30, 11, 10, 20, 23, 45, 37, 22, 12, 14]
    #calcRecFinal: destinos: [48, 39, 31, 29, 40, 34, 42, 39, 3, 44, 43, 38, 44, 46, 18, 48, 27, 33, 38, 22, 46, 41, 20, 27, 21, 7, 47, 24, 9, 50, 42, 36, 31, 37, 45, 47, 40, 35, 12, 17, 50, 35, 49, 41, 34, 10, 49]
    #Terminó, recorrido final es: [6, 28, 26, 6] con costo: 6512.685496902885
    #0.01467585563659668 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/iia80.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [25, 48, 8, 37, 7, 33, 40, 52, 14, 64, 31, 10, 20, 32, 32, 11, 3, 17, 12, 21, 55, 54, 19, 7, 29, 39, 56, 4, 25, 16, 67, 35, 26, 11, 5, 27, 47, 1, 9, 40, 50, 1, 4, 13, 14, 26, 28, 27, 8, 42, 5, 44, 53, 53, 12, 15, 29, 41, 76, 17, 31, 2, 68, 6, 28, 33, 59, 65, 46, 9, 34, 63, 2, 45, 23, 15, 49]
    #calcRecFinal: destinos: [30, 72, 22, 70, 37, 47, 48, 77, 79, 79, 52, 39, 54, 51, 80, 20, 13, 61, 35, 63, 71, 69, 44, 16, 75, 45, 73, 6, 36, 70, 69, 43, 57, 67, 78, 71, 68, 34, 50, 72, 60, 30, 21, 22, 38, 41, 59, 55, 23, 62, 76, 61, 65, 66, 43, 80, 64, 57, 78, 19, 77, 73, 75, 56, 42, 58, 62, 66, 49, 60, 38, 58, 46, 10, 3, 51, 36]
    #Terminó, recorrido final es: [18, 24, 74, 18] con costo: 7363.65703380696
    #0.08325505256652832 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/iia100.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [3, 29, 8, 77, 60, 6, 81, 4, 89, 35, 92, 6, 38, 27, 7, 48, 5, 65, 41, 7, 37, 41, 42, 5, 12, 13, 29, 15, 74, 27, 49, 26, 12, 58, 51, 55, 35, 4, 2, 68, 13, 90, 10, 22, 22, 31, 17, 9, 28, 39, 34, 45, 33, 36, 62, 32, 40, 53, 56, 17, 57, 2, 46, 24, 70, 63, 53, 16, 79, 48, 18, 54, 26, 60, 14, 20, 14, 16, 28, 10, 18, 32, 19, 96, 1, 11, 31, 11, 24, 89, 42, 56, 1, 82, 87, 3, 79]
    #calcRecFinal: destinos: [85, 47, 50, 88, 83, 64, 83, 8, 92, 44, 94, 9, 59, 44, 100, 58, 84, 66, 88, 96, 84, 77, 98, 30, 15, 25, 87, 45, 75, 30, 71, 34, 99, 72, 74, 69, 59, 50, 49, 85, 57, 98, 43, 66, 65, 62, 23, 64, 72, 52, 61, 70, 93, 52, 76, 54, 43, 55, 86, 19, 73, 20, 91, 78, 99, 90, 69, 51, 82, 97, 25, 67, 61, 81, 33, 71, 93, 37, 97, 40, 73, 67, 23, 100, 46, 36, 76, 39, 86, 94, 63, 78, 91, 38, 47, 68, 75]
    #Terminó, recorrido final es: [21, 80, 95, 21] con costo: 6500.004393879755
    #0.06962013244628906 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/iia150.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [38, 24, 7, 36, 6, 3, 24, 43, 51, 101, 10, 67, 49, 101, 47, 56, 27, 15, 92, 18, 9, 48, 87, 33, 103, 2, 31, 51, 136, 17, 13, 127, 30, 5, 20, 28, 75, 81, 80, 76, 44, 21, 30, 12, 16, 45, 40, 74, 21, 34, 135, 122, 45, 25, 10, 36, 78, 98, 52, 1, 63, 80, 32, 105, 125, 26, 1, 71, 88, 66, 108, 14, 35, 96, 93, 11, 29, 23, 55, 130, 95, 59, 90, 4, 12, 5, 35, 85, 81, 70, 29, 49, 69, 3, 139, 18, 72, 42, 114, 28, 11, 20, 124, 86, 14, 25, 19, 44, 108, 107, 31, 9, 100, 46, 116, 41, 54, 23, 7, 68, 88, 55, 17, 120, 57, 50, 19, 34, 27, 84, 37, 8, 61, 15, 4, 50, 71, 48, 47, 61, 74, 64, 66, 117, 91, 6, 46]
    #calcRecFinal: destinos: [133, 147, 60, 136, 77, 26, 87, 111, 52, 128, 134, 115, 93, 114, 67, 96, 56, 137, 119, 100, 141, 150, 116, 105, 145, 123, 92, 99, 141, 118, 70, 144, 84, 149, 79, 32, 131, 95, 134, 82, 118, 110, 102, 122, 58, 76, 148, 144, 107, 140, 148, 142, 62, 65, 98, 73, 145, 138, 99, 143, 79, 138, 37, 132, 147, 103, 8, 78, 106, 121, 123, 16, 126, 132, 137, 57, 82, 91, 83, 149, 112, 65, 106, 42, 54, 130, 120, 119, 112, 97, 62, 109, 85, 43, 142, 129, 113, 53, 128, 110, 113, 63, 146, 104, 58, 59, 89, 131, 121, 146, 69, 13, 150, 97, 125, 53, 139, 40, 117, 89, 90, 111, 64, 126, 72, 75, 68, 135, 33, 102, 77, 143, 104, 129, 41, 60, 133, 140, 115, 86, 127, 73, 2, 83, 124, 109, 38]
    #Terminó, recorrido final es: [39, 94, 22, 39] con costo: 8868.12201393324
    #0.598426103591919 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/iia200.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [36, 40, 41, 150, 65, 16, 21, 29, 120, 36, 7, 18, 41, 42, 43, 62, 107, 81, 62, 54, 4, 26, 17, 65, 67, 67, 7, 126, 118, 17, 30, 129, 140, 16, 48, 86, 32, 147, 138, 147, 130, 94, 102, 29, 21, 31, 45, 43, 135, 97, 182, 2, 51, 54, 18, 31, 179, 183, 83, 5, 88, 1, 14, 39, 35, 4, 110, 117, 112, 70, 99, 90, 61, 123, 75, 34, 23, 14, 87, 53, 125, 25, 1, 47, 73, 91, 2, 74, 116, 40, 134, 74, 6, 112, 10, 148, 101, 9, 8, 11, 57, 19, 111, 72, 108, 108, 8, 69, 94, 59, 64, 102, 82, 15, 155, 34, 105, 77, 12, 60, 6, 46, 50, 73, 42, 80, 25, 76, 20, 5, 79, 48, 148, 13, 85, 167, 15, 71, 22, 28, 38, 135, 172, 98, 53, 166, 56, 11, 66, 79, 83, 52, 162, 12, 121, 113, 19, 56, 71, 37, 72, 13, 44, 33, 22, 145, 55, 101, 119, 26, 89, 161, 84, 75, 55, 170, 81, 132, 168, 92, 35, 93, 23, 61, 89, 30, 114, 46, 159, 149, 76, 121, 51, 38, 165, 80, 10]
    #calcRecFinal: destinos: [103, 180, 91, 176, 120, 183, 106, 115, 181, 50, 136, 139, 128, 126, 178, 115, 137, 82, 131, 163, 27, 87, 156, 70, 92, 100, 185, 191, 177, 138, 118, 150, 142, 188, 58, 141, 95, 170, 165, 174, 141, 107, 129, 131, 103, 37, 168, 160, 140, 160, 198, 49, 64, 57, 85, 133, 186, 188, 106, 32, 171, 66, 58, 166, 77, 142, 172, 130, 182, 88, 122, 173, 173, 169, 187, 198, 123, 63, 181, 193, 153, 197, 20, 200, 98, 128, 184, 152, 185, 139, 146, 93, 113, 157, 47, 179, 155, 146, 78, 122, 163, 125, 190, 78, 158, 133, 86, 132, 137, 193, 117, 189, 114, 194, 196, 157, 109, 109, 68, 154, 127, 119, 156, 192, 191, 104, 69, 187, 143, 95, 145, 63, 186, 111, 159, 195, 175, 136, 96, 154, 180, 197, 184, 192, 59, 178, 176, 99, 158, 164, 194, 68, 196, 52, 143, 127, 153, 104, 116, 200, 84, 190, 96, 97, 44, 164, 60, 162, 124, 33, 199, 199, 167, 189, 149, 174, 144, 134, 175, 100, 105, 152, 169, 90, 161, 177, 144, 124, 49, 39, 9, 195, 45, 110, 171, 27, 28]
    #Terminó, recorrido final es: [3, 151, 24, 3] con costo: 9177.72650782031
    #2.385990858078003 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/iia500.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [111, 230, 223, 318, 114, 435, 437, 129, 189, 199, 86, 69, 69, 268, 297, 309, 83, 57, 180, 186, 142, 311, 149, 111, 83, 144, 136, 310, 388, 82, 176, 68, 178, 66, 208, 178, 72, 110, 152, 172, 38, 262, 71, 233, 95, 95, 27, 374, 32, 220, 40, 229, 92, 31, 284, 246, 106, 193, 215, 180, 79, 96, 76, 50, 27, 115, 71, 80, 303, 37, 39, 158, 58, 209, 390, 228, 110, 52, 175, 166, 160, 202, 34, 22, 280, 130, 222, 276, 177, 25, 61, 211, 44, 202, 408, 15, 65, 142, 41, 59, 203, 12, 43, 30, 138, 206, 34, 80, 44, 203, 19, 397, 246, 179, 187, 157, 213, 452, 217, 454, 54, 107, 343, 383, 32, 337, 138, 82, 316, 234, 249, 294, 64, 65, 31, 37, 125, 62, 134, 173, 298, 241, 196, 93, 353, 139, 168, 132, 136, 81, 77, 317, 54, 68, 147, 21, 137, 269, 173, 123, 268, 440, 217, 4, 478, 191, 78, 250, 47, 4, 182, 295, 159, 75, 195, 100, 259, 194, 91, 245, 414, 11, 133, 99, 59, 261, 116, 23, 170, 70, 22, 274, 92, 172, 216, 6, 365, 229, 346, 344, 73, 116, 164, 243, 20, 242, 137, 232, 144, 79, 370, 45, 126, 135, 30, 228, 100, 60, 14, 298, 253, 7, 55, 52, 411, 212, 455, 277, 90, 331, 9, 187, 126, 38, 124, 125, 35, 271, 206, 19, 176, 378, 25, 55, 23, 86, 156, 384, 423, 177, 133, 236, 219, 14, 459, 104, 342, 96, 140, 379, 438, 425, 304, 357, 128, 140, 238, 42, 76, 62, 6, 28, 143, 401, 148, 40, 132, 208, 97, 64, 219, 375, 106, 39, 107, 241, 152, 48, 45, 194, 195, 299, 12, 170, 403, 157, 184, 250, 253, 239, 183, 28, 17, 302, 104, 201, 99, 307, 336, 85, 281, 103, 354, 363, 9, 411, 115, 3, 78, 167, 24, 10, 164, 91, 156, 63, 1, 168, 325, 232, 263, 189, 74, 53, 75, 227, 435, 46, 303, 119, 314, 357, 167, 66, 16, 121, 94, 117, 102, 119, 302, 24, 336, 103, 21, 210, 48, 342, 158, 154, 255, 57, 11, 161, 7, 26, 17, 117, 42, 130, 401, 269, 114, 46, 255, 230, 97, 358, 209, 16, 63, 278, 483, 72, 389, 304, 299, 497, 127, 447, 498, 456, 221, 227, 267, 482, 148]
    #calcRecFinal: destinos: [396, 414, 262, 381, 395, 489, 489, 265, 312, 395, 310, 223, 372, 472, 312, 374, 405, 60, 321, 272, 166, 327, 499, 113, 155, 388, 149, 432, 491, 290, 434, 446, 216, 323, 276, 351, 129, 318, 371, 379, 470, 372, 472, 469, 494, 154, 47, 397, 113, 439, 139, 470, 387, 488, 413, 368, 361, 371, 329, 183, 279, 356, 280, 199, 322, 341, 345, 495, 405, 424, 101, 448, 459, 261, 473, 361, 381, 317, 376, 334, 383, 311, 448, 214, 431, 492, 450, 287, 300, 329, 408, 463, 413, 375, 424, 307, 108, 334, 101, 257, 466, 15, 493, 135, 221, 323, 290, 259, 443, 463, 165, 495, 346, 279, 482, 465, 243, 464, 469, 465, 385, 301, 369, 387, 196, 450, 330, 296, 477, 410, 322, 351, 191, 476, 428, 61, 376, 344, 362, 211, 461, 274, 201, 338, 452, 186, 369, 242, 478, 441, 476, 486, 247, 356, 445, 123, 151, 428, 370, 131, 345, 481, 233, 287, 499, 427, 362, 490, 249, 256, 500, 324, 301, 108, 410, 440, 309, 220, 147, 248, 493, 248, 500, 278, 90, 454, 145, 102, 484, 214, 70, 480, 160, 267, 296, 377, 430, 315, 368, 421, 492, 121, 212, 434, 409, 367, 378, 277, 439, 179, 447, 407, 429, 422, 422, 449, 161, 94, 442, 423, 295, 207, 338, 486, 453, 457, 497, 326, 257, 481, 143, 490, 263, 315, 236, 175, 120, 443, 477, 364, 491, 455, 215, 93, 352, 432, 337, 398, 461, 285, 325, 294, 400, 363, 498, 281, 458, 446, 270, 393, 458, 453, 407, 426, 377, 210, 382, 238, 431, 421, 128, 58, 485, 484, 358, 273, 367, 456, 127, 74, 314, 441, 449, 41, 159, 480, 193, 305, 120, 273, 234, 430, 347, 352, 426, 466, 473, 258, 324, 305, 258, 436, 20, 354, 384, 396, 330, 347, 353, 213, 398, 247, 366, 442, 485, 425, 291, 350, 134, 360, 436, 197, 457, 445, 222, 479, 402, 343, 360, 326, 429, 297, 483, 146, 77, 327, 437, 390, 479, 365, 400, 403, 341, 272, 165, 145, 240, 350, 284, 339, 366, 182, 464, 385, 131, 270, 239, 438, 207, 494, 300, 389, 245, 331, 393, 43, 409, 3, 382, 73, 81, 488, 50, 184, 285, 26, 1, 291, 146, 364, 155, 197, 427, 265, 240, 35, 339, 151, 402, 321, 85, 256, 10, 271, 124, 53, 316]
    #Terminó, recorrido final es: [235, 417, 8, 349, 188, 392, 286, 13, 391, 416, 386, 283, 355, 451, 237, 122, 87, 328, 467, 153, 226, 112, 462, 51, 335, 18, 254, 49, 204, 89, 174, 288, 308, 185, 412, 84, 98, 33, 348, 266, 169, 313, 468, 332, 380, 109, 163, 496, 394, 433, 487, 67, 293, 460, 264, 474, 105, 192, 224, 260, 141, 225, 340, 218, 319, 320, 419, 150, 36, 359, 373, 205, 399, 471, 190, 5, 475, 200, 282, 420, 415, 181, 404, 289, 244, 2, 292, 56, 162, 275, 444, 418, 231, 118, 252, 29, 333, 88, 306, 198, 251, 406, 171, 235] con costo: 9471.887238758145
    #1279.0391900539398 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/iia750.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [13, 356, 184, 370, 255, 530, 37, 453, 388, 508, 158, 249, 211, 388, 190, 205, 290, 290, 57, 43, 287, 87, 26, 567, 188, 500, 541, 280, 475, 225, 282, 598, 185, 357, 69, 133, 393, 522, 98, 296, 418, 26, 272, 11, 400, 19, 83, 416, 229, 378, 578, 6, 453, 321, 83, 41, 470, 323, 369, 381, 13, 192, 37, 429, 160, 278, 69, 323, 240, 171, 259, 513, 42, 132, 387, 47, 56, 344, 267, 115, 449, 472, 161, 514, 293, 48, 18, 55, 111, 114, 377, 315, 104, 134, 245, 358, 9, 167, 222, 100, 192, 506, 60, 214, 376, 98, 215, 91, 94, 352, 155, 227, 297, 238, 114, 320, 236, 309, 458, 99, 167, 322, 308, 129, 246, 36, 469, 385, 160, 10, 274, 35, 10, 173, 197, 331, 118, 435, 198, 241, 29, 447, 311, 256, 243, 214, 207, 343, 195, 267, 347, 52, 38, 65, 277, 338, 282, 637, 96, 258, 374, 195, 117, 165, 54, 461, 465, 89, 54, 42, 28, 404, 92, 141, 372, 174, 564, 306, 683, 86, 1, 308, 202, 437, 302, 335, 621, 168, 512, 203, 526, 86, 44, 298, 62, 651, 93, 305, 298, 16, 272, 120, 63, 116, 120, 61, 119, 219, 650, 14, 127, 217, 22, 4, 203, 1, 46, 108, 47, 102, 313, 138, 84, 150, 409, 70, 6, 231, 695, 246, 465, 128, 408, 413, 41, 310, 451, 76, 44, 211, 299, 175, 138, 158, 148, 19, 238, 142, 112, 478, 434, 359, 233, 212, 11, 236, 331, 455, 440, 137, 199, 159, 540, 485, 156, 291, 390, 225, 234, 317, 40, 15, 72, 166, 287, 125, 299, 14, 149, 124, 273, 302, 334, 360, 76, 48, 693, 406, 228, 148, 310, 100, 152, 300, 474, 16, 20, 207, 174, 450, 394, 139, 60, 319, 276, 363, 70, 441, 177, 122, 78, 548, 636, 52, 420, 315, 683, 254, 4, 68, 95, 188, 612, 223, 387, 132, 259, 3, 324, 22, 64, 562, 336, 75, 38, 153, 235, 655, 494, 442, 332, 179, 55, 20, 90, 560, 229, 88, 32, 119, 49, 544, 319, 40, 366, 640, 34, 220, 124, 35, 71, 185, 97, 49, 260, 135, 95, 456, 354, 633, 480, 74, 144, 103, 260, 480, 515, 23, 655, 233, 275, 73, 511, 157, 53, 251, 130, 99, 126, 221, 399, 50, 133, 586, 183, 470, 32, 244, 159, 7, 5, 205, 145, 208, 15, 263, 341, 435, 105, 85, 647, 402, 143, 610, 110, 131, 415, 59, 111, 252, 438, 97, 549, 373, 516, 115, 268, 361, 629, 175, 105, 492, 527, 136, 67, 107, 245, 359, 162, 428, 528, 393, 39, 423, 285, 253, 154, 163, 379, 34, 84, 72, 103, 268, 555, 417, 316, 3, 165, 145, 297, 9, 365, 436, 279, 582, 221, 575, 597, 431, 333, 570, 281, 326, 56, 102, 142, 410, 713, 153, 345, 304, 546, 193, 353, 89, 239, 523, 638, 27, 146, 79, 147, 544, 96, 533, 362, 149, 520, 421, 305, 365, 258, 68, 12, 206, 463, 191, 178, 646, 379, 27, 53, 434, 314, 499, 17, 588, 218, 232, 194, 232, 21, 24, 191, 101, 162, 637, 168, 263, 392, 563, 320, 334, 385, 582, 156, 92, 134, 218, 25, 725, 101, 130, 448, 12, 121, 295, 289, 107, 537, 476, 46, 201, 375, 572, 140, 5, 262, 324, 492, 568, 234, 400, 493, 532, 653, 431, 501, 437, 266, 460, 471, 489, 61, 164, 104, 51, 108, 2, 36, 147, 17, 583, 202, 125, 343, 364, 222, 201, 75, 131, 265, 216, 31, 516, 396, 269, 31, 242, 479, 650, 21, 171, 80, 18, 335, 394, 341, 66, 187, 180, 502, 356, 327, 641, 355, 289, 65, 30, 593, 157, 438, 71, 257, 250, 180, 549, 227, 186, 77, 210, 183, 94, 467, 224, 139, 373, 186, 318, 545, 386, 425, 407, 184, 150, 28, 30, 408, 33, 155, 39, 73, 455, 58, 616, 144, 247, 166, 264, 325, 112, 342, 340, 360, 347, 33, 303, 361, 189, 25, 384, 286, 253, 151, 196, 248, 113, 255, 230, 292, 484, 45, 405, 249, 294, 50, 220, 223, 507, 539, 591, 352, 251, 63, 432, 24, 476, 521, 606, 428, 402, 117, 270, 57, 392, 444, 64, 671, 348, 490, 371, 240, 209, 314, 178, 668, 29, 488, 110, 694, 154, 656, 395, 730, 182, 2, 609, 328, 204, 109, 170, 743, 536, 151, 421, 286, 630, 325, 329, 91, 296, 283, 113, 194, 135]
    #calcRecFinal: destinos: [620, 550, 511, 439, 280, 743, 520, 524, 645, 541, 740, 442, 482, 479, 632, 567, 510, 384, 717, 284, 536, 505, 106, 667, 274, 731, 744, 684, 482, 357, 579, 669, 391, 496, 141, 576, 411, 745, 550, 508, 723, 507, 346, 523, 670, 517, 161, 595, 495, 589, 672, 445, 491, 578, 598, 416, 589, 608, 514, 382, 322, 608, 339, 572, 339, 351, 215, 364, 412, 464, 410, 607, 716, 556, 595, 619, 58, 613, 584, 530, 670, 738, 209, 622, 674, 380, 45, 382, 247, 346, 712, 749, 177, 250, 309, 378, 719, 407, 718, 350, 711, 592, 88, 724, 457, 584, 509, 641, 173, 464, 446, 566, 723, 717, 600, 651, 629, 367, 477, 122, 557, 620, 487, 424, 409, 599, 553, 425, 411, 468, 677, 74, 248, 213, 726, 649, 413, 547, 576, 561, 243, 599, 348, 566, 354, 338, 242, 592, 691, 283, 461, 693, 176, 477, 454, 412, 441, 708, 454, 587, 719, 602, 607, 276, 706, 569, 703, 613, 123, 432, 635, 430, 137, 509, 534, 691, 618, 386, 713, 303, 312, 563, 389, 491, 336, 681, 696, 273, 665, 718, 565, 705, 500, 678, 353, 701, 443, 457, 498, 342, 600, 515, 547, 370, 377, 241, 128, 660, 657, 332, 405, 617, 169, 696, 518, 695, 254, 571, 127, 301, 345, 654, 488, 533, 680, 729, 78, 741, 735, 666, 501, 710, 485, 521, 397, 497, 709, 328, 731, 237, 638, 734, 291, 591, 399, 681, 746, 329, 739, 496, 662, 570, 733, 571, 219, 635, 444, 459, 727, 200, 330, 217, 556, 707, 538, 615, 748, 478, 517, 321, 396, 689, 193, 537, 389, 621, 398, 534, 625, 502, 585, 704, 727, 390, 460, 62, 726, 728, 657, 565, 471, 358, 710, 333, 634, 690, 23, 472, 279, 684, 686, 199, 535, 349, 368, 749, 164, 579, 505, 271, 445, 624, 697, 197, 631, 363, 732, 627, 736, 546, 237, 677, 720, 337, 397, 540, 611, 404, 528, 181, 281, 596, 351, 531, 531, 742, 401, 666, 617, 745, 372, 306, 735, 121, 374, 699, 414, 535, 292, 152, 469, 588, 552, 642, 519, 714, 190, 519, 594, 172, 702, 429, 462, 553, 737, 427, 475, 462, 590, 664, 498, 172, 483, 665, 614, 678, 712, 628, 680, 725, 316, 295, 562, 525, 181, 327, 667, 271, 543, 700, 526, 722, 198, 722, 383, 503, 583, 494, 244, 66, 426, 601, 658, 675, 448, 555, 631, 574, 616, 532, 697, 739, 261, 698, 212, 606, 433, 136, 694, 456, 605, 252, 714, 664, 628, 337, 350, 698, 744, 436, 715, 624, 643, 277, 206, 266, 452, 688, 204, 654, 577, 473, 143, 525, 383, 747, 235, 625, 406, 59, 614, 486, 512, 605, 741, 747, 401, 430, 368, 648, 418, 90, 474, 734, 602, 659, 265, 639, 672, 647, 451, 688, 427, 558, 301, 196, 349, 609, 732, 499, 483, 604, 623, 486, 380, 344, 466, 529, 709, 612, 560, 543, 239, 673, 126, 699, 690, 163, 630, 652, 376, 634, 495, 623, 487, 426, 590, 208, 452, 661, 728, 662, 169, 552, 604, 742, 403, 673, 269, 484, 490, 504, 226, 391, 340, 493, 369, 704, 585, 559, 443, 687, 701, 720, 564, 593, 230, 200, 419, 326, 398, 733, 182, 601, 689, 687, 311, 371, 575, 580, 730, 686, 627, 264, 721, 674, 304, 67, 682, 577, 548, 644, 559, 449, 682, 702, 676, 636, 703, 524, 580, 529, 489, 615, 561, 729, 648, 109, 573, 542, 626, 216, 318, 679, 422, 736, 506, 711, 503, 554, 176, 284, 700, 466, 481, 660, 642, 558, 294, 527, 645, 656, 77, 581, 643, 129, 721, 587, 420, 257, 518, 626, 594, 632, 568, 679, 542, 639, 458, 80, 659, 423, 706, 85, 668, 395, 447, 640, 256, 676, 226, 685, 285, 213, 740, 419, 330, 633, 653, 403, 663, 671, 618, 557, 439, 574, 307, 307, 707, 685, 270, 261, 82, 692, 381, 715, 313, 415, 646, 554, 459, 170, 362, 675, 748, 569, 210, 705, 610, 228, 652, 510, 539, 417, 658, 603, 468, 146, 450, 422, 597, 504, 424, 440, 522, 481, 586, 366, 288, 106, 278, 467, 581, 644, 312, 716, 116, 317, 118, 51, 692, 545, 513, 446, 746, 93, 649, 497, 179, 293, 375, 82, 724, 669, 140, 367, 7, 463, 737, 573, 433, 275, 189, 224, 661, 262, 355, 611, 231, 622, 43, 663, 288, 538, 87, 300, 708, 473, 123, 619, 414, 738, 79, 603, 596, 187]
    #Terminó, recorrido final es: [8, 551, 750, 81, 8] con costo: 7499.506089513672
    #4547.936460971832 s
#algoritmo('TSPLIB','ALL_tsp/euc2d/iia1000.tsp')
    #Se tiene mas de un lazo
    #calcRecFinal: origenes: [568, 650, 833, 217, 228, 499, 739, 725, 429, 222, 29, 29, 596, 313, 5, 166, 234, 165, 147, 677, 120, 165, 216, 504, 259, 425, 547, 659, 44, 696, 768, 263, 736, 776, 113, 228, 446, 143, 567, 328, 231, 54, 38, 533, 332, 173, 229, 336, 595, 817, 20, 437, 238, 156, 177, 691, 127, 160, 250, 494, 126, 947, 244, 687, 550, 285, 16, 429, 205, 322, 118, 3, 465, 191, 563, 458, 11, 87, 196, 64, 144, 582, 502, 868, 321, 73, 870, 408, 52, 405, 136, 229, 483, 141, 232, 550, 915, 193, 36, 223, 322, 91, 283, 395, 386, 57, 87, 753, 11, 217, 521, 60, 67, 954, 61, 416, 172, 203, 342, 374, 299, 453, 187, 609, 421, 195, 137, 280, 457, 471, 461, 738, 547, 68, 68, 705, 131, 888, 427, 81, 51, 722, 4, 438, 273, 265, 155, 57, 662, 454, 422, 182, 71, 725, 89, 226, 183, 351, 473, 644, 265, 548, 789, 191, 609, 247, 81, 601, 661, 342, 600, 249, 225, 338, 600, 410, 8, 152, 186, 98, 528, 5, 495, 644, 108, 488, 350, 51, 491, 225, 724, 369, 399, 425, 258, 95, 539, 742, 474, 109, 662, 390, 493, 341, 281, 440, 512, 371, 728, 24, 269, 159, 27, 886, 129, 39, 774, 785, 159, 260, 413, 209, 66, 198, 807, 157, 279, 769, 198, 194, 6, 364, 418, 649, 866, 145, 136, 115, 77, 304, 532, 764, 23, 212, 690, 455, 64, 414, 219, 761, 179, 548, 243, 375, 171, 93, 597, 35, 677, 634, 544, 183, 101, 487, 811, 143, 594, 373, 653, 78, 479, 16, 237, 102, 414, 687, 305, 208, 314, 76, 109, 49, 170, 25, 105, 316, 505, 221, 572, 259, 178, 21, 107, 125, 625, 881, 297, 624, 24, 520, 43, 771, 278, 511, 619, 144, 763, 646, 589, 92, 883, 8, 450, 88, 577, 103, 491, 706, 434, 127, 314, 105, 184, 374, 567, 476, 125, 445, 137, 19, 32, 834, 361, 249, 557, 73, 631, 409, 236, 148, 407, 595, 128, 419, 153, 446, 293, 368, 712, 848, 326, 536, 260, 420, 150, 98, 67, 55, 340, 448, 463, 310, 33, 510, 58, 914, 104, 154, 132, 112, 658, 89, 146, 543, 363, 96, 492, 111, 633, 70, 112, 13, 424, 383, 286, 587, 69, 182, 308, 893, 199, 402, 476, 403, 7, 296, 377, 752, 74, 264, 841, 248, 434, 93, 46, 242, 630, 415, 208, 682, 6, 304, 513, 442, 506, 396, 233, 310, 200, 55, 18, 72, 149, 415, 441, 142, 9, 369, 323, 324, 452, 53, 308, 100, 53, 951, 15, 59, 430, 380, 261, 303, 614, 216, 149, 100, 718, 205, 206, 272, 330, 616, 467, 190, 266, 486, 458, 464, 712, 65, 391, 200, 290, 487, 160, 48, 242, 835, 402, 69, 679, 9, 306, 142, 387, 346, 274, 72, 686, 531, 391, 345, 375, 669, 70, 227, 185, 382, 526, 608, 262, 637, 349, 240, 371, 91, 433, 281, 803, 445, 357, 511, 172, 674, 192, 331, 428, 300, 215, 350, 44, 456, 46, 338, 406, 504, 804, 674, 359, 793, 740, 508, 344, 185, 336, 269, 71, 177, 396, 83, 151, 230, 99, 586, 151, 673, 27, 666, 284, 406, 224, 90, 611, 717, 636, 654, 60, 135, 413, 206, 39, 167, 78, 522, 250, 99, 412, 420, 538, 503, 34, 207, 101, 31, 386, 647, 43, 282, 389, 517, 697, 122, 222, 368, 397, 467, 385, 189, 307, 218, 777, 605, 36, 2, 150, 174, 289, 298, 4, 224, 780, 138, 641, 363, 162, 570, 255, 578, 498, 207, 169, 356, 439, 749, 893, 264, 561, 114, 293, 122, 61, 546, 79, 535, 501, 45, 335, 604, 1, 129, 152, 499, 2, 83, 116, 709, 443, 261, 335, 591, 654, 220, 838, 333, 219, 480, 365, 321, 49, 710, 451, 715, 40, 168, 976, 578, 28, 354, 300, 398, 346, 337, 194, 306, 153, 339, 401, 359, 606, 162, 424, 701, 715, 574, 193, 26, 435, 106, 50, 50, 85, 34, 7, 201, 134, 45, 110, 12, 47, 389, 590, 248, 400, 788, 82, 124, 218, 719, 232, 297, 90, 275, 270, 723, 507, 580, 164, 555, 360, 107, 419, 245, 132, 118, 748, 79, 32, 583, 244, 35, 156, 631, 22, 119, 426, 235, 477, 241, 178, 475, 450, 408, 163, 449, 85, 164, 282, 75, 121, 17, 146, 294, 256, 267, 340, 528, 835, 241, 362, 17, 239, 84, 212, 74, 309, 443, 220, 353, 58, 388, 683, 430, 77, 175, 42, 15, 576, 133, 197, 130, 320, 877, 384, 47, 86, 23, 858, 271, 352, 920, 291, 317, 28, 209, 158, 256, 214, 404, 214, 278, 514, 161, 119, 354, 914, 711, 201, 179, 301, 523, 253, 3, 292, 167, 530, 12, 75, 462, 448, 204, 397, 459, 572, 364, 460, 252, 135, 188, 117, 175, 351, 546, 56, 708, 832, 510, 813, 559, 30, 104, 14, 721, 169, 25, 22, 131, 215, 468, 440, 102, 506, 288, 418, 10, 591, 253, 120, 80, 852, 432, 628, 171, 86, 246, 94, 348, 302, 108, 574, 603, 376, 251, 592, 296, 284, 503, 376, 380, 702, 139, 41, 239, 30, 37, 381, 385, 447, 452, 685, 552, 111, 383, 702, 181, 288, 365, 678, 784, 496, 48, 190, 41, 245, 37, 170, 638, 423, 166, 10, 291, 339, 759, 676, 80, 130, 115, 693, 488, 334, 334, 252, 466, 598, 428, 956, 822, 21, 26, 84, 399, 279, 40, 586, 676, 370, 565, 718, 96, 453, 213, 444, 176, 343, 19, 797, 566, 173, 521, 726, 988, 273, 934, 199, 262, 14, 140, 292, 33, 398, 918, 508, 148, 97, 315, 480, 770, 394, 285, 266, 901, 211, 343, 635, 800, 251, 912, 899, 944, 575, 782, 527, 517, 192, 484, 485, 603, 670, 579, 874, 97, 42, 133, 52, 307, 977, 56, 404, 95, 985, 812, 607, 549, 196, 959, 348, 855, 295, 571, 113, 895, 168, 771, 319, 967, 13, 630, 864]
    #calcRecFinal: destinos: [827, 862, 918, 720, 973, 596, 744, 888, 436, 659, 797, 904, 829, 935, 851, 341, 657, 923, 765, 812, 657, 366, 358, 724, 358, 805, 747, 867, 502, 872, 844, 949, 965, 851, 828, 753, 553, 850, 650, 864, 884, 791, 744, 663, 745, 237, 274, 923, 768, 860, 760, 605, 756, 694, 187, 896, 545, 437, 830, 821, 570, 967, 902, 743, 730, 319, 489, 438, 268, 472, 522, 939, 896, 952, 816, 728, 54, 801, 268, 614, 231, 846, 828, 966, 706, 295, 939, 767, 92, 839, 913, 634, 991, 347, 283, 750, 925, 197, 180, 263, 449, 701, 716, 474, 815, 648, 226, 885, 801, 704, 756, 180, 940, 961, 277, 518, 353, 817, 540, 898, 584, 623, 556, 686, 787, 760, 750, 830, 816, 961, 777, 927, 743, 176, 492, 714, 661, 992, 497, 333, 800, 767, 849, 988, 347, 423, 694, 94, 741, 651, 624, 372, 128, 992, 227, 791, 632, 455, 553, 766, 836, 713, 856, 211, 955, 558, 705, 620, 703, 729, 788, 255, 794, 377, 696, 1000, 672, 276, 962, 500, 623, 422, 843, 655, 558, 755, 372, 379, 533, 552, 827, 955, 501, 742, 379, 513, 787, 805, 842, 462, 855, 854, 755, 489, 515, 945, 660, 515, 935, 745, 356, 942, 464, 936, 672, 810, 818, 938, 213, 590, 483, 520, 277, 444, 887, 329, 944, 981, 707, 627, 59, 625, 495, 815, 871, 519, 995, 947, 689, 998, 886, 839, 203, 409, 783, 620, 922, 996, 525, 785, 352, 782, 665, 697, 984, 664, 741, 223, 965, 990, 945, 588, 652, 692, 900, 671, 938, 529, 849, 88, 772, 926, 690, 887, 640, 747, 757, 748, 573, 195, 762, 617, 243, 781, 470, 384, 704, 392, 589, 410, 564, 551, 733, 607, 907, 993, 622, 881, 332, 773, 581, 932, 769, 897, 722, 538, 772, 984, 869, 691, 982, 38, 937, 927, 937, 221, 663, 746, 729, 726, 943, 447, 833, 526, 862, 643, 648, 529, 536, 834, 542, 873, 576, 534, 957, 421, 826, 667, 781, 551, 891, 766, 367, 979, 316, 473, 573, 842, 877, 974, 905, 757, 645, 569, 959, 189, 126, 140, 933, 997, 890, 416, 764, 865, 629, 948, 863, 326, 894, 973, 919, 875, 317, 909, 780, 286, 925, 257, 732, 675, 653, 666, 613, 911, 799, 699, 460, 999, 312, 986, 626, 813, 484, 796, 779, 656, 889, 954, 727, 313, 860, 328, 540, 106, 158, 628, 681, 497, 819, 936, 382, 456, 924, 814, 857, 850, 525, 518, 931, 411, 298, 302, 154, 427, 733, 641, 62, 689, 958, 417, 633, 121, 571, 287, 157, 978, 355, 357, 700, 930, 400, 324, 922, 626, 905, 617, 838, 765, 627, 466, 810, 802, 950, 731, 636, 656, 929, 820, 928, 861, 879, 685, 411, 561, 932, 621, 639, 848, 987, 554, 826, 471, 496, 806, 454, 392, 990, 880, 916, 802, 564, 560, 758, 857, 512, 875, 582, 618, 898, 933, 345, 930, 629, 731, 593, 479, 792, 593, 892, 989, 618, 774, 390, 871, 789, 635, 895, 929, 978, 709, 783, 514, 585, 889, 698, 568, 943, 866, 719, 953, 993, 902, 670, 762, 651, 853, 367, 556, 612, 246, 996, 290, 994, 793, 381, 806, 820, 876, 941, 598, 678, 532, 642, 837, 920, 708, 123, 451, 991, 681, 330, 344, 738, 575, 280, 956, 524, 891, 884, 892, 362, 579, 331, 325, 649, 958, 539, 695, 856, 869, 758, 257, 867, 395, 693, 713, 968, 500, 387, 814, 982, 821, 123, 463, 555, 779, 960, 770, 885, 964, 977, 868, 673, 519, 403, 940, 534, 587, 798, 971, 825, 878, 824, 941, 976, 985, 692, 311, 804, 963, 66, 597, 439, 751, 655, 906, 809, 917, 299, 739, 688, 829, 890, 919, 254, 999, 776, 668, 969, 917, 908, 792, 876, 431, 233, 549, 516, 527, 287, 737, 599, 778, 457, 972, 986, 699, 735, 643, 557, 972, 926, 775, 957, 435, 680, 481, 980, 717, 716, 796, 874, 966, 975, 808, 632, 784, 778, 664, 861, 65, 124, 494, 174, 459, 584, 181, 210, 355, 611, 490, 645, 989, 668, 872, 968, 832, 370, 837, 606, 361, 682, 736, 790, 831, 924, 759, 858, 647, 481, 441, 949, 337, 675, 202, 819, 163, 272, 818, 823, 186, 581, 679, 913, 289, 746, 498, 859, 412, 879, 577, 475, 910, 824, 472, 594, 318, 388, 311, 329, 325, 254, 721, 531, 946, 608, 754, 974, 524, 621, 31, 562, 859, 667, 751, 478, 734, 433, 854, 349, 695, 808, 602, 138, 301, 276, 730, 622, 583, 323, 235, 900, 928, 680, 642, 210, 841, 946, 482, 845, 970, 537, 485, 530, 773, 585, 616, 897, 894, 912, 981, 998, 188, 960, 393, 997, 882, 469, 315, 366, 840, 723, 870, 853, 840, 735, 312, 114, 763, 948, 507, 710, 469, 795, 907, 554, 271, 599, 971, 852, 601, 431, 934, 407, 908, 899, 863, 987, 562, 478, 865, 505, 901, 516, 236, 318, 703, 951, 537, 544, 807, 669, 786, 843, 684, 592, 831, 234, 786, 983, 541, 639, 646, 110, 658, 610, 962, 880, 247, 683, 727, 844, 373, 604, 486, 749, 803, 847, 637, 711, 752, 303, 559, 309, 969, 640, 1000, 470, 732, 931, 794, 963, 394, 882, 906, 983, 825, 964, 790, 975, 103, 240, 417, 775, 809, 665, 915, 836, 909, 613, 468, 360, 910, 822, 117, 798, 545, 737, 493, 921, 566, 482, 542, 698, 560, 994, 903, 378, 270, 477, 565, 327, 563, 953, 903, 442, 980, 811, 799, 754, 942, 707, 740, 602, 873, 904, 921, 541, 238, 916, 436, 141, 76, 267, 671, 720, 612, 878, 405, 734, 184, 823, 378, 1, 845, 883, 18, 911, 401, 970, 294, 952, 204, 652, 258, 139, 580, 979, 327, 202, 950, 426, 795, 490, 393, 116, 535, 523, 161, 684, 134, 688, 847, 465, 320, 145, 569, 660, 700, 995, 275, 610, 461, 147, 588, 62, 20, 155, 305, 432, 230, 638, 543, 619, 846, 714, 82, 761]
    #Terminó, recorrido final es: [63, 509, 615, 63] con costo: 7487.193632745707
    #39489.38262796402 s
"""algoritmo('TSPLIB','ALL_tsp/euc2d/iia1500.tsp')
algoritmo('TSPLIB','ALL_tsp/euc2d/iia2000.tsp') """


In [104]:
## crear ciudades dentro de una circunferencia
import numpy as np
import matplotlib.pyplot as plt
import math
import io
from adjustText import adjust_text
from scipy import interpolate

#print(math.sin(math.pi))

#ciudades_a_armar= [5,10,15,20,30,50,80,100,150,200,500,750,1000,1500,2000]

#ciudades_a_armar= [250,260,270,280,290,300,310,320,330,340,350,360,370,380,390,400,410,420,430,440,450,460,470,480,490,500]
ciudades_a_armar=[20]
RADIO_CIRC = 1000

primera_ciudad = True
for a in ciudades_a_armar:
    cant_ciudades = a
    fi = 0

    np.random.seed(a)
    fi = np.random.uniform(0,2*np.pi, cant_ciudades)
    radio = np.array([RADIO_CIRC for a in range(cant_ciudades)])

    ciudades_x = RADIO_CIRC + radio * np.cos(fi)
    ciudades_y = RADIO_CIRC + radio * np.sin(fi)
    ciudades = np.array([c+1 for c in range(a)])

    """with open("ALL_TSP/euc2d/iia{}.tsp".format(a), 'w') as f:
        f.write("NAME : iia{}\n".format(a))
        f.write("COMMENT : {} ciudades en circunferencia radio {}\n".format(a, RADIO_CIRC))
        f.write("TYPE : TSP\n")
        f.write("DIMENSION: {}\n".format(a))
        f.write("EDGE_WEIGHT_TYPE : EUC_2D\n")
        f.write("NODE_COORD_SECTION\n")
        ciudad = 1
        for x,y in zip(ciudades_x,ciudades_y):
            f.write("{} {} {}\n".format(ciudad,x,y))
            ciudad += 1
        f.write("EOF\n")
        f.close()"""

    plt.figure(figsize=(30,25))
    plt.scatter(ciudades_x,ciudades_y, s=500)

    texts = []
    for x, y, s in zip(ciudades_x, ciudades_y, ciudades):
        texts.append(plt.text(x, y, s, size=50))

    plt.xlabel("Distancia")
    plt.ylabel("Distancia")
    plt.title("Ciudades")
    adjust_text(texts, only_move={'points':'y', 'texts':'y'}, arrowprops=dict(arrowstyle="->", color='r', lw=0.5))
    if (primera_ciudad):
        plt.show()
        #pass

    #plt.savefig("ALL_TSP/euc2d/iia{}_2.png".format(a))
    #plt.close()


        
    #b=0
    #for xy in zip(ciudades_x, ciudades_y):
    #    b += 1
    #    plt.annotate("{}".format(b), xy=xy, fontsize=10)

    #p1 = plt.plot(ciudades_x,ciudades_y,color="black", alpha=0.5)
        